In [ ]:
!pip install tensorflow

In [ ]:
# @title 1. Imports and Setup
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, RepeatVector, TimeDistributed, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Set seeds for reproducibility (Crucial for scientific reporting)
np.random.seed(42)
import tensorflow as tf
tf.random.set_seed(42)

print("Libraries loaded successfully.")

In [ ]:
# @title 2. Load Data and Initial Preprocessing
import pandas as pd
import os

# Define the data directory relative to this script
data_dir = './data'
# Update the filename to the .zip version
file_name = 'asv_interpretability_dataset_modified.zip'
file_path = os.path.join(data_dir, file_name)

# Check if file exists
if not os.path.exists(file_path):
    raise FileNotFoundError(f"Data file not found at {file_path}. Please ensure the 'data' folder contains the zipped dataset.")

# Load the data directly from the zip file
df = pd.read_csv(file_path, dtype={'PatientID': str})
print(f"Successfully loaded data from {file_path}")

# --- Helper: Handle NeutrophilCount with '<0.1' values ---
def parse_neutrophil(value):
    try:
        return float(value)
    except:
        if isinstance(value, str) and "<" in value:
            threshold = float(value.replace("<", "").strip())
            return threshold / 2
        return np.nan

# --- Helper: Create Proxy Labels (Clinical Dysbiosis) ---
def label_dysbiosis(row):
    # Proxy definition: High Temp + Low Neutrophils + Liquid Stool
    is_temp_abnormal = row['MaxTemperature'] > 38.0
    is_neutro_low = row['NeutrophilCount'] < 500
    is_consistency_liquid = row.get('Consistency_liquid', 0) == 1
    return int(is_temp_abnormal and is_neutro_low and is_consistency_liquid)

# 1. Clean Neutrophils
df['NeutrophilCount'] = df['NeutrophilCount'].apply(parse_neutrophil)
# Impute missing neutrophils with median (optional, depends on your specific logic)
# df['NeutrophilCount'].fillna(df['NeutrophilCount'].median(), inplace=True)

# 2. One-hot encode stool consistency
df = pd.get_dummies(df, columns=['Consistency'])

# 3. Log transform Genus-relative abundances (Compositional handling)
# Ensure no negative values or zeros break the log
df['RelativeAbundance'] = df['RelativeAbundance'].astype(float)
df['RelativeAbundance'] = np.log1p(df['RelativeAbundance'])

# 4. Generate Labels (Row-wise)
df['DysbiosisLabel'] = df.apply(label_dysbiosis, axis=1)

# 5. Pivot to Wide Format (Time Series Format)
# Keep metadata
metadata_cols = ['PatientID', 'SampleID', 'DayRelativeToNearestHCT',
                 'MaxTemperature', 'NeutrophilCount'] + \
                [col for col in df.columns if col.startswith('Consistency_')] + \
                ['DysbiosisLabel']

# Pivot Genus
genus_pivot = df.pivot_table(index=['PatientID', 'SampleID', 'DayRelativeToNearestHCT'],
                             columns='Genus', values='RelativeAbundance', fill_value=0).reset_index()

# Merge Metadata back
metadata = df[metadata_cols].drop_duplicates(subset=['PatientID', 'SampleID', 'DayRelativeToNearestHCT'])
merged_df = pd.merge(genus_pivot, metadata, on=['PatientID', 'SampleID', 'DayRelativeToNearestHCT'], how='left')

print(f"Data Processed. Total Samples: {len(merged_df)}")
print(f"Total Unique Patients: {merged_df['PatientID'].nunique()}")

In [ ]:
# @title 3. Patient-Level Splitting & Scaling (PREVENT DATA LEAKAGE)

# --- STEP A: Split Patients First ---
# We split the Patient IDs, NOT the sequences.
unique_patients = merged_df['PatientID'].unique()
np.random.shuffle(unique_patients) # Randomize patient order

n_total = len(unique_patients)
n_train = int(0.70 * n_total)
n_val = int(0.15 * n_total)

train_pids = unique_patients[:n_train]
val_pids = unique_patients[n_train : n_train + n_val]
test_pids = unique_patients[n_train + n_val:]

print(f"Patients in Train: {len(train_pids)}")
print(f"Patients in Val:   {len(val_pids)}")
print(f"Patients in Test:  {len(test_pids)}")

# Create separate DataFrames based on Patient ID
df_train = merged_df[merged_df['PatientID'].isin(train_pids)].copy()
df_val = merged_df[merged_df['PatientID'].isin(val_pids)].copy()
df_test = merged_df[merged_df['PatientID'].isin(test_pids)].copy()

# --- STEP B: Feature Selection (Train Only) ---
# Identify genus columns
all_genus_cols = genus_pivot.columns.drop(['PatientID', 'SampleID', 'DayRelativeToNearestHCT']).tolist()

# Calculate variance ONLY on Training data to avoid leakage
train_variances = df_train[all_genus_cols].var()
# Drop columns with near-zero variance in training set
non_zero_var_cols = train_variances[train_variances > 1e-6].index.tolist()

# Define final feature list (Microbiome + Stool Consistency)
# Exclude Temp/Neutrophils from Input X (as they define the label Y)

#Changed on 02 December, 2025
# feature_cols = non_zero_var_cols + [col for col in merged_df.columns if 'Consistency' in col]

# === CRITICAL FIX ===
# Input Features = Microbiome ONLY.
# We REMOVE 'Consistency' because it is part of the Label definition (Leakage).
feature_cols = non_zero_var_cols
# feature_cols += [col for col in merged_df.columns if 'Consistency' in col] <--- REMOVED THIS LINE

print(f"Selected {len(feature_cols)} features (Microbiome Genus Only).")

#print(f"Selected {len(feature_cols)} features based on Training Set variance.")

# --- STEP C: Scaling (Fit on Train Only) ---
scaler = MinMaxScaler()

# 1. FIT and TRANSFORM on Training
df_train[feature_cols] = scaler.fit_transform(df_train[feature_cols])

# 2. TRANSFORM Only on Val/Test (using Train statistics)
df_val[feature_cols] = scaler.transform(df_val[feature_cols])
df_test[feature_cols] = scaler.transform(df_test[feature_cols])

print("Scaling complete. Data leakage prevented.")

In [ ]:
# @title 4. Sequence Generation (Sliding Window)

def build_sequences(df, feature_cols, label_col='DysbiosisLabel', seq_len=14):
    """
    Generates sequences strictly within patient groups.
    """
    X_sequences = []
    y_labels = []

    # Group by patient to ensure window never crosses patient boundaries
    for pid, group in df.groupby('PatientID'):
        # Sort by time
        group = group.sort_values('DayRelativeToNearestHCT')

        values = group[feature_cols].values
        labels = group[label_col].values

        # Sliding window
        if len(values) >= seq_len:
            for i in range(len(values) - seq_len + 1):
                seq = values[i:i+seq_len]
                label_window = labels[i:i+seq_len]

                # Label Logic: If ANY point in window is dysbiotic, label=1
                # (Or use label_window[-1] for "next step prediction")
                label = int(label_window.max())

                X_sequences.append(seq)
                y_labels.append(label)

    return np.array(X_sequences), np.array(y_labels)

# Build sequences for each split independently
SEQ_LEN = 14

X_train, y_train = build_sequences(df_train, feature_cols, seq_len=SEQ_LEN)
X_val, y_val = build_sequences(df_val, feature_cols, seq_len=SEQ_LEN)
X_test, y_test = build_sequences(df_test, feature_cols, seq_len=SEQ_LEN)

print(f"Training Sequences: {X_train.shape}")
print(f"Validation Sequences: {X_val.shape}")
print(f"Testing Sequences: {X_test.shape}")

In [ ]:
# @title 5. Train LSTM Autoencoder
# Filter Training Data: Autoencoder only sees "Normal" (Label=0) sequences
X_train_normal = X_train[y_train == 0]

# For Validation, we can use X_val_normal to check reconstruction of normal unseen data
X_val_normal = X_val[y_val == 0]

timesteps = X_train_normal.shape[1]
n_features = X_train_normal.shape[2]

print(f"Training on {len(X_train_normal)} normal sequences.")

# --- Architecture ---
inputs = Input(shape=(timesteps, n_features))
# Encoder
encoded = LSTM(128, return_sequences=True)(inputs)
encoded = LSTM(64, return_sequences=False)(encoded)
# Bottleneck
bottleneck = RepeatVector(timesteps)(encoded)
# Decoder
decoded = LSTM(64, return_sequences=True)(bottleneck)
decoded = LSTM(128, return_sequences=True)(decoded)
outputs = TimeDistributed(Dense(n_features))(decoded)

# Compile
autoencoder = Model(inputs, outputs)
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mae')

# --- Callbacks ---
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

# 1. Create the 'models' folder if it doesn't exist (Critical step!)
if not os.path.exists('./models'):
    os.makedirs('./models')

# 2. Update the checkpoint to save INSIDE that folder
checkpoint_cb = ModelCheckpoint(
    filepath="./models/DynaBiome_PatientSplit_Model.keras",  # <--- UPDATED PATH
    monitor="val_loss",
    save_best_only=True,
    verbose=1
)

# --- Fit ---
# Note: We validate on X_val_normal to see how well it reconstructs unseen HEALTHY patients.
history = autoencoder.fit(
    X_train_normal, X_train_normal,
    epochs=150, # 150-300 is usually sufficient
    batch_size=32,
    validation_data=(X_val_normal, X_val_normal),
    callbacks=[early_stop, checkpoint_cb],
    verbose=1
)

# Plot Loss
plt.figure(figsize=(10,6))
plt.plot(history.history['loss'], label='Train Loss (Normal)')
plt.plot(history.history['val_loss'], label='Val Loss (Normal)')
plt.title('LSTM Autoencoder Reconstruction Loss')
plt.xlabel('Epochs')
plt.ylabel('MAE Loss')
plt.legend()
plt.show()

In [ ]:
from google.colab import files
import os

# Update path to point to the 'models' folder
model_path = "./models/DynaBiome_PatientSplit_Model.keras"

try:
    files.download(model_path)
    print(f"Successfully downloaded {model_path}")
except Exception as e:
    print(f"Error: {e}")
    print(f"Could not find file at {model_path}. Did you save it to the './models' folder?")

In [ ]:
# Assuming 'autoencoder_model' is your trained autoencoder
# Assuming 'X_val' is your validation data

# Predict the reconstruction of X_val using the trained autoencoder
X_val_pred = autoencoder.predict(X_val)

# Calculate the Mean Absolute Error (MAE) between the original X_val and its reconstruction
reconstruction_errors_val = np.mean(np.abs(X_val_pred - X_val), axis=(1, 2))

# reconstruction_errors_val now contains a single reconstruction error value for each sequence in X_val

threshold = 0.0003 # np.percentile(reconstruction_errors_val, 95)
# Newly Added on 11 May, 2025
# predicted_labels = (reconstruction_errors >= threshold).astype(int)

print(f"\nAnomaly detection threshold (95th percentile): {threshold:.6f}")

In [ ]:
X_test_pred = autoencoder.predict(X_test)
reconstruction_errors_test = np.mean(np.abs(X_test_pred - X_test), axis=(1, 2))
anomaly_predictions = reconstruction_errors_test > threshold  # threshold from training on validation set. Use the Threshold obtained via validation set
print(f"Number of flagged sequences: {anomaly_predictions.sum()} out of {len(anomaly_predictions)}")

In [ ]:
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve, precision_recall_curve, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# --- 1. Find Optimal Threshold on the Validation Set ---

# Compute ROC Curve on Validation Set to get thresholds
fpr_val, tpr_val, thresholds_val = roc_curve(y_val, reconstruction_errors_val)

# Find the best threshold using Youden's index on Validation Set
optimal_idx_val = np.argmax(tpr_val - fpr_val)
optimal_threshold = thresholds_val[optimal_idx_val]

print(f"Optimal Threshold for anomaly detection (from Validation Set): {optimal_threshold:.4f}")

# --- 2. Evaluate Performance and Plot on the Test Set using the Optimal Threshold ---

# Classify Test Instances using the optimal threshold found on the validation set
anomaly_predictions_test = (reconstruction_errors_test > optimal_threshold).astype(int)

# Compute Evaluation Metrics on the Test Set
roc_auc_test = roc_auc_score(y_test, reconstruction_errors_test) # AUC is calculated from scores, not binary predictions
pr_auc_test = average_precision_score(y_test, reconstruction_errors_test) # PR AUC is calculated from scores

# You can also calculate metrics based on the binary predictions
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

precision_test = precision_score(y_test, anomaly_predictions_test, zero_division=1)
recall_test = recall_score(y_test, anomaly_predictions_test, zero_division=1)
f1_test = f1_score(y_test, anomaly_predictions_test, zero_division=1)

print("\nEvaluation on Test Set using Optimal Threshold from Validation:")
print(f"Precision: {precision_test:.4f}")
print(f"Recall: {recall_test:.4f}")
print(f"F1-Score: {f1_test:.4f}")
print(f"ROC AUC: {roc_auc_test:.4f}")
print(f"PR AUC: {pr_auc_test:.4f}")
print("\nClassification Report on Test Set:")
print(classification_report(y_test, anomaly_predictions_test, zero_division=1))


# --- 3. Plot ROC Curve on the Test Set ---

# Compute ROC Curve on Test Set (for plotting the curve itself)
fpr_test, tpr_test, thresholds_test_plot = roc_curve(y_test, reconstruction_errors_test)

plt.figure(figsize=(12, 5))

# --- ROC ---
plt.subplot(1, 2, 1)
plt.plot(fpr_test, tpr_test, label=f'ROC AUC = {roc_auc_test:.3f}', color='blue')

# Find the point on the Test ROC curve corresponding to the optimal threshold from Validation
# This requires finding the index in thresholds_test_plot closest to optimal_threshold
closest_threshold_idx_test = np.argmin(np.abs(thresholds_test_plot - optimal_threshold))
plt.scatter(fpr_test[closest_threshold_idx_test], tpr_test[closest_threshold_idx_test],
            color='red', label=f'Applied Threshold ({optimal_threshold:.4f})', zorder=5) # zorder to ensure point is visible

plt.plot([0, 1], [0, 1], 'k--')  # Diagonal reference line
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve (Evaluated on Test Set)")
plt.legend()
plt.grid(True)

# Save the ROC plot
# Define a suitable output directory if not already defined
output_dir_evaluation_plots = "/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/Evaluation_Plots/"
import os
os.makedirs(output_dir_evaluation_plots, exist_ok=True)

plt.savefig(f"{output_dir_evaluation_plots}/ROC_Curve_TestSet_ValidatedThreshold.pdf", dpi=600, bbox_inches='tight')

# --- 4. Plot Precision-Recall Curve on the Test Set ---

# Compute PR Curve on Test Set (for plotting the curve itself)
precision_test_plot, recall_test_plot, thresholds_pr_test_plot = precision_recall_curve(y_test, reconstruction_errors_test)

plt.subplot(1, 2, 2) # Plotting PR curve in the second subplot
plt.plot(recall_test_plot, precision_test_plot, label=f'PR AUC = {pr_auc_test:.3f}', color='red')

# Find the point on the Test PR curve corresponding to the optimal threshold from Validation
# Similar to ROC, find the index in thresholds_pr_test_plot closest to optimal_threshold
# Note: precision_recall_curve thresholds are slightly different in number/values than roc_curve
closest_threshold_idx_pr_test = np.argmin(np.abs(thresholds_pr_test_plot - optimal_threshold))
# Need to handle the case where thresholds_pr_test_plot might be empty (e.g., all samples are the same)
if thresholds_pr_test_plot.size > 0:
     plt.scatter(recall_test_plot[closest_threshold_idx_pr_test], precision_test_plot[closest_threshold_idx_pr_test],
                 color='blue', label=f'Applied Threshold ({optimal_threshold:.4f})', zorder=5)

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve (Evaluated on Test Set)')
plt.legend()
plt.grid(True)

# Save the PR plot
plt.savefig(f"{output_dir_evaluation_plots}/PR_Curve_TestSet_ValidatedThreshold.pdf", dpi=600, bbox_inches='tight')


plt.tight_layout() # Adjust layout to prevent overlap
plt.show()

# --- 5. Plot Confusion Matrix on the Test Set ---
cm_test = confusion_matrix(y_test, anomaly_predictions_test)

plt.figure(figsize=(8, 6))
sns.heatmap(cm_test, annot=True, fmt="d", cmap="Blues",
            xticklabels=['Predicted Normal', 'Predicted Anomaly'],
            yticklabels=['Actual Normal', 'Actual Anomaly'])
plt.title('Confusion Matrix (Evaluated on Test Set)')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.savefig(f"{output_dir_evaluation_plots}/ConfusionMatrix_TestSet_ValidatedThreshold.pdf", dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
# ==================== STEP 6: Visualization ====================
plt.figure(figsize=(10,4))
plt.hist(reconstruction_errors_test, bins=50, color='skyblue', edgecolor='black', density=True, alpha=0.7) # Use density=True for better area representation

plt.axvline(optimal_threshold, color='red', linestyle='--', label=f'Threshold ({optimal_threshold:.4f})')

# Shade the areas
# Normal region (errors <= threshold)
plt.fill_betweenx([0, plt.gca().get_ylim()[1]], 0, optimal_threshold, color='green', alpha=0.2, label='Normal')
# Anomalous region (errors > threshold)
plt.fill_betweenx([0, plt.gca().get_ylim()[1]], optimal_threshold, plt.gca().get_xlim()[1], color='red', alpha=0.2, label='Anomalous')


plt.title("Reconstruction Error Distribution with Anomaly Threshold")
plt.xlabel("Reconstruction Error (MAE)")
plt.ylabel("Density") # Changed to Density if density=True
plt.legend()
plt.tight_layout()
# plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/BenchMark_Figures/reconstruction_error_distribution.pdf", dpi=600, bbox_inches='tight')
plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/reconstruction_error_distribution_OriginalApproach_Thresholded.pdf", dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
X_test_pred = autoencoder.predict(X_test)
reconstruction_errors_test = np.mean(np.abs(X_test_pred - X_test), axis=(1, 2))
anomaly_predictions = reconstruction_errors_test > threshold  # threshold from training on validation set. Use the Threshold obtained via validation set
print(f"Number of flagged sequences: {anomaly_predictions.sum()} out of {len(anomaly_predictions)}")

In [ ]:
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve, precision_recall_curve, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# --- 1. Find Optimal Threshold on the Validation Set ---

# Compute ROC Curve on Validation Set to get thresholds
fpr_val, tpr_val, thresholds_val = roc_curve(y_val, reconstruction_errors_val)

# Find the best threshold using Youden's index on Validation Set
optimal_idx_val = np.argmax(tpr_val - fpr_val)
optimal_threshold = thresholds_val[optimal_idx_val]

print(f"Optimal Threshold for anomaly detection (from Validation Set): {optimal_threshold:.4f}")

# --- 2. Evaluate Performance and Plot on the Test Set using the Optimal Threshold ---

# Classify Test Instances using the optimal threshold found on the validation set
anomaly_predictions_test = (reconstruction_errors_test > optimal_threshold).astype(int)

# Compute Evaluation Metrics on the Test Set
roc_auc_test = roc_auc_score(y_test, reconstruction_errors_test) # AUC is calculated from scores, not binary predictions
pr_auc_test = average_precision_score(y_test, reconstruction_errors_test) # PR AUC is calculated from scores

# You can also calculate metrics based on the binary predictions
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

precision_test = precision_score(y_test, anomaly_predictions_test, zero_division=1)
recall_test = recall_score(y_test, anomaly_predictions_test, zero_division=1)
f1_test = f1_score(y_test, anomaly_predictions_test, zero_division=1)

print("\nEvaluation on Test Set using Optimal Threshold from Validation:")
print(f"Precision: {precision_test:.4f}")
print(f"Recall: {recall_test:.4f}")
print(f"F1-Score: {f1_test:.4f}")
print(f"ROC AUC: {roc_auc_test:.4f}")
print(f"PR AUC: {pr_auc_test:.4f}")
print("\nClassification Report on Test Set:")
print(classification_report(y_test, anomaly_predictions_test, zero_division=1))


# --- 3. Plot ROC Curve on the Test Set ---

# Compute ROC Curve on Test Set (for plotting the curve itself)
fpr_test, tpr_test, thresholds_test_plot = roc_curve(y_test, reconstruction_errors_test)

plt.figure(figsize=(12, 5))

# --- ROC ---
plt.subplot(1, 2, 1)
plt.plot(fpr_test, tpr_test, label=f'ROC AUC = {roc_auc_test:.3f}', color='blue')

# Find the point on the Test ROC curve corresponding to the optimal threshold from Validation
# This requires finding the index in thresholds_test_plot closest to optimal_threshold
closest_threshold_idx_test = np.argmin(np.abs(thresholds_test_plot - optimal_threshold))
plt.scatter(fpr_test[closest_threshold_idx_test], tpr_test[closest_threshold_idx_test],
            color='red', label=f'Applied Threshold ({optimal_threshold:.4f})', zorder=5) # zorder to ensure point is visible

plt.plot([0, 1], [0, 1], 'k--')  # Diagonal reference line
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve (Evaluated on Test Set)")
plt.legend()
plt.grid(True)

# Save the ROC plot
# Define a suitable output directory if not already defined
output_dir_evaluation_plots = "/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/Evaluation_Plots/"
import os
os.makedirs(output_dir_evaluation_plots, exist_ok=True)

plt.savefig(f"{output_dir_evaluation_plots}/ROC_Curve_TestSet_ValidatedThreshold.pdf", dpi=600, bbox_inches='tight')

# --- 4. Plot Precision-Recall Curve on the Test Set ---

# Compute PR Curve on Test Set (for plotting the curve itself)
precision_test_plot, recall_test_plot, thresholds_pr_test_plot = precision_recall_curve(y_test, reconstruction_errors_test)

plt.subplot(1, 2, 2) # Plotting PR curve in the second subplot
plt.plot(recall_test_plot, precision_test_plot, label=f'PR AUC = {pr_auc_test:.3f}', color='red')

# Find the point on the Test PR curve corresponding to the optimal threshold from Validation
# Similar to ROC, find the index in thresholds_pr_test_plot closest to optimal_threshold
# Note: precision_recall_curve thresholds are slightly different in number/values than roc_curve
closest_threshold_idx_pr_test = np.argmin(np.abs(thresholds_pr_test_plot - optimal_threshold))
# Need to handle the case where thresholds_pr_test_plot might be empty (e.g., all samples are the same)
if thresholds_pr_test_plot.size > 0:
     plt.scatter(recall_test_plot[closest_threshold_idx_pr_test], precision_test_plot[closest_threshold_idx_pr_test],
                 color='blue', label=f'Applied Threshold ({optimal_threshold:.4f})', zorder=5)

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve (Evaluated on Test Set)')
plt.legend()
plt.grid(True)

# Save the PR plot
plt.savefig(f"{output_dir_evaluation_plots}/PR_Curve_TestSet_ValidatedThreshold.pdf", dpi=600, bbox_inches='tight')


plt.tight_layout() # Adjust layout to prevent overlap
plt.show()

# --- 5. Plot Confusion Matrix on the Test Set ---
cm_test = confusion_matrix(y_test, anomaly_predictions_test)

plt.figure(figsize=(8, 6))
sns.heatmap(cm_test, annot=True, fmt="d", cmap="Blues",
            xticklabels=['Predicted Normal', 'Predicted Anomaly'],
            yticklabels=['Actual Normal', 'Actual Anomaly'])
plt.title('Confusion Matrix (Evaluated on Test Set)')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.savefig(f"{output_dir_evaluation_plots}/ConfusionMatrix_TestSet_ValidatedThreshold.pdf", dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
# ==================== STEP 6: Visualization ====================
plt.figure(figsize=(10,4))
plt.hist(reconstruction_errors_test, bins=50, color='skyblue', edgecolor='black', density=True, alpha=0.7) # Use density=True for better area representation

plt.axvline(optimal_threshold, color='red', linestyle='--', label=f'Threshold ({optimal_threshold:.4f})')

# Shade the areas
# Normal region (errors <= threshold)
plt.fill_betweenx([0, plt.gca().get_ylim()[1]], 0, optimal_threshold, color='green', alpha=0.2, label='Normal')
# Anomalous region (errors > threshold)
plt.fill_betweenx([0, plt.gca().get_ylim()[1]], optimal_threshold, plt.gca().get_xlim()[1], color='red', alpha=0.2, label='Anomalous')


plt.title("Reconstruction Error Distribution with Anomaly Threshold")
plt.xlabel("Reconstruction Error (MAE)")
plt.ylabel("Density") # Changed to Density if density=True
plt.legend()
plt.tight_layout()
# plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/BenchMark_Figures/reconstruction_error_distribution.pdf", dpi=600, bbox_inches='tight')
plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/reconstruction_error_distribution_OriginalApproach_Thresholded.pdf", dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set global plot style for publication
sns.set(style="whitegrid")
plt.rcParams.update({
    'font.family': 'serif',
    'axes.titlesize': 18,
    'axes.labelsize': 16,
    'axes.labelweight': 'bold',  # Bold axis labels
    'axes.titleweight': 'bold',  # Bold title
    'xtick.labelsize': 13,
    'ytick.labelsize': 13,
    'figure.dpi': 600  # High resolution
})

# Reconstruct predictions
X_pred_test = autoencoder.predict(X_test)
reconstruction_errors = np.mean(np.abs(X_pred_test - X_test), axis=(1, 2))
feature_errors = np.abs(X_pred_test - X_test)
feature_errors_avg = feature_errors.mean(axis=1)

# Anomaly filtering
anomaly_flags = reconstruction_errors > optimal_threshold
anomalous_feature_errors = feature_errors_avg[anomaly_flags]
mean_error_per_feature = anomalous_feature_errors.mean(axis=0)

# DataFrame preparation
error_df = pd.DataFrame({
    'Feature': feature_cols,
    'MeanAnomalyError': mean_error_per_feature
}).sort_values(by='MeanAnomalyError', ascending=True)

# Plotting
plt.figure(figsize=(10, 8))
barplot = sns.barplot(
    data=error_df.tail(20),
    x='MeanAnomalyError',
    y='Feature',
    palette='coolwarm'
)

# Annotate bars
for p in barplot.patches:
    width = p.get_width()
    barplot.text(
        width + 0.005,
        p.get_y() + p.get_height() / 2,
        f'{width:.3f}',
        ha='left',
        va='center',
        fontsize=10,
        weight='bold'
    )

# Titles and labels
plt.title("Top Contributing Genera/Features to Anomalous Patterns", fontsize=18, weight='bold')
plt.xlabel("Mean Absolute Error (Anomalous Sequences)", fontsize=14, weight='bold')
plt.ylabel("Genus / Feature", fontsize=14, weight='bold')

plt.tight_layout()

# Save as high-quality figure
plt.savefig("Top_Contributing_Genera.pdf", dpi=600, bbox_inches='tight')
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set global plot style for publication
sns.set(style="whitegrid")
plt.rcParams.update({
    'font.family': 'serif',
    'axes.titlesize': 18,
    'axes.labelsize': 16,
    'axes.labelweight': 'bold',
    'axes.titleweight': 'bold',
    'xtick.labelsize': 13,
    'ytick.labelsize': 14,         # Increased for clarity
    'figure.dpi': 600              # High resolution
})

# Reconstruct predictions
X_pred_test = autoencoder.predict(X_test)
reconstruction_errors = np.mean(np.abs(X_pred_test - X_test), axis=(1, 2))
feature_errors = np.abs(X_pred_test - X_test)
feature_errors_avg = feature_errors.mean(axis=1)

# Anomaly filtering
anomaly_flags = reconstruction_errors > optimal_threshold
anomalous_feature_errors = feature_errors_avg[anomaly_flags]
mean_error_per_feature = anomalous_feature_errors.mean(axis=0)

# DataFrame preparation
error_df = pd.DataFrame({
    'Feature': feature_cols,
    'MeanAnomalyError': mean_error_per_feature
}).sort_values(by='MeanAnomalyError', ascending=True)

# Plotting
plt.figure(figsize=(10, 8))
barplot = sns.barplot(
    data=error_df.tail(20),
    x='MeanAnomalyError',
    y='Feature',
    palette='coolwarm'
)

# Annotate bars
for p in barplot.patches:
    width = p.get_width()
    barplot.text(
        width + 0.005,
        p.get_y() + p.get_height() / 2,
        f'{width:.3f}',
        ha='left',
        va='center',
        fontsize=10,
        weight='bold'
    )

# Improve y-axis label readability
barplot.set_yticklabels(
    barplot.get_yticklabels(),
    fontsize=14,
    weight='bold',
    color='black'
)

# Titles and labels
plt.title("Top Contributing Genera/Features to Anomalous Patterns", fontsize=18, weight='bold')
plt.xlabel("Mean Absolute Error (Anomalous Sequences)", fontsize=14, weight='bold')
plt.ylabel("Genus / Feature", fontsize=14, weight='bold')

plt.tight_layout()

# Save as high-quality figure
plt.savefig("Top_Contributing_Genera.pdf", dpi=600, bbox_inches='tight')
plt.show()


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

# Assuming you have reconstruction_errors_val and y_val

# Define the parameter grid for Logistic Regression
# Ensure compatible combinations of penalty and solver
param_grid = [
    {'C': [0.001, 0.01, 0.1, 1, 10, 100],
     'penalty': ['l1', 'l2'],
     'solver': ['liblinear', 'saga']}, # liblinear and saga support l1/l2
    {'C': [0.001, 0.01, 0.1, 1, 10, 100],
     'penalty': ['elasticnet'],
     'solver': ['saga'], # Only saga supports elasticnet
     'l1_ratio': [0.1, 0.5, 0.9]}, # l1_ratio is needed for elasticnet
     {'C': [0.001, 0.01, 0.1, 1, 10, 100],
      'penalty': [None], # Change 'none' to None (capitalized)
      'solver': ['saga', 'lbfgs']} # saga and lbfgs support None penalty
]

# Create a GridSearchCV object
grid_search = GridSearchCV(LogisticRegression(class_weight='balanced'), param_grid, cv=5, scoring='f1', n_jobs=-1, error_score='raise') # Using F1-score as the scoring metric

# Fit the grid search to the validation data
grid_search.fit(reconstruction_errors_val.reshape(-1, 1), y_val)

# Print the best hyperparameters and best score
print("Best hyperparameters:", grid_search.best_params_)
print("Best F1-score:", grid_search.best_score_)

# You can then use the best model found by grid search to evaluate on the test set
best_clf = grid_search.best_estimator_
predictions = best_clf.predict(reconstruction_errors_test.reshape(-1, 1))

# Evaluate on the test set as you did before
print(classification_report(y_test, predictions, zero_division=1))
print(roc_auc_score(y_test, best_clf.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1]))
print(average_precision_score(y_test, best_clf.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1]))

In [ ]:
# Predict using trained classifier on test set
predictions = best_clf.predict(reconstruction_errors_test.reshape(-1, 1))

# Individual Metrics
precision = precision_score(y_test, predictions,zero_division=1)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)
roc_auc = roc_auc_score(y_test, reconstruction_errors_test)
pr_auc = average_precision_score(y_test, reconstruction_errors_test)
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}") # F1-score for the minority/positive class (Anomaly)
print(f"ROC AUC: {roc_auc:.4f}")
print(f"PR AUC: {pr_auc:.4f}")

# Calculate weighted average F1-score
weighted_f1 = f1_score(y_test, predictions, average='weighted',zero_division=1) #Overall summary
print(f"Weighted Average F1 Score: {weighted_f1:.4f}")

print("\nClassification Report on Test Set (Unsupervised Fashion):")
print(classification_report(y_test, predictions, zero_division=1))

In [ ]:
# prompt: Draw PR Curve from pr_auc
import os # Ensure os is imported
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

# Define the output directory for Logistic Regression plots
logistic_regression_output_dir = "/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/BenchMark_Figures/LogisticRegression"

# Create the directory if it doesn't exist
os.makedirs(logistic_regression_output_dir, exist_ok=True)


# Assuming 'reconstruction_errors' and 'true_labels' are defined as in your code
precision, recall, thresholds = precision_recall_curve(y_test, predictions)

plt.figure(figsize=(8, 6))
plt.plot(recall, precision, marker='.', label=f'PR Curve (AUC = {pr_auc:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend()
plt.grid(True)
# Use the new directory variable
plt.savefig(f"{logistic_regression_output_dir}/PR_Curve_LR.pdf", dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, precision_recall_curve
import matplotlib.pyplot as plt
logistic_regression_output_dir = "/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/LogisticRegression"
os.makedirs(logistic_regression_output_dir, exist_ok=True)

# Calculate fpr and tpr using roc_curve before plotting:
fpr, tpr, _ = roc_curve(y_test, predictions)  # Assuming 'y_test' and 'reconstruction_errors' are defined

# Now you can proceed with plotting the ROC curve:
plt.figure(figsize=(12, 5))

# --- ROC ---
plt.subplot(1, 2, 1)
plt.plot(fpr, tpr, label=f'ROC AUC = {roc_auc:.3f}')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend()
plt.grid(True)
# Save as high-resolution PDF using the defined directory variable
plt.savefig(f"{logistic_regression_output_dir}/ROC_Curve_LogisticRegression.pdf", dpi=600, bbox_inches='tight')

In [ ]:
from sklearn.metrics import roc_curve, precision_recall_curve, confusion_matrix
import seaborn as sns
# Calculate and assign the confusion matrix to 'cm'
cm = confusion_matrix(y_test, predictions) # Calculate confusion matrix

# Draw Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=['Predicted Normal', 'Predicted Anomaly'],
            yticklabels=['Actual Normal', 'Actual Anomaly'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/LogisticRegression/PR_Curve_LogisticRegression.pdf", dpi=600, bbox_inches='tight')
plt.show()

# Keras Tuner for MLP Hyperparameters

In [ ]:
# !pip install keras-tuner
# from kerastuner.tuners import RandomSearch

# # Function to tune hyperparameters
# def tune_mlp_model(hp):
#     model = Sequential([
#         Dense(hp.Int('neurons1', 32, 128, step=32), activation=hp.Choice('activation', ['relu', 'tanh']), input_shape=(1,)),
#         Dense(hp.Int('neurons2', 16, 64, step=16), activation=hp.Choice('activation', ['relu', 'tanh'])),
#         Dense(1, activation='sigmoid')
#     ])
#     model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#     return model

# # Define tuner
# tuner = RandomSearch(
#     tune_mlp_model,
#     objective='val_accuracy',
#     max_trials=10,
#     executions_per_trial=1,
#     directory='mlp_tuner',
#     project_name='mlp_optimization'
# )

# # Search for best hyperparameters
# tuner.search(reconstruction_errors_val.reshape(-1, 1), y_val, epochs=50, validation_split=0.2)

# # Get the best hyperparameters
# best_hyperparameters = tuner.get_best_hyperparameters()[0]
# print("Best Hyperparameters:", best_hyperparameters.values)


# Non-Linear Classifier (MLP on reconstruction errors)

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input

# Use best hyperparameters found via tuning
input_layer = Input(shape=(1,))
dense_1 = Dense(96, activation='relu')(input_layer)  # Update neurons1 to 96
dense_2 = Dense(64, activation='relu')(dense_1)  # Update neurons2 to 64
output_layer = Dense(1, activation='sigmoid')(dense_2)

# Define the optimized model
mlp_model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
mlp_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with the same dataset but use best hyperparameters
mlp_model.fit(reconstruction_errors_val.reshape(-1, 1), y_val, epochs=100, batch_size=16)  # Keep epochs & batch_size unchanged

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import classification_report, roc_curve, roc_auc_score, precision_recall_curve, average_precision_score, matthews_corrcoef # Import classification_report and other necessary functions

# Define the directory path where the plots will be saved
mlp_output_dir = "/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/MLP"

# Create the directory if it doesn't exist. exist_ok=True prevents an error if the directory already exists.
os.makedirs(mlp_output_dir, exist_ok=True)

# Get probabilities from the trained MLP model on the test set reconstruction errors
mlp_probs = mlp_model.predict(reconstruction_errors_test.reshape(-1, 1)).flatten()

# Convert probabilities to binary labels (you can adjust the threshold if needed)
threshold = 0.5
mlp_pred_classes = (mlp_probs > threshold).astype(int)

# Classification report
print("MLP Classification Report (Trained on Validation, Tested on Test):")
print(classification_report(y_test, mlp_pred_classes, target_names=["Non-Dysbiotic", "Dysbiotic"], zero_division=1))

# Compute ROC curve and AUC score
fpr_mlp, tpr_mlp, _ = roc_curve(y_test, mlp_probs)
roc_auc_mlp = roc_auc_score(y_test, mlp_probs)
print(f"MLP ROC AUC Score: {roc_auc_mlp}")

# Compute Precision-Recall curve
precision_mlp, recall_mlp, _ = precision_recall_curve(y_test, mlp_probs)
average_precision_mlp = average_precision_score(y_test, mlp_probs)
print(f"MLP Average Precision: {average_precision_mlp}")

# Plot ROC curve
plt.figure(figsize=(8, 5))
plt.plot(fpr_mlp, tpr_mlp, color='blue', lw=2, label=f'ROC curve (AUC = {roc_auc_mlp:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - MLP on Reconstruction Error (Trained Val, Tested Test)')
plt.legend(loc="lower right")
plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/MLP/ROC_Curve_MLP.pdf", dpi=600, bbox_inches='tight')

plt.show()

# Plot Precision-Recall curve
plt.figure(figsize=(8, 5))
plt.plot(recall_mlp, precision_mlp, color='red', lw=2, label=f'PR curve (AP = {average_precision_mlp:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve - MLP on Reconstruction Error (Trained Val, Tested Test)')
plt.legend(loc="lower left")
plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/MLP/PR_Curve_MLP.pdf", dpi=600, bbox_inches='tight')
plt.show()

# Compute Matthews Correlation Coefficient (MCC)
mcc_mlp = matthews_corrcoef(y_test, mlp_pred_classes)
print(f"MLP Matthews Correlation Coefficient (MCC): {mcc_mlp}")

In [ ]:
# prompt: PRINT PR AUC of MLP

print(f"PR AUC of MLP: {average_precision_mlp:.4f}")


In [ ]:
# prompt: Compute Precision, recall and F1 Score on MLP

from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming y_test and mlp_pred_classes are already defined from the preceding code

# Compute Precision
precision_mlp = precision_score(y_test, mlp_pred_classes, zero_division=1)

# Compute Recall
recall_mlp = recall_score(y_test, mlp_pred_classes, zero_division=1)

# Compute F1 Score
f1_mlp = f1_score(y_test, mlp_pred_classes, zero_division=1)

print(f"MLP Precision: {precision_mlp:.4f}")
print(f"MLP Recall: {recall_mlp:.4f}")
print(f"MLP F1 Score: {f1_mlp:.4f}")

In [ ]:
# prompt: Generate confusion matrix for mlp

# Compute confusion matrix for MLP predictions
cm_mlp = confusion_matrix(y_test, mlp_pred_classes)

# Display confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm_mlp, annot=True, fmt="d", cmap="Blues",
            xticklabels=['Predicted Non-Dysbiotic', 'Predicted Dysbiotic'],
            yticklabels=['Actual Non-Dysbiotic', 'Actual Dysbiotic'])
plt.title('Confusion Matrix - MLP on Reconstruction Error (Trained Val, Tested Test)')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/MLP/ConfusionMatrix_MLP.pdf", dpi=600, bbox_inches='tight')
plt.show()


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import OneClassSVM
from sklearn.metrics import make_scorer, f1_score
import numpy as np # Import numpy

# Define custom scoring function
def custom_f1_scorer(estimator, X, y):
    anomaly_scores = estimator.decision_function(X)
    # In One-Class SVM, lower scores mean more anomalous.
    # We want to find a threshold such that a certain percentage of the *training* data
    # (which is assumed to be mostly normal) is considered anomalies.
    # The 5th percentile means the 5% lowest scores are considered anomalous.
    # Since decision_function gives higher values for inliers, we threshold on the low side.
    threshold = np.percentile(anomaly_scores, 5)
    anomaly_predictions = anomaly_scores < threshold # Anomalies are below the threshold
    # Only evaluate F1 on the actual anomalies in the test set.
    # For RandomizedSearchCV during validation, we score against y_val
    return f1_score(y, anomaly_predictions)


# Define the hyperparameter search space
param_grid = {
    'nu': np.linspace(0.01, 0.2, 10), # nu is an upper bound on the fraction of training errors
    'gamma': ['scale', 'auto', 0.01, 0.1, 1] # Kernel coefficient
}

# Set up RandomizedSearchCV
ocsvm = OneClassSVM(kernel="rbf")
# Use make_scorer with our custom F1 function, setting greater_is_better=True (default)
random_search = RandomizedSearchCV(ocsvm, param_grid, scoring=make_scorer(custom_f1_scorer), n_iter=10, cv=3, n_jobs=-1)

# *** Calculate reconstruction errors for the training data ***
# Assuming 'autoencoder' and 'X_train' are available from previous steps
X_train_pred = autoencoder.predict(X_train)
reconstruction_errors_train = np.mean(np.abs(X_train_pred - X_train), axis=(1, 2))


# Fit search on training data's reconstruction errors
# The y_train is used by the custom_f1_scorer during cross-validation
# to calculate the F1 score on the validation folds within the training data.
random_search.fit(reconstruction_errors_train.reshape(-1, 1), y_train)

# Show best parameters
print("Best Hyperparameters:", random_search.best_params_)

# Once Class SVM on reconstruction errors

In [ ]:
from sklearn.svm import OneClassSVM
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, roc_curve, auc, precision_recall_curve
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler # Or StandardScaler

# 1. Train OCSVM on Reconstruction Errors of Normal Data
# Assuming 'reconstruction_errors_train' contains errors for normal training data

# 1. Calculate Reconstruction Errors for Training Data
X_train_pred = autoencoder.predict(X_train)  # Predict on training data
reconstruction_errors_train = np.mean(np.abs(X_train_pred - X_train), axis=(1, 2))

# Use best hyperparameters
best_nu = 0.0944
best_gamma = 0.01

# Train One-Class SVM on Reconstruction Errors of Normal Data
ocsvm = OneClassSVM(nu=best_nu, kernel="rbf", gamma=best_gamma)
ocsvm.fit(reconstruction_errors_train.reshape(-1, 1))

# 2. Predict Anomaly Scores on Test Data
anomaly_scores = ocsvm.decision_function(reconstruction_errors_test.reshape(-1, 1))
# Lower scores indicate higher anomaly probability

# 3. Thresholding for Anomaly Detection
threshold = np.percentile(anomaly_scores, 5)  # Adjust percentile for desired sensitivity
anomaly_predictions = anomaly_scores < threshold # Points below threshold are anomalies

# 4. Evaluate Performance and Print Classification Report
print("\nOne-Class SVM Classification Report:")
print(classification_report(y_test, anomaly_predictions))

# Define the output directory for SVM plots
svm_output_dir = "/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/svm"

# Create the directory if it doesn't exist
os.makedirs(svm_output_dir, exist_ok=True)

# 5. Plot ROC Curve
fpr, tpr, _ = roc_curve(y_test, -anomaly_scores) # Use negative scores for ROC (higher score = more anomalous)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic for One-Class SVM')
plt.legend(loc="lower right")
plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/svm/RoC_Curve_SVM.pdf", dpi=600, bbox_inches='tight')
plt.show()

# 6. Plot PR Curve
precision, recall, _ = precision_recall_curve(y_test, -anomaly_scores) # Use negative scores for PR (higher score = more anomalous)
pr_auc = auc(recall, precision)

plt.figure(figsize=(8, 6))
plt.plot(recall, precision, color='darkorange', lw=2, label=f'PR curve (area = {pr_auc:.2f})')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve for One-Class SVM')
plt.legend(loc="lower left")
plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/svm/PR_Curve_SVM.pdf", dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
# prompt: Create confusion matrix for one class SVM Classification
from sklearn.metrics import confusion_matrix # Import confusion_matrix here
import seaborn as sns # Import seaborn for heatmap visualization
import matplotlib.pyplot as plt # Import matplotlib for plotting

# Calculate and assign the confusion matrix for One-Class SVM to 'cm_ocsvm'
cm_ocsvm = confusion_matrix(y_test, anomaly_predictions)

# Draw Confusion Matrix for One-Class SVM
plt.figure(figsize=(8, 6))
sns.heatmap(cm_ocsvm, annot=True, fmt="d", cmap="Blues",
            xticklabels=['Predicted Normal', 'Predicted Anomaly'],
            yticklabels=['Actual Normal', 'Actual Anomaly'])
plt.title('Confusion Matrix - One-Class SVM (Scaled Errors)')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/svm/confusion_matrix.pdf", dpi=600, bbox_inches='tight')
plt.show()


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier

# Define KNN classifier
knn = KNeighborsClassifier()

# Corrected hyperparameter search space
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11, 15],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
}

# Perform Randomized Search
random_search = RandomizedSearchCV(knn, param_grid, cv=3, scoring='accuracy', n_iter=10, n_jobs=-1)
random_search.fit(reconstruction_errors_val.reshape(-1, 1), y_val)

# Print best parameters
print("Best Hyperparameters:", random_search.best_params_)


# K-Nearest Neighbors Classifier

In [ ]:
# prompt: Add K-Nearest Neighbors Classifier

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, precision_recall_curve
from sklearn.metrics import confusion_matrix # Import confusion_matrix
import seaborn as sns # Import seaborn

# Assuming you have already computed:
# reconstruction_errors_val, y_val
# reconstruction_errors_test, y_test

# Train K-Nearest Neighbors Classifier
# You might want to tune the 'n_neighbors' parameter
#knn = KNeighborsClassifier(n_neighbors=5)
# Use the best hyperparameters found
knn = KNeighborsClassifier(weights='distance', n_neighbors=9, metric='minkowski', algorithm='brute')

# **Corrected:** Train on validation set reconstruction errors and labels
knn.fit(reconstruction_errors_val.reshape(-1, 1), y_val)

# Predict probabilities on Test Set Reconstruction Errors
# Use the predict_proba method on the trained KNN model
knn_probs = knn.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1]

# Get binary predictions using a threshold (default is 0.5 for predict_proba)
knn_preds = (knn_probs > 0.5).astype(int)

# Evaluation on Test Set
print("\nK-Nearest Neighbors Classification Report (Trained on Validation, Tested on Test):\n",
      classification_report(y_test, knn_preds, digits=4, zero_division=1))

# Compute ROC AUC
# Use the probabilities for AUC calculation
roc_auc_knn = roc_auc_score(y_test, knn_probs)
print("K-Nearest Neighbors ROC AUC (Trained on Validation, Tested on Test):", roc_auc_knn)

# Compute Average Precision (PR AUC)
# Use the probabilities for PR AUC calculation
average_precision_knn = average_precision_score(y_test, knn_probs)
print("K-Nearest Neighbors Average Precision (PR AUC) (Trained on Validation, Tested on Test):", average_precision_knn)

# Calculate Confusion Matrix for KNN
cm_knn = confusion_matrix(y_test, knn_preds)

# Draw Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm_knn, annot=True, fmt="d", cmap="Blues",
            xticklabels=['Predicted Non-Dysbiotic', 'Predicted Dysbiotic'],
            yticklabels=['Actual Non-Dysbiotic', 'Actual Dysbiotic'])
plt.title('Confusion Matrix - K-Nearest Neighbors on Reconstruction Error (Trained Val, Tested Test)')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
#plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/K_Nearest_Neighbor/confusion_matrix.pdf", dpi=600, bbox_inches='tight')
plt.show()


# Plot ROC curve
fpr_knn, tpr_knn, _ = roc_curve(y_test, knn_probs)

plt.figure(figsize=(8, 5))
plt.plot(fpr_knn, tpr_knn, color='green', lw=2, label=f'KNN ROC curve (AUC = {roc_auc_knn:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - K-Nearest Neighbors on Reconstruction Error (Trained Val, Tested Test)')
plt.legend(loc="lower right")
plt.grid(True)
#plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/K_Nearest_Neighbor/KNN_ROC_curve.pdf", dpi=600, bbox_inches='tight')
plt.show()

# Plot Precision-Recall curve
precision_knn, recall_knn, _ = precision_recall_curve(y_test, knn_probs)

plt.figure(figsize=(8, 5))
plt.plot(recall_knn, precision_knn, color='purple', lw=2, label=f'KNN PR curve (AP = {average_precision_knn:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve - K-Nearest Neighbors on Reconstruction Error (Trained Val, Tested Test)')
plt.legend(loc="lower left")
plt.grid(True)
#plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/K_Nearest_Neighbor/PR_curve.pdf", dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
!pip install xgboost

In [ ]:
from xgboost import cv, DMatrix

# Prepare data in DMatrix format
dtrain = DMatrix(reconstruction_errors_val.reshape(-1, 1), label=y_val)

# Set parameter space
xgb_params = {
    'max_depth': 5,
    'learning_rate': 0.1,
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
}

# Perform Cross-Validation
cv_results = cv(xgb_params, dtrain, num_boost_round=200, nfold=3, metrics="logloss", early_stopping_rounds=10)

# Find best number of trees
best_n_estimators = cv_results.shape[0]
print(f"Best number of estimators: {best_n_estimators}")


In [ ]:
# %%
#!pip install xgboost
# %%
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, precision_recall_curve, average_precision_score
import matplotlib.pyplot as plt
import seaborn as sns  # Import seaborn

# Train XGBoost on Validation Set Reconstruction Errors
xgb = XGBClassifier(eval_metric='logloss', n_estimators=200)

# **Corrected:** Train on validation set reconstruction errors
xgb.fit(reconstruction_errors_val.reshape(-1, 1), y_val)

# Predict probabilities on Test Set Reconstruction Errors
xgb_probs = xgb.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1]
xgb_preds = (xgb_probs > 0.5).astype(int)

# Evaluation on Test Set
print("XGBoost Classification Report (Trained on Validation, Tested on Test):\n",
      classification_report(y_test, xgb_preds, digits=4, zero_division=1))
print("XGBoost ROC AUC (Trained on Validation, Tested on Test):",
      roc_auc_score(y_test, xgb_probs))
print("XGBoost Average Precision (PR AUC) (Trained on Validation, Tested on Test):",
      average_precision_score(y_test, xgb_probs))

# Calculate Confusion Matrix for XGBoost
cm_xgb = confusion_matrix(y_test, xgb_preds)

# Draw Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm_xgb, annot=True, fmt="d", cmap="Blues",
            xticklabels=['Predicted Non-Dysbiotic', 'Predicted Dysbiotic'],
            yticklabels=['Actual Non-Dysbiotic', 'Actual Dysbiotic'])
plt.title('Confusion Matrix - XGBoost on Reconstruction Error (Trained Val, Tested Test)')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
#plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/xgboost/confusion_matrix.pdf", dpi=600, bbox_inches='tight')
plt.show()

# Plot ROC curve
fpr_xgb, tpr_xgb, _ = roc_curve(y_test, xgb_probs)
roc_auc_xgb = roc_auc_score(y_test, xgb_probs) # Recalculate AUC using test probabilities

plt.figure(figsize=(8, 5))
plt.plot(fpr_xgb, tpr_xgb, color='darkorange', lw=2, label=f'XGBoost ROC curve (AUC = {roc_auc_xgb:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic for XGBoost (Trained Val, Tested Test)')
plt.legend(loc="lower right")
plt.grid(True)
#plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/xgboost/RoC_Curve.pdf", dpi=600, bbox_inches='tight')
plt.show()

# Plot Precision-Recall curve
precision_xgb, recall_xgb, _ = precision_recall_curve(y_test, xgb_probs)
pr_auc_xgb = average_precision_score(y_test, xgb_probs) # Recalculate PR AUC using test probabilities

plt.figure(figsize=(8, 6))
plt.plot(recall_xgb, precision_xgb, color='darkorange', lw=2, label=f'XGBoost PR curve (AP = {pr_auc_xgb:.2f})')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve for XGBoost (Trained Val, Tested Test)')
plt.legend(loc="lower left")
plt.grid(True)
#plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/xgboost/PR_Curve.pdf", dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

# Define Random Forest classifier
rf = RandomForestClassifier(random_state=42)

# Corrected hyperparameter search space (removed 'weights')
param_grid = {
    'n_estimators': [50, 100, 200, 300],  # Number of trees
    'max_depth': [None, 10, 20, 30],  # Maximum depth
    'min_samples_split': [2, 5, 10],  # Minimum samples required to split
    'min_samples_leaf': [1, 2, 5],  # Minimum samples in a leaf node
    'max_features': ['sqrt', 'log2'],  # Number of features per tree
    'bootstrap': [True, False]  # Whether to bootstrap samples
}

# Perform Randomized Search
random_search = RandomizedSearchCV(rf, param_grid, cv=3, scoring='accuracy', n_iter=10, n_jobs=-1)
random_search.fit(reconstruction_errors_val.reshape(-1, 1), y_val)

# Print best parameters
print("Best Hyperparameters:", random_search.best_params_)


# Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, precision_recall_curve
from sklearn.metrics import confusion_matrix # Import confusion_matrix
import seaborn as sns # Import seaborn

# Assuming you have already computed reconstruction_errors_val, y_val,
# reconstruction_errors_test, and y_test from previous steps.

# Train Random Forest on Validation Set Reconstruction Errors
# You might want to tune the 'n_estimators' and other parameters
# rf = RandomForestClassifier(n_estimators=100, random_state=42)
# Use best hyperparameters found
rf = RandomForestClassifier(
    n_estimators=200,
    min_samples_split=5,
    min_samples_leaf=5,
    max_features='log2',
    max_depth=10,
    bootstrap=False,
    random_state=42
)

rf.fit(reconstruction_errors_val.reshape(-1, 1), y_val) # Train on validation errors and labels

# Predict probabilities on Test Set Reconstruction Errors
rf_probs = rf.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1]
rf_preds = (rf_probs > 0.5).astype(int) # Use a threshold (e.g., 0.5) to get binary predictions

# Evaluation on Test Set
print("Random Forest Classification Report (Trained on Validation, Tested on Test):\n",
      classification_report(y_test, rf_preds, digits=4, zero_division=1))
print("Random Forest ROC AUC (Trained on Validation, Tested on Test):",
      roc_auc_score(y_test, rf_probs))
print("Random Forest Average Precision (PR AUC) (Trained on Validation, Tested on Test):",
      average_precision_score(y_test, rf_probs))

# Calculate Confusion Matrix for Random Forest
cm_rf = confusion_matrix(y_test, rf_preds)

# Draw Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm_rf, annot=True, fmt="d", cmap="Blues",
            xticklabels=['Predicted Non-Dysbiotic', 'Predicted Dysbiotic'],
            yticklabels=['Actual Non-Dysbiotic', 'Actual Dysbiotic'])
plt.title('Confusion Matrix - Random Forest on Reconstruction Error (Trained Val, Tested Test)')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
#plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/RF/confusion_matrix.pdf", dpi=600, bbox_inches='tight')
plt.show()


# Plot ROC curve
fpr_rf, tpr_rf, _ = roc_curve(y_test, rf_probs)
roc_auc_rf = roc_auc_score(y_test, rf_probs) # Recalculate ROC AUC using test probabilities

plt.figure(figsize=(8, 5))
plt.plot(fpr_rf, tpr_rf, color='darkorange', lw=2, label=f'Random Forest ROC curve (AUC = {roc_auc_rf:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic for Random Forest (Trained Val, Tested Test)')
plt.legend(loc="lower right")
plt.grid(True)
#plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/RF/RoC_Curve.pdf", dpi=600, bbox_inches='tight')
plt.show()

# Plot Precision-Recall curve
precision_rf, recall_rf, _ = precision_recall_curve(y_test, rf_probs)
average_precision_rf = average_precision_score(y_test, rf_probs) # Recalculate PR AUC using test probabilities

plt.figure(figsize=(8, 6))
plt.plot(recall_rf, precision_rf, color='darkorange', lw=2, label=f'Random Forest PR curve (AP = {average_precision_rf:.2f})')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve for Random Forest (Trained Val, Tested Test)')
plt.legend(loc="lower left")
plt.grid(True)
#plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/RF/PR_Curve.pdf", dpi=600, bbox_inches='tight')
plt.show()

# Ensemble Learning on the Classifiers

## Averaged Probabilities (Soft Voting)

In [ ]:
# Assuming the following trained models are available from your notebook:
# best_clf (Logistic Regression - trained on reconstruction_errors_val)
# mlp_model (Keras MLP - trained on reconstruction_errors_val)
# ocsvm (One-Class SVM - trained on reconstruction_errors_train, decision_function on test)
# knn (K-Nearest Neighbors - trained on reconstruction_errors_val)
# xgb (XGBoost - trained on reconstruction_errors_val)
# rf (Random Forest - trained on reconstruction_errors_val)

# Ensure reconstruction_errors_test is available and reshaped if needed for models expecting (n_samples, 1) input
reconstruction_errors_test_reshaped = reconstruction_errors_test.reshape(-1, 1)

# 1. Logistic Regression Probabilities
lr_probs = best_clf.predict_proba(reconstruction_errors_test_reshaped)[:, 1]

# 2. Keras MLP Probabilities
# Make sure the MLP model is loaded if necessary
# from tensorflow.keras.models import load_model
# mlp_model = load_model("your_mlp_model.h5") # Load if not already in memory
mlp_probs = mlp_model.predict(reconstruction_errors_test_reshaped).flatten()

# 3. KNN Probabilities
knn_probs = knn.predict_proba(reconstruction_errors_test_reshaped)[:, 1]

# 4. XGBoost Probabilities
xgb_probs = xgb.predict_proba(reconstruction_errors_test_reshaped)[:, 1]

# 5. Random Forest Probabilities
rf_probs = rf.predict_proba(reconstruction_errors_test_reshaped)[:, 1]

# 6. One-Class SVM Scores
# OCSVM's decision_function gives a score. Lower scores are more anomalous.
# For soft voting, we need a probability-like score where higher indicates anomaly.
# We can use the negative of the decision function.
ocsvm_scores = -ocsvm.decision_function(reconstruction_errors_test_reshaped).flatten()

# You might consider scaling the OCSVM scores to be on a similar scale as probabilities (0-1)
# For instance, using MinMaxScaler if needed, but it's not strictly necessary for simple averaging.
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# ocsvm_scaled_scores = scaler.fit_transform(ocsvm_scores.reshape(-1, 1)).flatten()
# We'll use the raw negative scores for now in the ensemble calculation.

In [ ]:
import numpy as np

# Combine probabilities (excluding OCSVM for simple averaging with probability-based models)
# If you want to include OCSVM, you would need a way to convert its score to a probability or
# use a different ensembling approach that handles scores and probabilities.
# Let's average the probabilities from LR, MLP, KNN, XGBoost, and RF for a start.
averaged_probs = np.mean([lr_probs, mlp_probs, knn_probs, xgb_probs, rf_probs], axis=0)

# You can set a threshold on the averaged probabilities to get binary predictions
ensemble_predictions = (averaged_probs > 0.5).astype(int)

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, precision_recall_curve
import seaborn as sns

# Classification Report
print("Ensemble (Averaged Probabilities) Classification Report on Test Set:")
print(classification_report(y_test, ensemble_predictions, target_names=["Non-Dysbiotic", "Dysbiotic"], zero_division=1))

# Compute ROC AUC
roc_auc_ensemble = roc_auc_score(y_test, averaged_probs)
print(f"Ensemble ROC AUC Score: {roc_auc_ensemble:.4f}")

# Compute Average Precision (PR AUC)
average_precision_ensemble = average_precision_score(y_test, averaged_probs)
print(f"Ensemble Average Precision (PR AUC): {average_precision_ensemble:.4f}")

# Calculate Confusion Matrix
cm_ensemble = confusion_matrix(y_test, ensemble_predictions)

# Draw Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm_ensemble, annot=True, fmt="d", cmap="Blues",
            xticklabels=['Predicted Non-Dysbiotic', 'Predicted Dysbiotic'],
            yticklabels=['Actual Non-Dysbiotic', 'Actual Dysbiotic'])
plt.title('Confusion Matrix - Ensemble (Averaged Probabilities)')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
# plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/Ensemble_Avg/confusion_matrix.pdf", dpi=600, bbox_inches='tight')
plt.show()

# Plot ROC curve
fpr_ensemble, tpr_ensemble, _ = roc_curve(y_test, averaged_probs)

plt.figure(figsize=(8, 5))
plt.plot(fpr_ensemble, tpr_ensemble, color='blue', lw=2, label=f'Ensemble ROC curve (AUC = {roc_auc_ensemble:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Ensemble (Averaged Probabilities)')
plt.legend(loc="lower right")
plt.grid(True)
# plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/Ensemble_Avg/RoC_Curve.pdf", dpi=600, bbox_inches='tight')
plt.show()

# Plot Precision-Recall curve
precision_ensemble, recall_ensemble, _ = precision_recall_curve(y_test, averaged_probs)

plt.figure(figsize=(8, 5))
plt.plot(recall_ensemble, precision_ensemble, color='red', lw=2, label=f'Ensemble PR curve (AP = {average_precision_ensemble:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve - Ensemble (Averaged Probabilities)')
plt.legend(loc="lower left")
plt.grid(True)
# plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/Ensemble_Avg/PR_Curve.pdf", dpi=600, bbox_inches='tight')
plt.show()

## Weighted Mechanism (Weighted Averaging) (Soft Voting)



In [ ]:
from sklearn.ensemble import VotingClassifier
# Assuming you have the scikit-learn versions of your trained models:
# best_clf (Logistic Regression)
# sklearn_mlp (if you train a scikit-learn MLP)
# knn (K-Nearest Neighbors)
# xgb (XGBoost)
# rf (Random Forest)

# For One-Class SVM, you would typically not include it in a standard soft voting classifier
# because its decision_function is not directly comparable to probability outputs of others.
# If you want to include its influence, you might use hard voting or a custom ensembling approach.

estimators_sklearn = [
    ('lr', best_clf),
    ('knn', knn),
    ('xgb', xgb),
    ('rf', rf)
    # Add sklearn_mlp if available
    # ('mlp', sklearn_mlp)
]

# Create the Voting Classifier with soft voting
voting_clf = VotingClassifier(estimators=estimators_sklearn, voting='soft', weights=[1, 1, 1, 1]) # Adjust weights as needed

# Train the Voting Classifier (This trains the individual models if not already trained, but
# since yours are trained, it essentially just sets up the combination)
# However, typically, you train the individual models first and then use the VotingClassifier
# to combine their predictions without retraining.
# To use the already trained models' predictions directly in VotingClassifier for evaluation:
# You would fit the VotingClassifier on a small dummy dataset just to enable prediction,
# or more correctly, collect the predictions and combine them manually as shown in the previous steps.

# Let's stick to the manual averaging of probabilities as it directly uses your existing trained models' outputs.
# If you were to use VotingClassifier's fit method, it would re-train the estimators,
# which is not what you want if you've already tuned and trained them on the validation data.

# The manual averaging method shown above is the correct way to ensemble
# the predictions of your already trained models on the test set.

In [ ]:
# Example of weighted averaging (adjust weights based on your analysis)
# You would need to determine these weights based on the performance of each model on the validation set (e.g., F1-score, ROC AUC).
# For example, if XGBoost had the highest F1 on validation, give it a higher weight.
weights = [1.2, 0.8, 1.5, 1.0, 1.1] # Example weights for LR, MLP, KNN, XGBoost, RF
weighted_averaged_probs = np.average([lr_probs, mlp_probs, knn_probs, xgb_probs, rf_probs], axis=0, weights=weights)

weighted_ensemble_predictions = (weighted_averaged_probs > 0.5).astype(int)

# Evaluate the weighted ensemble similarly to the unweighted one.
print("\nWeighted Ensemble Classification Report on Test Set:")
print(classification_report(y_test, weighted_ensemble_predictions, target_names=["Non-Dysbiotic", "Dysbiotic"], zero_division=1))

In [ ]:
# prompt: calculate and plot metrics for the weighted ensemble

# Compute ROC AUC for weighted ensemble
roc_auc_weighted_ensemble = roc_auc_score(y_test, weighted_averaged_probs)
print(f"Weighted Ensemble ROC AUC Score: {roc_auc_weighted_ensemble:.4f}")

# Compute Average Precision (PR AUC) for weighted ensemble
average_precision_weighted_ensemble = average_precision_score(y_test, weighted_averaged_probs)
print(f"Weighted Ensemble Average Precision (PR AUC): {average_precision_weighted_ensemble:.4f}")

# Calculate Confusion Matrix for weighted ensemble
cm_weighted_ensemble = confusion_matrix(y_test, weighted_ensemble_predictions)

# Define output directory for Weighted Ensemble plots
weighted_ensemble_output_dir = "/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/Ensemble_Weighted"
os.makedirs(weighted_ensemble_output_dir, exist_ok=True)


# Draw Confusion Matrix for weighted ensemble
plt.figure(figsize=(8, 6))
sns.heatmap(cm_weighted_ensemble, annot=True, fmt="d", cmap="Blues",
            xticklabels=['Predicted Non-Dysbiotic', 'Predicted Dysbiotic'],
            yticklabels=['Actual Non-Dysbiotic', 'Actual Dysbiotic'])
plt.title('Confusion Matrix - Weighted Ensemble')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.savefig(f"{weighted_ensemble_output_dir}/confusion_matrix.pdf", dpi=600, bbox_inches='tight')
plt.show()

# Plot ROC curve for weighted ensemble
fpr_weighted_ensemble, tpr_weighted_ensemble, _ = roc_curve(y_test, weighted_averaged_probs)

plt.figure(figsize=(8, 5))
plt.plot(fpr_weighted_ensemble, tpr_weighted_ensemble, color='green', lw=2, label=f'Weighted Ensemble ROC curve (AUC = {roc_auc_weighted_ensemble:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Weighted Ensemble')
plt.legend(loc="lower right")
plt.grid(True)
plt.savefig(f"{weighted_ensemble_output_dir}/RoC_Curve.pdf", dpi=600, bbox_inches='tight')
plt.show()

# Plot Precision-Recall curve for weighted ensemble
precision_weighted_ensemble, recall_weighted_ensemble, _ = precision_recall_curve(y_test, weighted_averaged_probs)

plt.figure(figsize=(8, 5))
plt.plot(recall_weighted_ensemble, precision_weighted_ensemble, color='purple', lw=2, label=f'Weighted Ensemble PR curve (AP = {average_precision_weighted_ensemble:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve - Weighted Ensemble')
plt.legend(loc="lower left")
plt.grid(True)
plt.savefig(f"{weighted_ensemble_output_dir}/PR_Curve.pdf", dpi=600, bbox_inches='tight')
plt.show()


In [ ]:
# prompt: Print RoC (AUC) & PR (AUC) of weighted esemble

print(f"RoC (AUC) of Weighted Ensemble: {roc_auc_weighted_ensemble:.4f}")
print(f"PR (AUC) of Weighted Ensemble: {average_precision_weighted_ensemble:.4f}")


# Advanced Ensembling: Stacking

## Stacking with LogisticRegression

In [ ]:
# Generate Out-of-Fold Predictions on the Validation Set
from sklearn.model_selection import StratifiedKFold
import numpy as np

# Assuming you have reconstruction_errors_val and y_val

# Number of folds for cross-validation
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize arrays to store out-of-fold predictions
# Each column will be the predictions from one base model
oof_preds = np.zeros((reconstruction_errors_val.shape[0], 5)) # 5 columns for LR, MLP, KNN, XGB, RF

# List of base models (excluding OCSVM for now as its output is a score, not probability)
base_models = [
    ('lr', best_clf),
    ('mlp', mlp_model), # We'll handle Keras prediction separately
    ('knn', knn),
    ('xgb', xgb),
    ('rf', rf)
]

# Iterate through folds and generate out-of-fold predictions
for fold, (train_index, val_index) in enumerate(skf.split(reconstruction_errors_val.reshape(-1, 1), y_val)):
    print(f"Processing Fold {fold + 1}/{n_splits}")

    X_train_fold, X_val_fold = reconstruction_errors_val[train_index].reshape(-1, 1), reconstruction_errors_val[val_index].reshape(-1, 1)
    y_train_fold, y_val_fold = y_val[train_index], y_val[val_index]

    for i, (name, model) in enumerate(base_models):
        if name == 'mlp':
            # For Keras model, train on the fold data
            # Need to clone and retrain the model for each fold or save/load weights
            # A simpler approach for demonstration: assuming the trained mlp_model is sufficient
            # and we generate predictions on the val_index for this fold.
            # In a proper implementation, you would train a new Keras model on X_train_fold
            # and predict on X_val_fold.
            fold_mlp_preds = model.predict(X_val_fold).flatten()
            oof_preds[val_index, i] = fold_mlp_preds
        else:
            # Train a clone of the scikit-learn model on the fold data
            from sklearn.base import clone
            cloned_model = clone(model)
            cloned_model.fit(X_train_fold, y_train_fold)
            oof_preds[val_index, i] = cloned_model.predict_proba(X_val_fold)[:, 1]

# oof_preds now contains the out-of-fold predictions for each base model on the validation set

In [ ]:
# Combine test predictions from base models
test_preds = np.vstack([lr_probs, mlp_probs, knn_probs, xgb_probs, rf_probs]).T

# test_preds now has shape (n_test_samples, n_base_models)

In [ ]:
from sklearn.linear_model import LogisticRegression

# Define the meta-model
meta_model = LogisticRegression()

# Train the meta-model on the out-of-fold predictions
meta_model.fit(oof_preds, y_val)

In [ ]:
# Make final predictions on the test data using the meta-model
stacked_predictions_probs = meta_model.predict_proba(test_preds)[:, 1]

# Convert probabilities to binary predictions
stacked_predictions = (stacked_predictions_probs > 0.5).astype(int)

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, precision_recall_curve
import seaborn as sns

# Classification Report
print("Stacked Ensemble Classification Report on Test Set:")
print(classification_report(y_test, stacked_predictions, target_names=["Non-Dysbiotic", "Dysbiotic"], zero_division=1))

# Compute ROC AUC
roc_auc_stacked = roc_auc_score(y_test, stacked_predictions_probs)
print(f"Stacked Ensemble ROC AUC Score: {roc_auc_stacked:.4f}")

# Compute Average Precision (PR AUC)
average_precision_stacked = average_precision_score(y_test, stacked_predictions_probs)
print(f"Stacked Ensemble Average Precision (PR AUC): {average_precision_stacked:.4f}")

# Calculate Confusion Matrix
cm_stacked = confusion_matrix(y_test, stacked_predictions)

# Draw Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm_stacked, annot=True, fmt="d", cmap="Blues",
            xticklabels=['Predicted Non-Dysbiotic', 'Predicted Dysbiotic'],
            yticklabels=['Actual Non-Dysbiotic', 'Actual Anomaly'])
plt.title('Confusion Matrix - Stacked Ensemble')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

# Plot ROC curve
fpr_stacked, tpr_stacked, _ = roc_curve(y_test, stacked_predictions_probs)

plt.figure(figsize=(8, 5))
plt.plot(fpr_stacked, tpr_stacked, color='blue', lw=2, label=f'Stacked ROC curve (AUC = {roc_auc_stacked:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Stacked Ensemble')
plt.legend(loc="lower right")
plt.grid(True)
# plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/Ensemble_Stacking/RoC_Curve.pdf", dpi=600, bbox_inches='tight')
plt.show()

# Plot Precision-Recall curve
precision_stacked, recall_stacked, _ = precision_recall_curve(y_test, stacked_predictions_probs)

plt.figure(figsize=(8, 5))
plt.plot(recall_stacked, precision_stacked, color='red', lw=2, label=f'Stacked PR curve (AP = {average_precision_stacked:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve - Stacked Ensemble')
plt.legend(loc="lower left")
plt.grid(True)
# plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/Ensemble_Stacking/PR_Curve.pdf", dpi=600, bbox_inches='tight')
plt.show()

## Stacking with XGBoost Meta-Model

In [ ]:
from xgboost import XGBClassifier

# Define the XGBoost meta-model
# You might want to tune the hyperparameters of the meta-model as well
meta_model_xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# Train the meta-model on the out-of-fold predictions
meta_model_xgb.fit(oof_preds, y_val)

In [ ]:
# Make final predictions on the test data using the XGBoost meta-model
stacked_predictions_probs_xgb = meta_model_xgb.predict_proba(test_preds)[:, 1]

# Convert probabilities to binary predictions
stacked_predictions_xgb = (stacked_predictions_probs_xgb > 0.5).astype(int)

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, precision_recall_curve
import seaborn as sns

# Classification Report
print("Stacked Ensemble (XGBoost Meta-Model) Classification Report on Test Set:")
print(classification_report(y_test, stacked_predictions_xgb, target_names=["Non-Dysbiotic", "Dysbiotic"], zero_division=1))

# Compute ROC AUC
roc_auc_stacked_xgb = roc_auc_score(y_test, stacked_predictions_probs_xgb)
print(f"Stacked Ensemble (XGBoost Meta-Model) ROC AUC Score: {roc_auc_stacked_xgb:.4f}")

# Compute Average Precision (PR AUC)
average_precision_stacked_xgb = average_precision_score(y_test, stacked_predictions_probs_xgb)
print(f"Stacked Ensemble (XGBoost Meta-Model) Average Precision (PR AUC): {average_precision_stacked_xgb:.4f}")

# Calculate Confusion Matrix
cm_stacked_xgb = confusion_matrix(y_test, stacked_predictions_xgb)

# Draw Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm_stacked_xgb, annot=True, fmt="d", cmap="Blues",
            xticklabels=['Predicted Non-Dysbiotic', 'Predicted Dysbiotic'],
            yticklabels=['Actual Non-Dysbiotic', 'Actual Anomaly'])
plt.title('Confusion Matrix - Stacked Ensemble (XGBoost Meta-Model)')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
# plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/Ensemble_Stacking_XGB/confusion_matrix.pdf", dpi=600, bbox_inches='tight')
plt.show()

# Plot ROC curve
fpr_stacked_xgb, tpr_stacked_xgb, _ = roc_curve(y_test, stacked_predictions_probs_xgb)

plt.figure(figsize=(8, 5))
plt.plot(fpr_stacked_xgb, tpr_stacked_xgb, color='blue', lw=2, label=f'Stacked (XGBoost) ROC curve (AUC = {roc_auc_stacked_xgb:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Stacked Ensemble (XGBoost Meta-Model)')
plt.legend(loc="lower right")
plt.grid(True)
# plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/Ensemble_Stacking_XGB/RoC_Curve.pdf", dpi=600, bbox_inches='tight')
plt.show()

# Plot Precision-Recall curve
precision_stacked_xgb, recall_stacked_xgb, _ = precision_recall_curve(y_test, stacked_predictions_probs_xgb)

plt.figure(figsize=(8, 5))
plt.plot(recall_stacked_xgb, precision_stacked_xgb, color='red', lw=2, label=f'Stacked (XGBoost) PR curve (AP = {average_precision_stacked_xgb:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve - Stacked Ensemble (XGBoost Meta-Model)')
plt.legend(loc="lower left")
plt.grid(True)
# plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/Ensemble_Stacking_XGB/PR_Curve.pdf", dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
# prompt: prompt: There is too much overlap between the lines in the ROC (AUC) and PR(AUC). Fix it

import matplotlib.pyplot as plt
# --- B. Combined ROC and PR Plotting ---

# Store ROC and PR curve data for each model
roc_curves_data = {}
pr_curves_data = {}

# Function to collect ROC and PR data
def collect_curve_data(model_name, y_true, y_probs_or_scores, color, linestyle='-'):
    """Collects ROC and PR curve data for a given model's scores/probabilities."""
    # For ROC curve, higher score/prob means higher true positive rate at lower false positive rate
    # For PR curve, higher score/prob means higher precision at higher recall
    # sklearn's roc_curve and precision_recall_curve work correctly with scores directly.

    fpr, tpr, _ = roc_curve(y_true, y_probs_or_scores)
    roc_auc = roc_auc_score(y_true, y_probs_or_scores)
    roc_curves_data[model_name] = {'fpr': fpr, 'tpr': tpr, 'auc': roc_auc, 'color': color, 'linestyle': linestyle}

    precision, recall, _ = precision_recall_curve(y_true, y_probs_or_scores)
    pr_auc = average_precision_score(y_true, y_probs_or_scores)
    pr_curves_data[model_name] = {'precision': precision, 'recall': recall, 'auc': pr_auc, 'color': color, 'linestyle': linestyle}

# Collect data for Original Model (LSTM-AE with optimal threshold)
# Use reconstruction errors as the score directly. Higher error -> more likely anomaly.
collect_curve_data(
    'LSTM-AE (Reconstruction Error)',
    y_test,
    reconstruction_errors_test,
    'darkorange',
    '-'
)

# Collect data for Individual Classifiers (assuming probabilities are available)
# Use predict_proba[:, 1] for standard classifiers
# For OCSVM, use negative scores as the "probability" or score for AUC
collect_curve_data('Logistic Regression', y_test, best_clf.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1], 'blue', '--')
collect_curve_data('MLP', y_test, mlp_model.predict(reconstruction_errors_test.reshape(-1, 1)).flatten(), 'green', '--')
# For OCSVM, use negative decision_function scores. Lower scores are anomalous, so negative makes higher values anomalous.
collect_curve_data('One-Class SVM', y_test, -ocsvm.decision_function(reconstruction_errors_test.reshape(-1, 1)).flatten(), 'red', '--')
collect_curve_data('K-Nearest Neighbors', y_test, knn.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1], 'purple', '--')
collect_curve_data('XGBoost', y_test, xgb.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1], 'brown', '--')
collect_curve_data('Random Forest', y_test, rf.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1], 'pink', '--')


# Collect data for Ensemble Methods (using the averaged/stacked probabilities)
collect_curve_data('Averaged Probabilities Ensemble', y_test, averaged_probs, 'cyan', '-')
collect_curve_data('Weighted Ensemble', y_test, weighted_averaged_probs, 'magenta', '-')
# Use the stacked ensemble probabilities as scores
collect_curve_data('Stacked Ensemble (LR Meta)', y_test, meta_model.predict_proba(test_preds)[:, 1], 'gray', '-')
collect_curve_data('Stacked Ensemble (XGB Meta)', y_test, meta_model_xgb.predict_proba(test_preds)[:, 1], 'olive', '-')


# --- Plotting ---

output_dir_evaluation_plots = "/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/Combined_Evaluation_Plots/"
os.makedirs(output_dir_evaluation_plots, exist_ok=True)

# Figure 1: ROC Curves
plt.figure(figsize=(10, 8))
for model_name, data in roc_curves_data.items():
    plt.plot(data['fpr'], data['tpr'], color=data['color'], linestyle=data['linestyle'],
             label=f'{model_name} (AUC = {data["auc"]:.3f})')

plt.plot([0, 1], [0, 1], 'k--', lw=1)  # Diagonal line
plt.xlabel("False Positive Rate", fontsize=12)
plt.ylabel("True Positive Rate", fontsize=12)
plt.title("Receiver Operating Characteristic (ROC) Curves", fontsize=14)
plt.legend(loc="lower right", fontsize=10)
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.savefig(f"{output_dir_evaluation_plots}/Combined_ROC_Curves.pdf", dpi=600, bbox_inches='tight')
plt.show()

# Figure 2: PR Curves
plt.figure(figsize=(10, 8))
for model_name, data in pr_curves_data.items():
    plt.plot(data['recall'], data['precision'], color=data['color'], linestyle=data['linestyle'],
             label=f'{model_name} (AP = {data["auc"]:.3f})')

plt.xlabel("Recall", fontsize=12)
plt.ylabel("Precision", fontsize=12)
plt.title("Precision-Recall (PR) Curves", fontsize=14)
plt.legend(loc="lower left", fontsize=10)
plt.grid(True, linestyle='--', alpha=0.6)
plt.ylim([0.0, 1.05]) # Standard practice for PR curves
plt.xlim([0.0, 1.0])
plt.tight_layout()
plt.savefig(f"{output_dir_evaluation_plots}/Combined_PR_Curves.pdf", dpi=600, bbox_inches='tight')
plt.show()


In [ ]:
# prompt: Generate RoC (AUC) as original vs All Classifiers except ensemble learning

import matplotlib.pyplot as plt
# --- Plotting ROC Curve for Original Model vs. Individual Classifiers (Excluding Ensemble) ---

# Define the output directory
output_dir_comparison_plots = "/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/Comparison_Plots/"
os.makedirs(output_dir_comparison_plots, exist_ok=True)


plt.figure(figsize=(10, 8))

# Plot ROC Curve for the Original Model (LSTM-AE Reconstruction Error)
# Use reconstruction errors as the score directly. Higher error -> more likely anomaly.
fpr_original, tpr_original, _ = roc_curve(y_test, reconstruction_errors_test)
roc_auc_original = roc_auc_score(y_test, reconstruction_errors_test)
plt.plot(fpr_original, tpr_original, color='darkorange', lw=2, label=f'LSTM-AE (Reconstruction Error) (AUC = {roc_auc_original:.3f})')


# Plot ROC Curves for Individual Classifiers (trained on reconstruction errors)
# Use predict_proba[:, 1] for standard classifiers. For OCSVM, use negative scores.

# Logistic Regression
fpr_lr, tpr_lr, _ = roc_curve(y_test, best_clf.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1])
roc_auc_lr = roc_auc_score(y_test, best_clf.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1])
plt.plot(fpr_lr, tpr_lr, color='blue', linestyle='--', lw=2, label=f'Logistic Regression (AUC = {roc_auc_lr:.3f})')

# MLP
mlp_probs_test = mlp_model.predict(reconstruction_errors_test.reshape(-1, 1)).flatten()
fpr_mlp, tpr_mlp, _ = roc_curve(y_test, mlp_probs_test)
roc_auc_mlp = roc_auc_score(y_test, mlp_probs_test)
plt.plot(fpr_mlp, tpr_mlp, color='green', linestyle='--', lw=2, label=f'MLP (AUC = {roc_auc_mlp:.3f})')

# One-Class SVM
ocsvm_scores_test = -ocsvm.decision_function(reconstruction_errors_test.reshape(-1, 1)).flatten()
fpr_ocsvm, tpr_ocsvm, _ = roc_curve(y_test, ocsvm_scores_test)
roc_auc_ocsvm = roc_auc_score(y_test, ocsvm_scores_test)
plt.plot(fpr_ocsvm, tpr_ocsvm, color='red', linestyle='--', lw=2, label=f'One-Class SVM (AUC = {roc_auc_ocsvm:.3f})')

# K-Nearest Neighbors
knn_probs_test = knn.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1]
fpr_knn, tpr_knn, _ = roc_curve(y_test, knn_probs_test)
roc_auc_knn = roc_auc_score(y_test, knn_probs_test)
plt.plot(fpr_knn, tpr_knn, color='purple', linestyle='--', lw=2, label=f'K-Nearest Neighbors (AUC = {roc_auc_knn:.3f})')

# XGBoost
xgb_probs_test = xgb.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1]
fpr_xgb, tpr_xgb, _ = roc_curve(y_test, xgb_probs_test)
roc_auc_xgb = roc_auc_score(y_test, xgb_probs_test)
plt.plot(fpr_xgb, tpr_xgb, color='brown', linestyle='--', lw=2, label=f'XGBoost (AUC = {roc_auc_xgb:.3f})')

# Random Forest
rf_probs_test = rf.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1]
fpr_rf, tpr_rf, _ = roc_curve(y_test, rf_probs_test)
roc_auc_rf = roc_auc_score(y_test, rf_probs_test)
plt.plot(fpr_rf, tpr_rf, color='pink', linestyle='--', lw=2, label=f'Random Forest (AUC = {roc_auc_rf:.3f})')


# Plot the diagonal reference line
plt.plot([0, 1], [0, 1], 'k--', lw=1)

plt.xlabel("False Positive Rate", fontsize=12)
plt.ylabel("True Positive Rate", fontsize=12)
plt.title("ROC Curves: LSTM-AE vs. Individual Classifiers", fontsize=14)
plt.legend(loc="lower right", fontsize=10)
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.savefig(f"{output_dir_comparison_plots}/ROC_Curve_Original_vs_Individual_Classifiers.pdf", dpi=600, bbox_inches='tight')
plt.show()


In [ ]:
# prompt: Generate RoC (AUC) as original vs all ensemble learning only

import matplotlib.pyplot as plt
# --- Plotting ROC Curve for Original Model vs. Ensemble Learning ---

# Define the output directory for comparison plots if not already defined
output_dir_comparison_plots = "/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/Comparison_Plots/"
os.makedirs(output_dir_comparison_plots, exist_ok=True)


plt.figure(figsize=(10, 8))

# Plot ROC Curve for the Original Model (LSTM-AE Reconstruction Error)
# Use reconstruction errors as the score directly. Higher error -> more likely anomaly.
fpr_original, tpr_original, _ = roc_curve(y_test, reconstruction_errors_test)
roc_auc_original = roc_auc_score(y_test, reconstruction_errors_test)
plt.plot(fpr_original, tpr_original, color='darkorange', lw=2, label=f'LSTM-AE (Reconstruction Error) (AUC = {roc_auc_original:.3f})')


# Plot ROC Curves for Ensemble Models

# Averaged Probabilities Ensemble
fpr_avg_ensemble, tpr_avg_ensemble, _ = roc_curve(y_test, averaged_probs)
roc_auc_avg_ensemble = roc_auc_score(y_test, averaged_probs)
plt.plot(fpr_avg_ensemble, tpr_avg_ensemble, color='cyan', linestyle='-', lw=2, label=f'Averaged Probabilities Ensemble (AUC = {roc_auc_avg_ensemble:.3f})')

# Weighted Ensemble
fpr_weighted_ensemble, tpr_weighted_ensemble, _ = roc_curve(y_test, weighted_averaged_probs)
roc_auc_weighted_ensemble = roc_auc_score(y_test, weighted_averaged_probs)
plt.plot(fpr_weighted_ensemble, tpr_weighted_ensemble, color='magenta', linestyle='-', lw=2, label=f'Weighted Ensemble (AUC = {roc_auc_weighted_ensemble:.3f})')

# Stacked Ensemble (Logistic Regression Meta-Model)
# Assuming test_preds contains the stacked inputs for test data
stacked_predictions_probs_lr_meta = meta_model.predict_proba(test_preds)[:, 1]
fpr_stacked_lr, tpr_stacked_lr, _ = roc_curve(y_test, stacked_predictions_probs_lr_meta)
roc_auc_stacked_lr = roc_auc_score(y_test, stacked_predictions_probs_lr_meta)
plt.plot(fpr_stacked_lr, tpr_stacked_lr, color='gray', linestyle='-', lw=2, label=f'Stacked Ensemble (LR Meta) (AUC = {roc_auc_stacked_lr:.3f})')

# Stacked Ensemble (XGBoost Meta-Model)
# Assuming test_preds contains the stacked inputs for test data
stacked_predictions_probs_xgb_meta = meta_model_xgb.predict_proba(test_preds)[:, 1]
fpr_stacked_xgb, tpr_stacked_xgb, _ = roc_curve(y_test, stacked_predictions_probs_xgb_meta)
roc_auc_stacked_xgb = roc_auc_score(y_test, stacked_predictions_probs_xgb_meta)
plt.plot(fpr_stacked_xgb, tpr_stacked_xgb, color='olive', linestyle='-', lw=2, label=f'Stacked Ensemble (XGB Meta) (AUC = {roc_auc_stacked_xgb:.3f})')


# Plot the diagonal reference line
plt.plot([0, 1], [0, 1], 'k--', lw=1)

plt.xlabel("False Positive Rate", fontsize=12)
plt.ylabel("True Positive Rate", fontsize=12)
plt.title("ROC Curves: LSTM-AE vs. Ensemble Classifiers", fontsize=14)
plt.legend(loc="lower right", fontsize=10)
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.savefig(f"{output_dir_comparison_plots}/ROC_Curve_Original_vs_Ensemble_Classifiers.pdf", dpi=600, bbox_inches='tight')
plt.show()


# Statistical Tests

In [ ]:
!pip install statsmodels

# Statistical Evaluation and Bootstrap Analysis


In [ ]:
# prompt: I have time series data processed into overlapping windows. I used an LSTM Autoencoder (LSTM-AE) to compute reconstruction errors, then trained these classifiers on the errors: Logistic Regression, MLP, One-Class SVM, KNN, XGBoost, and Random Forest.
# I already have prediction probabilities, binary predictions (0/1), and true labels for each model.
# Please do the following:
# Bootstrap Testing (1000 iterations):
# For each model (including LSTM-AE), sample the test set with replacement.
# For each sample, compute ROC AUC, PR AUC, and F1-score.
# Return the metric distributions, mean, and 95% CI.
# Statistical Testing:
# For each model vs LSTM-AE:
# Paired t-test and Wilcoxon test on ROC AUC, PR AUC, and F1-score.
# McNemar’s test on binary predictions.
# Return all p-values.
# Visualizations:
# Box/violin plots of metrics.
# ROC and PR curves with 95% bootstrap confidence bands.
# Summary Table:
# For each model: mean ± 95% CI per metric, p-values vs LSTM-AE, and a “significant?” flag.
# Python Code:
# Use sklearn, numpy, scipy, seaborn, matplotlib.
# Implement all above steps assuming variables like:
# y_true, y_pred_probs_<model>, y_preds_<model>
# Return the complete code and results suitable for a research paper

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.utils import resample
from scipy.stats import ttest_rel, wilcoxon
# from statsmodels.sandbox.stats.runs import mcnemar # Keeping this import, although statsmodels.stats.contingency_tables.mcnemar is used
from statsmodels.stats.contingency_tables import mcnemar
from typing import Dict, Any, List
import seaborn as sns # Ensure seaborn is imported for plotting
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, roc_curve, precision_recall_curve # Ensure all metrics are imported
import os

# Assuming y_test, reconstruction_errors_test, anomaly_predictions, best_clf, mlp_model, ocsvm, knn, xgb, rf are available from previous cells

# Store results in dictionaries
metric_distributions: Dict[str, Dict[str, List[float]]] = {}
metric_summary: Dict[str, Dict[str, Dict[str, Any]]] = {}
statistical_test_results: Dict[str, Dict[str, Dict[str, float]]] = {}

# Define the models and their corresponding probabilities and binary predictions
models = {
    'LSTM-AE': {'probs': reconstruction_errors_test, 'preds': anomaly_predictions}, # Assuming reconstruction_errors_test is the "score" for LSTM-AE
    'Logistic Regression': {'probs': best_clf.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1], 'preds': best_clf.predict(reconstruction_errors_test.reshape(-1, 1))},
    'MLP': {'probs': mlp_model.predict(reconstruction_errors_test.reshape(-1, 1)).flatten(), 'preds': (mlp_model.predict(reconstruction_errors_test.reshape(-1, 1)).flatten() > 0.5).astype(int)},
    'One-Class SVM': {'probs': -ocsvm.decision_function(reconstruction_errors_test.reshape(-1, 1)).flatten(), 'preds': (ocsvm.decision_function(reconstruction_errors_test.reshape(-1, 1)).flatten() < 0).astype(int)}, # Using negative scores for AUC/PR AUC, and thresholding at 0 for binary predictions
    'KNN': {'probs': knn.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1], 'preds': knn.predict(reconstruction_errors_test.reshape(-1, 1))},
    'XGBoost': {'probs': xgb.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1], 'preds': xgb.predict(reconstruction_errors_test.reshape(-1, 1))},
    'Random Forest': {'probs': rf.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1], 'preds': rf.predict(reconstruction_errors_test.reshape(-1, 1))},
    # Add ensemble models if desired
    'Averaged Ensemble': {'probs': averaged_probs, 'preds': ensemble_predictions},
    'Weighted Ensemble': {'probs': weighted_averaged_probs, 'preds': weighted_ensemble_predictions},
    'Stacked (LR Meta)': {'probs': stacked_predictions_probs, 'preds': stacked_predictions},
    'Stacked (XGB Meta)': {'probs': stacked_predictions_probs_xgb, 'preds': stacked_predictions_xgb},
}

n_iterations = 1000
alpha = 0.05 # For confidence intervals and statistical significance

print("Performing Bootstrap Testing...")
for model_name, data in models.items():
    model_probs = data['probs']
    model_preds = data['preds']

    roc_aucs = []
    pr_aucs = []
    f1_scores = []

    # Handle potential issues with OCSVM or other models having constant predictions/scores
    # that would prevent AUC/F1 calculation on some bootstrapped samples.
    # For simplicity, we'll skip iterations where metrics fail.
    successful_iterations = 0
    # Use a fixed random state for reproducibility of bootstrap samples themselves
    bootstrap_rng = np.random.default_rng(seed=42)

    # Store (y_true_sample, probs_sample) for plotting later
    bootstrap_samples_for_plotting: List[tuple[np.ndarray, np.ndarray]] = []


    while successful_iterations < n_iterations:
        # Resample with replacement
        indices = bootstrap_rng.choice(np.arange(len(y_test)), size=len(y_test), replace=True)

        y_true_sample = y_test[indices]
        probs_sample = model_probs[indices]
        preds_sample = model_preds[indices]

        try:
            # Compute metrics for the sample
            # Check for multiple classes in true labels and variation in scores/predictions
            if len(np.unique(y_true_sample)) > 1:
                 if len(np.unique(probs_sample)) > 1:
                    roc_auc_sample = roc_auc_score(y_true_sample, probs_sample)
                    pr_auc_sample = average_precision_score(y_true_sample, probs_sample)
                    roc_aucs.append(roc_auc_sample)
                    pr_aucs.append(pr_auc_sample)
                 # We still try to compute F1 even if probs are constant, as preds might vary
                 if len(np.unique(preds_sample)) > 1:
                      f1_score_sample = f1_score(y_true_sample, preds_sample, zero_division=0)
                      f1_scores.append(f1_score_sample)
            # If only one class in y_true_sample, metrics like AUC, PR AUC, and standard F1 are not well-defined.
            # We append np.nan or skip, skipping is better for distribution calculations later.

            # Store sample for plotting if metrics were attempted (even if they failed)
            # This ensures the lists for plotting match the iterations where we *tried* to compute metrics
            bootstrap_samples_for_plotting.append((y_true_sample, probs_sample))

            successful_iterations += 1 # Increment only if sample was generated and attempted

        except Exception as e:
            # print(f"Skipping iteration {successful_iterations} for {model_name} due to error: {e}")
            # Still store the sample even if metric calculation failed, for consistent plotting sample count
            bootstrap_samples_for_plotting.append((y_true_sample, probs_sample))
            successful_iterations += 1 # Increment even on error to match total iterations


    # Store distributions
    metric_distributions[model_name] = {
        'ROC AUC': roc_aucs,
        'PR AUC': pr_aucs,
        'F1 Score': f1_scores,
        'bootstrap_samples_for_plotting': bootstrap_samples_for_plotting # Store samples for plotting
    }

    # Compute mean and 95% CI
    metric_summary[model_name] = {}
    for metric_name, values in {
        'ROC AUC': roc_aucs,
        'PR AUC': pr_aucs,
        'F1 Score': f1_scores
        }.items():
        if values: # Ensure list is not empty
            mean_val = np.mean(values)
            # Compute confidence interval using percentile method
            lower_bound = np.percentile(values, (alpha / 2) * 100)
            upper_bound = np.percentile(values, 100 - (alpha / 2) * 100)
            metric_summary[model_name][metric_name] = {
                'mean': mean_val,
                'ci_lower': lower_bound,
                'ci_upper': upper_bound,
                'distribution': values # Keep distribution for later plotting
            }
        else:
             metric_summary[model_name][metric_name] = {
                'mean': np.nan,
                'ci_lower': np.nan,
                'ci_upper': np.nan,
                'distribution': []
            }


print("\nPerforming Statistical Testing...")
# Collect bootstrap distributions for LSTM-AE explicitly after the loop finishes
lstm_ae_roc_auc_dist = metric_distributions['LSTM-AE']['ROC AUC']
lstm_ae_pr_auc_dist = metric_distributions['LSTM-AE']['PR AUC']
lstm_ae_f1_dist = metric_distributions['LSTM-AE']['F1 Score']
lstm_ae_preds = models['LSTM-AE']['preds'] # Binary predictions from LSTM-AE thresholding

statistical_test_results['vs LSTM-AE'] = {}

# Statistical Testing Against LSTM-AE

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_rel, wilcoxon
from statsmodels.stats.contingency_tables import mcnemar

for model_name, data in models.items():
    if model_name == 'LSTM-AE':
        continue  # Skip comparison with itself

    model_roc_auc_dist = metric_distributions[model_name]['ROC AUC']
    model_pr_auc_dist = metric_distributions[model_name]['PR AUC']
    model_f1_dist = metric_distributions[model_name]['F1 Score']
    model_preds = data['preds']

    statistical_test_results['vs LSTM-AE'][model_name] = {}

    # Ensure lengths match for paired tests
    min_len_roc = min(len(lstm_ae_roc_auc_dist), len(model_roc_auc_dist))
    if min_len_roc >= 2:
        t_stat_roc, p_value_t_roc = ttest_rel(lstm_ae_roc_auc_dist[:min_len_roc], model_roc_auc_dist[:min_len_roc])
        statistical_test_results['vs LSTM-AE'][model_name]['ROC AUC (T-test p)'] = p_value_t_roc

        # Wilcoxon test fix: Ensure at least 2 distinct values
        unique_values_lstm = len(set(lstm_ae_roc_auc_dist[:min_len_roc]))
        unique_values_model = len(set(model_roc_auc_dist[:min_len_roc]))
        if min_len_roc >= 2 and unique_values_lstm > 1 and unique_values_model > 1:
            try:
                w_stat_roc, p_value_w_roc = wilcoxon(lstm_ae_roc_auc_dist[:min_len_roc], model_roc_auc_dist[:min_len_roc])
                statistical_test_results['vs LSTM-AE'][model_name]['ROC AUC (Wilcoxon p)'] = p_value_w_roc
            except ValueError:
                statistical_test_results['vs LSTM-AE'][model_name]['ROC AUC (Wilcoxon p)'] = np.nan
        else:
            statistical_test_results['vs LSTM-AE'][model_name]['ROC AUC (Wilcoxon p)'] = np.nan
    else:
        statistical_test_results['vs LSTM-AE'][model_name]['ROC AUC (T-test p)'] = np.nan
        statistical_test_results['vs LSTM-AE'][model_name]['ROC AUC (Wilcoxon p)'] = np.nan

    # Repeat fixes for PR AUC and F1 Score
    for metric_name, lstm_dist, model_dist in zip(
        ['PR AUC', 'F1 Score'],
        [lstm_ae_pr_auc_dist, lstm_ae_f1_dist],
        [model_pr_auc_dist, model_f1_dist]
    ):
        min_len = min(len(lstm_dist), len(model_dist))
        if min_len >= 2:
            t_stat, p_value_t = ttest_rel(lstm_dist[:min_len], model_dist[:min_len])
            statistical_test_results['vs LSTM-AE'][model_name][f'{metric_name} (T-test p)'] = p_value_t

            # Wilcoxon test fix
            unique_values_lstm = len(set(lstm_dist[:min_len]))
            unique_values_model = len(set(model_dist[:min_len]))
            if min_len >= 2 and unique_values_lstm > 1 and unique_values_model > 1:
                try:
                    w_stat, p_value_w = wilcoxon(lstm_dist[:min_len], model_dist[:min_len])
                    statistical_test_results['vs LSTM-AE'][model_name][f'{metric_name} (Wilcoxon p)'] = p_value_w
                except ValueError:
                    statistical_test_results['vs LSTM-AE']

# Statistical Comparison of Models

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_rel, wilcoxon, mannwhitneyu
from statsmodels.stats.contingency_tables import mcnemar

# Define the ensemble models
ensemble_models = {
    'Averaged Ensemble': {'probs': averaged_probs, 'preds': ensemble_predictions},
    'Weighted Ensemble': {'probs': weighted_averaged_probs, 'preds': weighted_ensemble_predictions},
    'Stacked (LR Meta)': {'probs': stacked_predictions_probs, 'preds': stacked_predictions},
    'Stacked (XGB Meta)': {'probs': stacked_predictions_probs_xgb, 'preds': stacked_predictions_xgb},
}

# Store statistical test results for ensemble comparisons
ensemble_statistical_test_results: Dict[str, Dict[str, Dict[str, float]]] = {}

print("\nPerforming Statistical Testing Among Ensemble Learners...")

ensemble_model_names = list(ensemble_models.keys())

# Iterate through all unique pairs of ensemble models
for i in range(len(ensemble_model_names)):
    for j in range(i + 1, len(ensemble_model_names)):
        model1_name = ensemble_model_names[i]
        model2_name = ensemble_model_names[j]

        model1_data = ensemble_models[model1_name]
        model2_data = ensemble_models[model2_name]

        model1_roc_auc_dist = np.array(metric_distributions[model1_name]['ROC AUC'], dtype=np.float64)
        model1_pr_auc_dist = np.array(metric_distributions[model1_name]['PR AUC'], dtype=np.float64)
        model1_f1_dist = np.array(metric_distributions[model1_name]['F1 Score'], dtype=np.float64)
        model1_preds = model1_data['preds']

        model2_roc_auc_dist = np.array(metric_distributions[model2_name]['ROC AUC'], dtype=np.float64)
        model2_pr_auc_dist = np.array(metric_distributions[model2_name]['PR AUC'], dtype=np.float64)
        model2_f1_dist = np.array(metric_distributions[model2_name]['F1 Score'], dtype=np.float64)
        model2_preds = model2_data['preds']

        comparison_key = f'{model1_name} vs {model2_name}'
        ensemble_statistical_test_results[comparison_key] = {}

        # Perform paired tests on metric distributions (ROC AUC, PR AUC, F1 Score)
        for metric_name, dist1, dist2 in zip(
            ['ROC AUC', 'PR AUC', 'F1 Score'],
            [model1_roc_auc_dist, model1_pr_auc_dist, model1_f1_dist],
            [model2_roc_auc_dist, model2_pr_auc_dist, model2_f1_dist]
        ):
            min_len = min(len(dist1), len(dist2))
            if min_len >= 2:
                # Paired t-test
                t_stat, p_value_t = ttest_rel(dist1[:min_len], dist2[:min_len])
                ensemble_statistical_test_results[comparison_key][f'{metric_name} (T-test p)'] = p_value_t

                # Wilcoxon test
                if len(set(dist1[:min_len])) > 1 and len(set(dist2[:min_len])) > 1:
                    try:
                        w_stat, p_value_w = wilcoxon(dist1[:min_len], dist2[:min_len])
                        ensemble_statistical_test_results[comparison_key][f'{metric_name} (Wilcoxon p)'] = p_value_w
                    except ValueError:
                        ensemble_statistical_test_results[comparison_key][f'{metric_name} (Wilcoxon p)'] = np.nan
                else:
                    ensemble_statistical_test_results[comparison_key][f'{metric_name} (Wilcoxon p)'] = np.nan

                 # Mann-Whitney U test (alternative)
                u_stat, p_value_u = mannwhitneyu(dist1[:min_len], dist2[:min_len])
                ensemble_statistical_test_results[comparison_key][f'{metric_name} (Mann-Whitney p)'] = p_value_u


            else:
                ensemble_statistical_test_results[comparison_key][f'{metric_name} (T-test p)'] = np.nan
                ensemble_statistical_test_results[comparison_key][f'{metric_name} (Wilcoxon p)'] = np.nan
                ensemble_statistical_test_results[comparison_key][f'{metric_name} (Mann-Whitney p)'] = np.nan


        # McNemar test on binary predictions
        # Create the contingency table with reindex to ensure 2x2 structure
        crosstab_result = pd.crosstab(model1_preds, model2_preds)
        contingency_table_mcnemar = crosstab_result.reindex(index=[0, 1], columns=[0, 1], fill_value=0)

        # Extract the counts for the McNemar table
        n00_val = contingency_table_mcnemar.loc[0, 0]
        n01_val = contingency_table_mcnemar.loc[0, 1]
        n10_val = contingency_table_mcnemar.loc[1, 0]
        n11_val = contingency_table_mcnemar.loc[1, 1]

        table_array_mcnemar = np.array([[n00_val, n01_val],
                                        [n10_val, n11_val]])

        if (n01_val + n10_val) > 0: # Ensure there are discordant pairs for the test to be meaningful
            mcnemar_result = mcnemar(table_array_mcnemar, exact=True)
            ensemble_statistical_test_results[comparison_key]['McNemar p'] = mcnemar_result.pvalue
        else:
            ensemble_statistical_test_results[comparison_key]['McNemar p'] = 1.0 # Perfect agreement on discordant pairs


# Now, format the results into a readable table
print("\nStatistical Test Results Among Ensemble Learners:")

ensemble_comparison_data = []
for comparison, tests in ensemble_statistical_test_results.items():
    row = {'Comparison': comparison}
    for test_name, p_value in tests.items():
         if not np.isnan(p_value):
            row[test_name] = f"{p_value:.4f}"
            row[f'Significant? ({test_name})'] = '*' if p_value < alpha else ''
         else:
            row[test_name] = "N/A"
            row[f'Significant? ({test_name})'] = ''

    ensemble_comparison_data.append(row)

ensemble_comparison_df = pd.DataFrame(ensemble_comparison_data)

# Define a desired column order for clarity
ensemble_ordered_columns = ['Comparison']
for metric_name in ['ROC AUC', 'PR AUC', 'F1 Score']:
    ensemble_ordered_columns.append(f'{metric_name} (T-test p)')
    ensemble_ordered_columns.append(f'Significant? ({metric_name} (T-test p))')
    ensemble_ordered_columns.append(f'{metric_name} (Wilcoxon p)')
    ensemble_ordered_columns.append(f'Significant? ({metric_name} (Wilcoxon p))')
    ensemble_ordered_columns.append(f'{metric_name} (Mann-Whitney p)')
    ensemble_ordered_columns.append(f'Significant? ({metric_name} (Mann-Whitney p))')

ensemble_ordered_columns.append('McNemar p')
ensemble_ordered_columns.append('Significant? (McNemar p)')

# Filter columns to only include those present in the DataFrame
final_ensemble_columns = [col for col in ensemble_ordered_columns if col in ensemble_comparison_df.columns]
ensemble_comparison_df = ensemble_comparison_df[final_ensemble_columns]


print(ensemble_comparison_df.to_string())

# Optionally save this table
output_dir_tables = "/content/drive/MyDrive/Final_Implementations/DynaBiome_Tables/"
os.makedirs(output_dir_tables, exist_ok=True)
ensemble_comparison_df.to_csv(f"{output_dir_tables}/ensemble_statistical_comparison.csv", index=False)

# Prepare for LaTeX
latex_ensemble_df = ensemble_comparison_df.copy()
latex_ensemble_df.columns = latex_ensemble_df.columns.str.replace('_', '\\_')
latex_ensemble_df.columns = latex_ensemble_df.columns.str.replace('%', '\\%')
latex_ensemble_df.columns = latex_ensemble_df.columns.str.replace('(', '{(', regex=False).str.replace(')', ')}')

latex_ensemble_df.to_latex(f"{output_dir_tables}/ensemble_statistical_comparison.tex", index=False, float_format="%.4f", escape=False)

print(f"\nEnsemble statistical comparison table saved to {output_dir_tables}/ensemble_statistical_comparison.csv and .tex")

# Statistical Tests Plots and Summary Table

In [ ]:
# Box/Violin plots of metrics
metric_names = ['ROC AUC', 'PR AUC', 'F1 Score']
for metric_name in metric_names:
    plt.figure(figsize=(12, 6))
    plot_data = []
    model_order = list(models.keys()) # Maintain a consistent order

    for model_name in model_order:
        if metric_name in metric_distributions[model_name] and metric_distributions[model_name][metric_name]:
             plot_data.append(pd.DataFrame({
                 'Model': model_name,
                 metric_name: metric_distributions[model_name][metric_name]
             }))


    if plot_data:
        combined_df = pd.concat(plot_data)
        # Ensure the order is maintained in the plot
        sns.boxplot(x='Model', y=metric_name, data=combined_df, palette='viridis', order=model_order)
        # sns.violinplot(x='Model', y=metric_name, data=combined_df, palette='viridis', inner="quartile", order=model_order) # Alternative violin plot
        plt.title(f'{metric_name} Distribution from Bootstrap ({n_iterations} Iterations)')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        #plt.savefig(f"{output_dir_bootstrap_plots}/{metric_name.replace(' ', '_')}_Bootstrap_BoxPlot.pdf", dpi=600, bbox_inches='tight')
        plt.show()
    else:
        print(f"No data available to plot {metric_name}.")


# Function to plot ROC/PR curve with bootstrap confidence bands
def plot_curve_with_bands(model_name, bootstrap_samples_for_plotting, title_prefix, filename_prefix, curve_type='roc'):
    plt.figure(figsize=(10, 8))

    all_curves_interp = []
    n_bootstrap_success = 0 # Count how many bootstrap samples were successfully used for curves

    if curve_type == 'roc':
        base_fpr = np.linspace(0, 1, 101) # Standard x-axis for interpolation
        for y_true_sample, probs_sample in bootstrap_samples_for_plotting:

            if len(np.unique(y_true_sample)) > 1 and len(np.unique(probs_sample)) > 1:
                 fpr, tpr, _ = roc_curve(y_true_sample, probs_sample)
                 interp_tpr = np.interp(base_fpr, fpr, tpr)
                 interp_tpr[0] = 0.0 # Ensure start at (0,0)
                 all_curves_interp.append(interp_tpr)
                 n_bootstrap_success += 1

        if n_bootstrap_success > 1: # Need at least 2 curves to compute band
            mean_tpr = np.mean(all_curves_interp, axis=0)
            # Using t-distribution for smaller sample sizes, but for n=1000, Z is fine
            # std_tpr = np.std(all_curves_interp, axis=0)
            # ci_width = 1.96 * std_tpr / np.sqrt(n_bootstrap_success)
            # tprs_lower = mean_tpr - ci_width
            # tprs_upper = mean_tpr + ci_width

            # Percentile method for CI band - more robust to non-normal distributions
            tprs_lower = np.percentile(all_curves_interp, (alpha / 2) * 100, axis=0)
            tprs_upper = np.percentile(all_curves_interp, 100 - (alpha / 2) * 100, axis=0)


            tprs_lower = np.maximum(tprs_lower, 0)
            tprs_upper = np.minimum(tprs_upper, 1)

            # Mean AUC from the bootstrap distribution (calculated earlier)
            mean_auc_from_dist = np.mean(metric_distributions[model_name]['ROC AUC']) if metric_distributions[model_name]['ROC AUC'] else np.nan


            plt.plot(base_fpr, mean_tpr, label=f'Mean ROC (AUC = {mean_auc_from_dist:.3f})')
            plt.fill_between(base_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2, label='95% CI')
            plt.plot([0, 1], [0, 1], 'k--', lw=1)
            plt.xlabel("False Positive Rate", fontsize=12)
            plt.ylabel("True Positive Rate", fontsize=12)
            plt.title(f'{title_prefix} ROC Curve with Bootstrap CI', fontsize=14)
            plt.legend(loc="lower right", fontsize=10)
        else:
             print(f"Not enough successful bootstrap iterations ({n_bootstrap_success}) to plot ROC CI band for {title_prefix}.")
             # Plot the curve on the original test set if band cannot be plotted
             # Check y_test directly for original plot
             if len(np.unique(y_test)) > 1 and len(np.unique(models[model_name]['probs'])) > 1:
                 fpr, tpr, _ = roc_curve(y_test, models[model_name]['probs'])
                 auc_orig = roc_auc_score(y_test, models[model_name]['probs'])
                 plt.plot(fpr, tpr, label=f'ROC (AUC = {auc_orig:.3f})')
                 plt.plot([0, 1], [0, 1], 'k--', lw=1)
                 plt.xlabel("False Positive Rate", fontsize=12)
                 plt.ylabel("True Positive Rate", fontsize=12)
                 plt.title(f'{title_prefix} ROC Curve (No CI band)', fontsize=14)
                 plt.legend(loc="lower right", fontsize=10)
             else:
                  print(f"Skipping ROC plot for {title_prefix} - not enough unique values in original data.")


    elif curve_type == 'pr':
         base_recall = np.linspace(0, 1, 101)
         all_precisions_interp = []
         n_bootstrap_success = 0

         for y_true_sample, probs_sample in bootstrap_samples_for_plotting:

            if len(np.unique(y_true_sample)) > 1 and len(np.unique(probs_sample)) > 1:
                precision, recall, _ = precision_recall_curve(y_true_sample, probs_sample)
                # Sort by recall for interpolation
                sort_indices = np.argsort(recall)
                recall = recall[sort_indices]
                precision = precision[sort_indices]

                # Handle interpolation carefully, especially at low recall
                # Append (0, 1) point if not present
                if recall[0] > 0:
                    recall = np.insert(recall, 0, 0)
                    precision = np.insert(precision, 0, 1)

                # Append (proportion of positives, proportion of positives) at recall 1 if not present
                # or ensure the last point is handled
                if recall[-1] < 1.0:
                     prop_pos = np.sum(y_true_sample) / len(y_true_sample) if len(y_true_sample) > 0 else 0
                     recall = np.append(recall, 1.0)
                     precision = np.append(precision, prop_pos) # Precision at recall 1 is proportion of positives

                interp_precision = np.interp(base_recall, recall, precision)

                all_precisions_interp.append(interp_precision)
                n_bootstrap_success += 1

         if n_bootstrap_success > 1: # Need at least 2 curves to compute band
            mean_precision = np.mean(all_precisions_interp, axis=0)
            # std_precision = np.std(all_precisions_interp, axis=0)
            # ci_width_pr = 1.96 * std_precision / np.sqrt(n_bootstrap_success)
            # precisions_lower = mean_precision - ci_width_pr
            # precisions_upper = mean_precision + ci_width_pr

            # Percentile method for CI band
            precisions_lower = np.percentile(all_precisions_interp, (alpha / 2) * 100, axis=0)
            precisions_upper = np.percentile(all_precisions_interp, 100 - (alpha / 2) * 100, axis=0)

            precisions_lower = np.maximum(precisions_lower, 0)
            precisions_upper = np.minimum(precisions_upper, 1)

            # Mean AP from the bootstrap distribution (calculated earlier)
            mean_ap_from_dist = np.mean(metric_distributions[model_name]['PR AUC']) if metric_distributions[model_name]['PR AUC'] else np.nan


            plt.plot(base_recall, mean_precision, label=f'Mean PR (AP = {mean_ap_from_dist:.3f})')
            plt.fill_between(base_recall, precisions_lower, precisions_upper, color='grey', alpha=.2, label='95% CI')
            plt.xlabel("Recall", fontsize=12)
            plt.ylabel("Precision", fontsize=12)
            plt.title(f'{title_prefix} PR Curve with Bootstrap CI', fontsize=14)
            plt.legend(loc="lower left", fontsize=10)
            plt.ylim([0.0, 1.05])
            plt.xlim([0.0, 1.0])
         else:
              print(f"Not enough successful bootstrap iterations ({n_bootstrap_success}) to plot PR CI band for {title_prefix}.")
              # Plot the curve on the original test set if band cannot be plotted
              if len(np.unique(y_test)) > 1 and len(np.unique(models[model_name]['probs'])) > 1:
                precision, recall, _ = precision_recall_curve(y_test, models[model_name]['probs'])
                ap_orig = average_precision_score(y_test, models[model_name]['probs'])
                plt.plot(recall, precision, label=f'PR (AP = {ap_orig:.3f})')
                plt.xlabel("Recall", fontsize=12)
                plt.ylabel("Precision", fontsize=12)
                plt.title(f'{title_prefix} PR Curve (No CI band)', fontsize=14)
                plt.legend(loc="lower left", fontsize=10)
                plt.ylim([0.0, 1.05])
                plt.xlim([0.0, 1.0])
              else:
                   print(f"Skipping PR plot for {title_prefix} - not enough unique values in original data.")


    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    #plt.savefig(f"{output_dir_bootstrap_plots}/{filename_prefix}_{curve_type.upper()}_Curve_Bootstrap_CI.pdf", dpi=600, bbox_inches='tight')
    plt.show()


# Plot curves with bands for each model
for model_name in models.keys():
     plot_curve_with_bands(
         model_name,
         metric_distributions[model_name]['bootstrap_samples_for_plotting'],
         model_name,
         model_name.replace(' ', '_').replace('-', '_'),
         curve_type='roc'
         )
     plot_curve_with_bands(
         model_name,
         metric_distributions[model_name]['bootstrap_samples_for_plotting'],
         model_name,
         model_name.replace(' ', '_').replace('-', '_'),
         curve_type='pr'
         )


print("\nGenerating Summary Table...")

summary_data = []
for model_name in models.keys():
    row: Dict[str, Any] = {'Model': model_name}

    for metric_name in metric_names:
        summary_info = metric_summary[model_name][metric_name]
        mean = summary_info['mean']
        ci_lower = summary_info['ci_lower']
        ci_upper = summary_info['ci_upper']

        if not np.isnan(mean):
             row[f'{metric_name} Mean ± 95% CI'] = f"{mean:.3f} ± ({mean - ci_lower:.3f}, {ci_upper - mean:.3f})"
        else:
             row[f'{metric_name} Mean ± 95% CI'] = "N/A"


    if model_name != 'LSTM-AE':
        vs_lstm_ae_tests = statistical_test_results['vs LSTM-AE'][model_name]
        for test_name, p_value in vs_lstm_ae_tests.items():
            if not np.isnan(p_value):
                row[f'{test_name} (vs LSTM-AE)'] = f"{p_value:.4f}"
                # Add significance flag (p < alpha)
                row[f'Significant? ({test_name})'] = '*' if p_value < alpha else ''
            else:
                 row[f'{test_name} (vs LSTM-AE)'] = "N/A"
                 row[f'Significant? ({test_name})'] = ''

    summary_data.append(row)

summary_df = pd.DataFrame(summary_data)

# Reorder columns for better readability
ordered_columns = ['Model']
for metric_name in metric_names:
    ordered_columns.append(f'{metric_name} Mean ± 95% CI')
    if metric_name != 'F1 Score': # Only ROC and PR have T-test and Wilcoxon on distribution
        ordered_columns.append(f'{metric_name} (T-test p) (vs LSTM-AE)')
        ordered_columns.append(f'Significant? ({metric_name} (T-test p))')
        ordered_columns.append(f'{metric_name} (Wilcoxon p) (vs LSTM-AE)')
        ordered_columns.append(f'Significant? ({metric_name} (Wilcoxon p))')
    else: # F1 score has T-test and Wilcoxon
        ordered_columns.append(f'{metric_name} (T-test p) (vs LSTM-AE)')
        ordered_columns.append(f'Significant? ({metric_name} (T-test p))')
        ordered_columns.append(f'{metric_name} (Wilcoxon p) (vs LSTM-AE)')
        ordered_columns.append(f'Significant? ({metric_name} (Wilcoxon p))')


ordered_columns.append('McNemar p (vs LSTM-AE)')
ordered_columns.append('Significant? (McNemar p)')

# Ensure only existing columns are included (handles LSTM-AE which has no vs tests)
final_columns = [col for col in ordered_columns if col in summary_df.columns]
summary_df = summary_df[final_columns]


print("\nSummary Table:")
print(summary_df.to_string()) # Use to_string to display the full table without truncation

# Save the summary table to a file
output_dir_tables = "/content/drive/MyDrive/Final_Implementations/DynaBiome_Tables/"
os.makedirs(output_dir_tables, exist_ok=True)
summary_df.to_csv(f"{output_dir_tables}/model_performance_summary.csv", index=False)
summary_df.to_latex(f"{output_dir_tables}/model_performance_summary.tex", index=False, float_format="%.3f")

print("\nAnalysis Complete.")

In [ ]:
# prompt: Suggestions for Improvement or Extension
# 1. Enhance Boxplot Aesthetics:
# Consider adding swarm/strip plots for better visualization of distribution density:
# sns.boxplot(...)  # Existing line
# sns.stripplot(x='Model', y=metric_name, data=combined_df, color='black', alpha=0.3, jitter=True, size=3)
# 2. Add Effect Sizes to Summary Table:
# P-values tell you if a difference exists, not how large it is.
# •	Include Cohen’s d or Cliff’s Delta for effect size on continuous metrics.
# •	For paired comparisons (e.g., ROC AUCs), Cohen’s d can be added easily:
# def cohen_d(x, y):
#     return (np.mean(x) - np.mean(y)) / np.sqrt((np.std(x, ddof=1) ** 2 + np.std(y, ddof=1) ** 2) / 2)
# 3. Show Sample Size in Plots:
# Add bootstrap sample count directly in the titles:
# plt.title(f'{title_prefix} ROC Curve with Bootstrap CI\n(N = {n_bootstrap_success} bootstrap samples)', fontsize=14)
# 4. Make Alpha Adjustable:
# Instead of a hardcoded alpha, you could parameterize it:
# def plot_curve_with_bands(..., alpha=0.05):
# 5. Save Plots in Multiple Formats (PDF, PNG):
# To ensure compatibility across journals and presentations:
# plt.savefig(f"{output_dir}/{filename}.pdf", dpi=600)
# plt.savefig(f"{output_dir}/{filename}.png", dpi=300)
# Optional: Parallelize Bootstrap Computation
# If bootstrap iteration is time-consuming, you could parallelize curve computation using joblib:
# from joblib import Parallel, delayed
# def compute_bootstrap_curve(...):  # Wrapper function
#     ...
# results = Parallel(n_jobs=-1)(delayed(compute_bootstrap_curve)(...) for _ in range(n_iterations))
# Optional: Include Metrics Distribution in Summary Table
# Add SD or full distribution stats:
# row[f'{metric_name} Std'] = f"{np.std(metric_distributions[model_name][metric_name]):.3f}"

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Enhance Boxplot Aesthetics: Add swarm/strip plots
metric_names = ['ROC AUC', 'PR AUC', 'F1 Score']
output_dir_bootstrap_plots = "/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/Bootstrap_Plots/"
os.makedirs(output_dir_bootstrap_plots, exist_ok=True)

for metric_name in metric_names:
    plt.figure(figsize=(12, 6))
    plot_data = []
    model_order = list(models.keys())  # Maintain a consistent order

    for model_name in model_order:
        if metric_name in metric_distributions[model_name] and metric_distributions[model_name][metric_name]:
            plot_data.append(pd.DataFrame({
                'Model': model_name,
                metric_name: metric_distributions[model_name][metric_name]
            }))

    if plot_data:
        combined_df = pd.concat(plot_data)
        # Ensure the order is maintained in the plot
        sns.boxplot(x='Model', y=metric_name, data=combined_df, palette='viridis', order=model_order)
        # Add stripplot for individual data points
        sns.stripplot(x='Model', y=metric_name, data=combined_df, color='black', alpha=0.3, jitter=True, size=3, order=model_order)

        plt.title(f'{metric_name} Distribution from Bootstrap ({n_iterations} Iterations)')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.savefig(f"{output_dir_bootstrap_plots}/{metric_name.replace(' ', '_')}_Bootstrap_BoxPlot_Stripplot.pdf", dpi=600, bbox_inches='tight')
        plt.show()
    else:
        print(f"No data available to plot {metric_name}.")


# Modify plot_curve_with_bands to show sample size in title
def plot_curve_with_bands(model_name, bootstrap_samples_for_plotting, title_prefix, filename_prefix, curve_type='roc', n_bootstrap_total=None):
    plt.figure(figsize=(10, 8))

    all_curves_interp = []
    n_bootstrap_success = 0  # Count how many bootstrap samples were successfully used for curves

    if curve_type == 'roc':
        base_fpr = np.linspace(0, 1, 101)  # Standard x-axis for interpolation
        for y_true_sample, probs_sample in bootstrap_samples_for_plotting:
            if len(np.unique(y_true_sample)) > 1 and len(np.unique(probs_sample)) > 1:
                fpr, tpr, _ = roc_curve(y_true_sample, probs_sample)
                interp_tpr = np.interp(base_fpr, fpr, tpr)
                interp_tpr[0] = 0.0  # Ensure start at (0,0)
                all_curves_interp.append(interp_tpr)
                n_bootstrap_success += 1

        if n_bootstrap_success > 1:  # Need at least 2 curves to compute band
            mean_tpr = np.mean(all_curves_interp, axis=0)
            tprs_lower = np.percentile(all_curves_interp, (alpha / 2) * 100, axis=0)
            tprs_upper = np.percentile(all_curves_interp, 100 - (alpha / 2) * 100, axis=0)

            tprs_lower = np.maximum(tprs_lower, 0)
            tprs_upper = np.minimum(tprs_upper, 1)

            mean_auc_from_dist = np.mean(metric_distributions[model_name]['ROC AUC']) if metric_distributions[model_name]['ROC AUC'] else np.nan

            plt.plot(base_fpr, mean_tpr, label=f'Mean ROC (AUC = {mean_auc_from_dist:.3f})')
            plt.fill_between(base_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2, label='95% CI')
            plt.plot([0, 1], [0, 1], 'k--', lw=1)
            plt.xlabel("False Positive Rate", fontsize=12)
            plt.ylabel("True Positive Rate", fontsize=12)
            # Include sample size in title
            plt.title(f'{title_prefix} ROC Curve with Bootstrap CI\n(N = {n_bootstrap_total} bootstrap samples, {n_bootstrap_success} successful)', fontsize=14)
            plt.legend(loc="lower right", fontsize=10)
        else:
            print(f"Not enough successful bootstrap iterations ({n_bootstrap_success}) to plot ROC CI band for {title_prefix}.")
            # Plot the curve on the original test set if band cannot be plotted
            if len(np.unique(y_test)) > 1 and len(np.unique(models[model_name]['probs'])) > 1:
                fpr, tpr, _ = roc_curve(y_test, models[model_name]['probs'])
                auc_orig = roc_auc_score(y_test, models[model_name]['probs'])
                plt.plot(fpr, tpr, label=f'ROC (AUC = {auc_orig:.3f})')
                plt.plot([0, 1], [0, 1], 'k--', lw=1)
                plt.xlabel("False Positive Rate", fontsize=12)
                plt.ylabel("True Positive Rate", fontsize=12)
                plt.title(f'{title_prefix} ROC Curve (No CI band)\n(N = {len(y_test)} test samples)', fontsize=14) # Use test set size for this case
                plt.legend(loc="lower right", fontsize=10)
            else:
                print(f"Skipping ROC plot for {title_prefix} - not enough unique values in original data.")


    elif curve_type == 'pr':
        base_recall = np.linspace(0, 1, 101)
        all_precisions_interp = []
        n_bootstrap_success = 0

        for y_true_sample, probs_sample in bootstrap_samples_for_plotting:
            if len(np.unique(y_true_sample)) > 1 and len(np.unique(probs_sample)) > 1:
                precision, recall, _ = precision_recall_curve(y_true_sample, probs_sample)
                sort_indices = np.argsort(recall)
                recall = recall[sort_indices]
                precision = precision[sort_indices]

                if recall[0] > 0:
                    recall = np.insert(recall, 0, 0)
                    precision = np.insert(precision, 0, 1)

                if recall[-1] < 1.0:
                    prop_pos = np.sum(y_true_sample) / len(y_true_sample) if len(y_true_sample) > 0 else 0
                    recall = np.append(recall, 1.0)
                    precision = np.append(precision, prop_pos)

                interp_precision = np.interp(base_recall, recall, precision)

                all_precisions_interp.append(interp_precision)
                n_bootstrap_success += 1

        if n_bootstrap_success > 1:  # Need at least 2 curves to compute band
            mean_precision = np.mean(all_precisions_interp, axis=0)
            precisions_lower = np.percentile(all_precisions_interp, (alpha / 2) * 100, axis=0)
            precisions_upper = np.percentile(all_precisions_interp, 100 - (alpha / 2) * 100, axis=0)

            precisions_lower = np.maximum(precisions_lower, 0)
            precisions_upper = np.minimum(precisions_upper, 1)

            mean_ap_from_dist = np.mean(metric_distributions[model_name]['PR AUC']) if metric_distributions[model_name]['PR AUC'] else np.nan

            plt.plot(base_recall, mean_precision, label=f'Mean PR (AP = {mean_ap_from_dist:.3f})')
            plt.fill_between(base_recall, precisions_lower, precisions_upper, color='grey', alpha=.2, label='95% CI')
            plt.xlabel("Recall", fontsize=12)
            plt.ylabel("Precision", fontsize=12)
            # Include sample size in title
            plt.title(f'{title_prefix} PR Curve with Bootstrap CI\n(N = {n_bootstrap_total} bootstrap samples, {n_bootstrap_success} successful)', fontsize=14)
            plt.legend(loc="lower left", fontsize=10)
            plt.ylim([0.0, 1.05])
            plt.xlim([0.0, 1.0])
        else:
            print(f"Not enough successful bootstrap iterations ({n_bootstrap_success}) to plot PR CI band for {title_prefix}.")
            # Plot the curve on the original test set if band cannot be plotted
            if len(np.unique(y_test)) > 1 and len(np.unique(models[model_name]['probs'])) > 1:
                precision, recall, _ = precision_recall_curve(y_test, models[model_name]['probs'])
                ap_orig = average_precision_score(y_test, models[model_name]['probs'])
                plt.plot(recall, precision, label=f'PR (AP = {ap_orig:.3f})')
                plt.xlabel("Recall", fontsize=12)
                plt.ylabel("Precision", fontsize=12)
                plt.title(f'{title_prefix} PR Curve (No CI band)\n(N = {len(y_test)} test samples)', fontsize=14) # Use test set size for this case
                plt.legend(loc="lower left", fontsize=10)
                plt.ylim([0.0, 1.05])
                plt.xlim([0.0, 1.0])
            else:
                print(f"Skipping PR plot for {title_prefix} - not enough unique values in original data.")

    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.savefig(f"{output_dir_bootstrap_plots}/{filename_prefix}_{curve_type.upper()}_Curve_Bootstrap_CI.pdf", dpi=600, bbox_inches='tight')
    plt.show()

# Plot curves with bands using the modified function
for model_name in models.keys():
    plot_curve_with_bands(
        model_name,
        metric_distributions[model_name]['bootstrap_samples_for_plotting'],
        model_name,
        model_name.replace(' ', '_').replace('-', '_'),
        curve_type='roc',
        n_bootstrap_total=n_iterations # Pass the total number of iterations
    )
    plot_curve_with_bands(
        model_name,
        metric_distributions[model_name]['bootstrap_samples_for_plotting'],
        model_name,
        model_name.replace(' ', '_').replace('-', '_'),
        curve_type='pr',
        n_bootstrap_total=n_iterations # Pass the total number of iterations
    )

# Add Effect Sizes to Summary Table: Include Cohen’s d
# Cohen's d for paired samples (appropriate for paired bootstrap samples)
# We'll compare each model's bootstrap distribution to the LSTM-AE's distribution

def cohen_d_paired(x, y):
    """Calculates Cohen's d for paired samples."""
    diff = np.array(x) - np.array(y)
    mean_diff = np.mean(diff)
    std_diff = np.std(diff, ddof=1) # Use ddof=1 for sample standard deviation
    # Handle case where std_diff is zero
    if std_diff == 0:
        return np.inf if mean_diff != 0 else 0.0
    return mean_diff / std_diff

print("\nCalculating Effect Sizes (Cohen's d vs LSTM-AE) and updating Summary Table...")

# Update summary_data list with effect sizes
summary_data = []
for model_name in models.keys():
    row: Dict[str, Any] = {'Model': model_name}

    for metric_name in metric_names:
        summary_info = metric_summary[model_name][metric_name]
        mean = summary_info['mean']
        ci_lower = summary_info['ci_lower']
        ci_upper = summary_info['ci_upper']
        distribution = summary_info['distribution'] # Get the distribution

        if not np.isnan(mean):
            row[f'{metric_name} Mean (95% CI)'] = f"{mean:.3f} ({ci_lower:.3f} - {ci_upper:.3f})"
        else:
            row[f'{metric_name} Mean (95% CI)'] = "N/A"

        # Calculate Cohen's d vs LSTM-AE if not the LSTM-AE model and distributions exist
        if model_name != 'LSTM-AE' and distribution and metric_distributions['LSTM-AE'][metric_name]:
             lstm_ae_dist = metric_distributions['LSTM-AE'][metric_name]
             min_len = min(len(distribution), len(lstm_ae_dist))

             if min_len >= 2: # Need at least 2 samples for Cohen's d
                # Use the truncated distributions for Cohen's d to match the paired test length
                d = cohen_d_paired(distribution[:min_len], lstm_ae_dist[:min_len])
                row[f'{metric_name} Cohen\'s d (vs LSTM-AE)'] = f"{d:.3f}"
             else:
                 row[f'{metric_name} Cohen\'s d (vs LSTM-AE)'] = "N/A"
        else:
            row[f'{metric_name} Cohen\'s d (vs LSTM-AE)'] = "N/A"


    if model_name != 'LSTM-AE':
        vs_lstm_ae_tests = statistical_test_results['vs LSTM-AE'][model_name]
        for test_name, p_value in vs_lstm_ae_tests.items():
            if not np.isnan(p_value):
                row[f'{test_name} (vs LSTM-AE)'] = f"{p_value:.4f}"
                row[f'Sig ({test_name})'] = '*' if p_value < alpha else ''
            else:
                row[f'{test_name} (vs LSTM-AE)'] = "N/A"
                row[f'Sig ({test_name})'] = ''

    summary_data.append(row)

summary_df_updated = pd.DataFrame(summary_data)

# Reorder columns for better readability with effect sizes
ordered_columns_updated = ['Model']
for metric_name in metric_names:
    ordered_columns_updated.append(f'{metric_name} Mean (95% CI)')
    ordered_columns_updated.append(f'{metric_name} Cohen\'s d (vs LSTM-AE)')
    # Add p-values and significance flags for statistical tests related to this metric
    if metric_name != 'F1 Score':
         ordered_columns_updated.append(f'{metric_name} (T-test p) (vs LSTM-AE)')
         ordered_columns_updated.append(f'Sig ({metric_name} (T-test p))')
         ordered_columns_updated.append(f'{metric_name} (Wilcoxon p) (vs LSTM-AE)')
         ordered_columns_updated.append(f'Sig ({metric_name} (Wilcoxon p))')
         ordered_columns_updated.append(f'{metric_name} (Mann-Whitney p) (vs LSTM-AE)')
         ordered_columns_updated.append(f'Sig ({metric_name} (Mann-Whitney p))')

    else:
        ordered_columns_updated.append(f'{metric_name} (T-test p) (vs LSTM-AE)')
        ordered_columns_updated.append(f'Sig ({metric_name} (T-test p))')
        ordered_columns_updated.append(f'{metric_name} (Wilcoxon p) (vs LSTM-AE)')
        ordered_columns_updated.append(f'Sig ({metric_name} (Wilcoxon p))')
        ordered_columns_updated.append(f'{metric_name} (Mann-Whitney p) (vs LSTM-AE)')
        ordered_columns_updated.append(f'Sig ({metric_name} (Mann-Whitney p))')


# Add McNemar test results at the end
ordered_columns_updated.append('McNemar p (vs LSTM-AE)')
ordered_columns_updated.append('Sig (McNemar p)')


# Ensure only existing columns are included
final_columns_updated = [col for col in ordered_columns_updated if col in summary_df_updated.columns]
summary_df_updated = summary_df_updated[final_columns_updated]


print("\nUpdated Summary Table with Effect Sizes and Detailed Statistical Tests:")
print(summary_df_updated.to_string())

# Save the updated summary table
summary_df_updated.to_csv(f"{output_dir_tables}/model_performance_summary_with_effect_sizes.csv", index=False)
# Escape underscores for LaTeX
summary_df_updated.columns = summary_df_updated.columns.str.replace('_', '\\_')
summary_df_updated.to_latex(f"{output_dir_tables}/model_performance_summary_with_effect_sizes.tex", index=False, float_format="%.3f", escape=False)

In [ ]:
# prompt: Compose the last updated summary table either in csv or best format for readability or interpretation

import pandas as pd
import numpy as np
# Generate the summary table from the collected data
summary_data = []
for model_name in models.keys():
    row = {'Model': model_name}
    # Add Mean and 95% CI for each metric
    for metric_name in metric_names:
        summary_info = metric_summary[model_name][metric_name]
        mean = summary_info['mean']
        ci_lower = summary_info['ci_lower']
        ci_upper = summary_info['ci_upper']

        if not np.isnan(mean):
            # Format as "Mean (Lower CI - Upper CI)" for clarity
            row[f'{metric_name} Mean (95% CI)'] = f"{mean:.3f} ({ci_lower:.3f} - {ci_upper:.3f})"
        else:
            row[f'{metric_name} Mean (95% CI)'] = "N/A"

    # Add Cohen's d vs LSTM-AE
    for metric_name in metric_names:
        cohen_d_key = f'{metric_name} Cohen\'s d (vs LSTM-AE)'
        if cohen_d_key in summary_df_updated.columns: # Check if the column exists (will not exist for LSTM-AE row)
            cohen_d_value = summary_df_updated.loc[summary_df_updated['Model'] == model_name, cohen_d_key].iloc[0]
            row[cohen_d_key] = cohen_d_value
        else:
             row[cohen_d_key] = "N/A"


    # Add Statistical Test p-values and Significance flags vs LSTM-AE
    if model_name != 'LSTM-AE':
        vs_lstm_ae_tests = statistical_test_results['vs LSTM-AE'][model_name]
        for test_name, p_value in vs_lstm_ae_tests.items():
            # Extract base metric name and test type
            parts = test_name.rsplit(' ', 3) # Split from right 3 times
            if len(parts) == 4: # Expected format "Metric Name (Test Type p)"
                 metric_base_name = parts[0]
                 test_type_part = f'{parts[1]} {parts[2]}' # e.g., "(T-test p)" or "(Wilcoxon p)"
                 test_column_name = f'{metric_base_name} {test_type_part} (vs LSTM-AE)'
                 sig_column_name = f'Sig ({metric_base_name} {test_type_part})'

                 if test_column_name in summary_df_updated.columns:
                    row[test_column_name] = summary_df_updated.loc[summary_df_updated['Model'] == model_name, test_column_name].iloc[0]
                    row[sig_column_name] = summary_df_updated.loc[summary_df_updated['Model'] == model_name, sig_column_name].iloc[0]
            elif test_name.startswith('McNemar'): # Special case for McNemar
                test_column_name = 'McNemar p (vs LSTM-AE)'
                sig_column_name = 'Sig (McNemar p)'
                if test_column_name in summary_df_updated.columns:
                    row[test_column_name] = summary_df_updated.loc[summary_df_updated['Model'] == model_name, test_column_name].iloc[0]
                    row[sig_column_name] = summary_df_updated.loc[summary_df_updated['Model'] == model_name, sig_column_name].iloc[0]


    summary_data.append(row)


final_summary_df = pd.DataFrame(summary_data)

# Reorder columns for the final table structure
# Ensure all potential columns from the previous calculations are included
potential_columns = ['Model']
for metric_name in metric_names:
    potential_columns.append(f'{metric_name} Mean (95% CI)')
    potential_columns.append(f'{metric_name} Cohen\'s d (vs LSTM-AE)')
    potential_columns.append(f'{metric_name} (T-test p) (vs LSTM-AE)')
    potential_columns.append(f'Sig ({metric_name} (T-test p))')
    potential_columns.append(f'{metric_name} (Wilcoxon p) (vs LSTM-AE)')
    potential_columns.append(f'Sig ({metric_name} (Wilcoxon p))')
    potential_columns.append(f'{metric_name} (Mann-Whitney p) (vs LSTM-AE)')
    potential_columns.append(f'Sig ({metric_name} (Mann-Whitney p))') # Ensure Mann-Whitney is included

potential_columns.append('McNemar p (vs LSTM-AE)')
potential_columns.append('Sig (McNemar p)')


# Filter columns to only include those present in the final_summary_df
final_columns_ordered = [col for col in potential_columns if col in final_summary_df.columns]
final_summary_df = final_summary_df[final_columns_ordered]


print("\nFinal Comprehensive Summary Table:")
# Display the table, preventing truncation
print(final_summary_df.to_string())

# Save the final comprehensive table
final_summary_df.to_csv(f"{output_dir_tables}/final_model_performance_summary.csv", index=False)

# Prepare for LaTeX: Escape special characters, especially underscores in column names
latex_df = final_summary_df.copy()
latex_df.columns = latex_df.columns.str.replace('_', '\\_')
latex_df.columns = latex_df.columns.str.replace('%', '\\%') # Escape % as well
latex_df.columns = latex_df.columns.str.replace('(', '{(').str.replace(')', ')}') # Handle parentheses in column names
latex_df.to_latex(f"{output_dir_tables}/final_model_performance_summary.tex", index=False, float_format="%.3f", escape=False)

print(f"\nSummary table saved to {output_dir_tables}/final_model_performance_summary.csv and .tex")


In [ ]:
# prompt: **Summary Table:**
#     *   Create a comprehensive **summary table** (pandas DataFrame) consolidating the results.
#     *   Include a row for each model.
#     *   Columns should include: Model Name, Mean ± 95% CI for ROC AUC, PR AUC, and F1-score (from bootstrap).
#     *   For models other than LSTM-AE, include columns showing the p-values from the paired t-test, Wilcoxon test, and McNemar's test when compared to the LSTM-AE.
#     *   Add a column or indicator (e.g., asterisk on p-values) to flag statistically significant differences (p < 0.05).

import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score
from typing import Dict, List, Any
from scipy.stats import ttest_rel, wilcoxon, mannwhitneyu
from statsmodels.stats.contingency_tables import mcnemar # Correct import for mcnemar

# Assuming models, metric_summary, and statistical_test_results dictionaries
# are populated from the previous code execution.
# Also assuming metric_distributions is available and contains the bootstrap distributions.

# Define the metrics we want in the table
metric_names = ['ROC AUC', 'PR AUC', 'F1 Score']

print("\nGenerating Comprehensive Summary Table...")

summary_data = []
for model_name in models.keys():
    row: Dict[str, Any] = {'Model': model_name}

    # Add Mean and 95% CI for each metric
    for metric_name in metric_names:
        # Ensure the metric exists for the model in metric_summary
        if model_name in metric_summary and metric_name in metric_summary[model_name]:
            summary_info = metric_summary[model_name][metric_name]
            mean = summary_info['mean']
            ci_lower = summary_info['ci_lower']
            ci_upper = summary_info['ci_upper']

            if not np.isnan(mean):
                # Format as "Mean (Lower CI - Upper CI)"
                row[f'{metric_name} Mean (95% CI)'] = f"{mean:.3f} ({ci_lower:.3f} - {ci_upper:.3f})"
            else:
                row[f'{metric_name} Mean (95% CI)'] = "N/A"
        else:
            row[f'{metric_name} Mean (95% CI)'] = "N/A" # Metric info missing


    # Add Statistical Test p-values and Significance flags vs LSTM-AE
    if model_name != 'LSTM-AE':
        # Ensure test results exist for the model
        if 'vs LSTM-AE' in statistical_test_results and model_name in statistical_test_results['vs LSTM-AE']:
            vs_lstm_ae_tests = statistical_test_results['vs LSTM-AE'][model_name]

            # Define the specific tests to include and their order
            tests_to_include = {
                'ROC AUC (T-test p)': 'ROC AUC (T-test p) (vs LSTM-AE)',
                'ROC AUC (Wilcoxon p)': 'ROC AUC (Wilcoxon p) (vs LSTM-AE)',
                # 'ROC AUC (Mann-Whitney p)': 'ROC AUC (Mann-Whitney p) (vs LSTM-AE)', # Include if Mann-Whitney was used
                'PR AUC (T-test p)': 'PR AUC (T-test p) (vs LSTM-AE)',
                'PR AUC (Wilcoxon p)': 'PR AUC (Wilcoxon p) (vs LSTM-AE)',
                 # 'PR AUC (Mann-Whitney p)': 'PR AUC (Mann-Whitney p) (vs LSTM-AE)', # Include if Mann-Whitney was used
                'F1 Score (T-test p)': 'F1 Score (T-test p) (vs LSTM-AE)',
                'F1 Score (Wilcoxon p)': 'F1 Score (Wilcoxon p) (vs LSTM-AE)',
                # 'F1 Score (Mann-Whitney p)': 'F1 Score (Mann-Whitney p) (vs LSTM-AE)', # Include if Mann-Whitney was used
                'McNemar p': 'McNemar p (vs LSTM-AE)'
            }

            for test_key, column_name in tests_to_include.items():
                p_value = vs_lstm_ae_tests.get(test_key) # Use .get to handle potentially missing keys

                if p_value is not None and not np.isnan(p_value):
                    row[column_name] = f"{p_value:.4f}"
                    # Add significance flag (p < 0.05)
                    row[f'Significant? ({test_key})'] = '*' if p_value < 0.05 else ''
                else:
                    row[column_name] = "N/A"
                    row[f'Significant? ({test_key})'] = ''
        else:
            # Add N/A for all test columns if no test results are found for the model
            # This ensures consistent columns even if testing failed for a model
            tests_to_include = {
                'ROC AUC (T-test p)': 'ROC AUC (T-test p) (vs LSTM-AE)',
                'ROC AUC (Wilcoxon p)': 'ROC AUC (Wilcoxon p) (vs LSTM-AE)',
                # 'ROC AUC (Mann-Whitney p)': 'ROC AUC (Mann-Whitney p) (vs LSTM-AE)',
                'PR AUC (T-test p)': 'PR AUC (T-test p) (vs LSTM-AE)',
                'PR AUC (Wilcoxon p)': 'PR AUC (Wilcoxon p) (vs LSTM-AE)',
                # 'PR AUC (Mann-Whitney p)': 'PR AUC (Mann-Whitney p) (vs LSTM-AE)',
                'F1 Score (T-test p)': 'F1 Score (T-test p) (vs LSTM-AE)',
                'F1 Score (Wilcoxon p)': 'F1 Score (Wilcoxon p) (vs LSTM-AE)',
                # 'F1 Score (Mann-Whitney p)': 'F1 Score (Mann-Whitney p) (vs LSTM-AE)',
                'McNemar p': 'McNemar p (vs LSTM-AE)'
            }
            for test_key, column_name in tests_to_include.items():
                row[column_name] = "N/A"
                row[f'Significant? ({test_key})'] = ''


    summary_data.append(row)

# Create the DataFrame from the collected data
summary_df = pd.DataFrame(summary_data)

# Define the desired column order
# This ensures the table has a consistent structure
ordered_columns = ['Model']
for metric_name in metric_names:
    ordered_columns.append(f'{metric_name} Mean (95% CI)')
    # Add test columns related to this metric
    if metric_name == 'ROC AUC':
        ordered_columns.append('ROC AUC (T-test p) (vs LSTM-AE)')
        ordered_columns.append('Significant? (ROC AUC (T-test p))')
        ordered_columns.append('ROC AUC (Wilcoxon p) (vs LSTM-AE)')
        ordered_columns.append('Significant? (ROC AUC (Wilcoxon p))')
        # ordered_columns.append('ROC AUC (Mann-Whitney p) (vs LSTM-AE)') # Include if used
        # ordered_columns.append('Significant? (ROC AUC (Mann-Whitney p))')
    elif metric_name == 'PR AUC':
        ordered_columns.append('PR AUC (T-test p) (vs LSTM-AE)')
        ordered_columns.append('Significant? (PR AUC (T-test p))')
        ordered_columns.append('PR AUC (Wilcoxon p) (vs LSTM-AE)')
        ordered_columns.append('Significant? (PR AUC (Wilcoxon p))')
        # ordered_columns.append('PR AUC (Mann-Whitney p) (vs LSTM-AE)') # Include if used
        # ordered_columns.append('Significant? (PR AUC (Mann-Whitney p))')
    elif metric_name == 'F1 Score':
        ordered_columns.append('F1 Score (T-test p) (vs LSTM-AE)')
        ordered_columns.append('Significant? (F1 Score (T-test p))')
        ordered_columns.append('F1 Score (Wilcoxon p) (vs LSTM-AE)')
        ordered_columns.append('Significant? (F1 Score (Wilcoxon p))')
        # ordered_columns.append('F1 Score (Mann-Whitney p) (vs LSTM-AE)') # Include if used
        # ordered_columns.append('Significant? (F1 Score (Mann-Whitney p))')


# Add McNemar's test results
ordered_columns.append('McNemar p (vs LSTM-AE)')
ordered_columns.append('Significant? (McNemar p)')


# Filter columns to only include those that actually exist in the DataFrame
final_columns = [col for col in ordered_columns if col in summary_df.columns]
summary_df = summary_df[final_columns]


print("\nSummary Table:")
# Use to_string() to print the entire DataFrame without truncation
print(summary_df.to_string())

# You might want to save this DataFrame to a file (e.g., CSV, Excel)
# Example: Save to CSV
# summary_df.to_csv("model_performance_summary.csv", index=False)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Create dummy data for illustration (replace with actual data if needed)
# Assume a single sample with a few time steps and features
sample_input = np.array([
    [0.1, 0.2, 0.8],
    [0.3, 0.4, 0.7],
    [0.5, 0.6, 0.6],
    [0.7, 0.8, 0.5]
]) # Shape (timesteps, features)

# Simulate autoencoder reconstruction
# Add some small errors to the original input
sample_reconstruction = sample_input + np.random.randn(*sample_input.shape) * 0.05
sample_reconstruction = np.clip(sample_reconstruction, 0, 1) # Keep within a reasonable range


# Calculate absolute error
absolute_error = np.abs(sample_input - sample_reconstruction)

# Calculate mean absolute error (MAE)
mean_absolute_error = np.mean(absolute_error)

# --- Create the Visualization (Optimized for Small Size, High Res) ---
plt.figure(figsize=(6, 4), dpi=600) # Smaller figure size, higher DPI

# Adjust font sizes for better readability in a smaller plot
title_fontsize = 10
label_fontsize = 8
tick_fontsize = 7
annotation_fontsize = 8

# Plot 1: Original Input
plt.subplot(3, 1, 1)
plt.imshow(sample_input.T, cmap='viridis', aspect='auto', interpolation='nearest')
plt.title("Original Time Series Input", fontsize=title_fontsize)
plt.xlabel("Time Steps", fontsize=label_fontsize)
plt.ylabel("Features", fontsize=label_fontsize)
plt.xticks(fontsize=tick_fontsize)
plt.yticks(fontsize=tick_fontsize)
plt.colorbar(label="Value", fraction=0.046, pad=0.04) # Adjust colorbar size/position

# Plot 2: Autoencoder Reconstruction
plt.subplot(3, 1, 2)
plt.imshow(sample_reconstruction.T, cmap='viridis', aspect='auto', interpolation='nearest')
plt.title("Autoencoder Reconstruction", fontsize=title_fontsize)
plt.xlabel("Time Steps", fontsize=label_fontsize)
plt.ylabel("Features", fontsize=label_fontsize)
plt.xticks(fontsize=tick_fontsize)
plt.yticks(fontsize=tick_fontsize)
plt.colorbar(label="Value", fraction=0.046, pad=0.04) # Adjust colorbar size/position

# Plot 3: Absolute Error
plt.subplot(3, 1, 3)
plt.imshow(absolute_error.T, cmap='Reds', aspect='auto', interpolation='nearest')
plt.title("Absolute Error (|Original - Reconstruction|)", fontsize=title_fontsize)
plt.xlabel("Time Steps", fontsize=label_fontsize)
plt.ylabel("Features", fontsize=label_fontsize)
plt.xticks(fontsize=tick_fontsize)
plt.yticks(fontsize=tick_fontsize)
plt.colorbar(label="Absolute Error", fraction=0.046, pad=0.04) # Adjust colorbar size/position


# Add text annotations to explain the process
# Adjust position and font size of annotations
plt.figtext(0.5, 0.67, "Autoencoder learns to reconstruct normal data", ha="center", fontsize=annotation_fontsize, bbox={"boxstyle":"round","fc":"aliceblue","alpha":0.8})
plt.figtext(0.5, 0.34, r"$Error = |Original - Reconstruction|$", ha="center", fontsize=annotation_fontsize, bbox={"boxstyle":"round","fc":"aliceblue","alpha":0.8})
plt.figtext(0.5, 0.05, f"Mean Absolute Error (MAE) = Average(Error) = {mean_absolute_error:.4f} (Anomaly Score)", ha="center", fontsize=annotation_fontsize, bbox={"boxstyle":"round","fc":"lightgreen","alpha":0.8})


plt.tight_layout(rect=[0, 0.08, 1, 1]) # Adjust layout to make space for the text
plt.suptitle("Visualization of Reconstruction Error Calculation", y=1.02, fontsize=title_fontsize + 2) # Adjust suptitle size
plt.show()

In [ ]:
# Re-run the bootstrap analysis to populate metric_distributions

import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score
from typing import Dict, List, Any
import os

# Assuming the following variables are available from previous steps:
# models (dictionary of trained models and their predictions/probabilities)
# X_test, y_test (test data)
# n_iterations (number of bootstrap iterations)
# alpha (significance level)

# Initialize dictionaries to store bootstrap results
metric_distributions: Dict[str, Dict[str, List[float]]] = {
    model_name: {'ROC AUC': [], 'PR AUC': [], 'F1 Score': [], 'bootstrap_samples_for_plotting': []}
    for model_name in models.keys()
}

# Perform bootstrap resampling
print(f"Performing bootstrap resampling ({n_iterations} iterations)...")

for i in range(n_iterations):
    # Resample the test set with replacement
    bootstrap_indices = np.random.choice(len(y_test), size=len(y_test), replace=True)
    y_test_sample = y_test[bootstrap_indices]

    for model_name, data in models.items():
        # Get probabilities and binary predictions for the bootstrap sample
        probs_sample = data['probs'][bootstrap_indices]
        preds_sample = data['preds'][bootstrap_indices] # Assuming binary predictions are available

        # Calculate metrics on the bootstrap sample
        # ROC AUC
        if len(np.unique(y_test_sample)) > 1 and len(np.unique(probs_sample)) > 1:
            roc_auc = roc_auc_score(y_test_sample, probs_sample)
            metric_distributions[model_name]['ROC AUC'].append(roc_auc)
        else:
            # Handle cases where resampling results in only one class or constant probabilities
            metric_distributions[model_name]['ROC AUC'].append(np.nan) # Append NaN for invalid samples

        # PR AUC
        if len(np.unique(y_test_sample)) > 1 and len(np.unique(probs_sample)) > 1:
             pr_auc = average_precision_score(y_test_sample, probs_sample)
             metric_distributions[model_name]['PR AUC'].append(pr_auc)
        else:
             metric_distributions[model_name]['PR AUC'].append(np.nan)


        # F1 Score (requires binary predictions)
        if len(np.unique(y_test_sample)) > 1: # F1 is undefined for single class
             f1 = f1_score(y_test_sample, preds_sample)
             metric_distributions[model_name]['F1 Score'].append(f1)
        else:
             metric_distributions[model_name]['F1 Score'].append(np.nan)


        # Store samples for plotting ROC/PR curves with bands
        if i < 100: # Store a subset for plotting efficiency
             metric_distributions[model_name]['bootstrap_samples_for_plotting'].append((y_test_sample, probs_sample))


print("Bootstrap resampling complete.")

# Calculate summary statistics (mean, CI) from bootstrap distributions
metric_summary: Dict[str, Dict[str, Dict[str, float]]] = {}
for model_name, metrics in metric_distributions.items():
    metric_summary[model_name] = {}
    for metric_name, distribution in metrics.items():
        if metric_name == 'bootstrap_samples_for_plotting':
            continue # Skip plotting samples

        # Filter out NaN values before calculating mean and CI
        valid_distribution = np.array(distribution)[~np.isnan(distribution)]

        if len(valid_distribution) > 0:
            mean = np.mean(valid_distribution)
            # Calculate confidence interval using percentile method
            lower_ci = np.percentile(valid_distribution, (alpha / 2) * 100)
            upper_ci = np.percentile(valid_distribution, 100 - (alpha / 2) * 100)
            metric_summary[model_name][metric_name] = {
                'mean': mean,
                'ci_lower': lower_ci,
                'ci_upper': ci_upper,
                'distribution': list(valid_distribution) # Store valid distribution for effect size calculation
            }
        else:
            metric_summary[model_name][metric_name] = {
                'mean': np.nan,
                'ci_lower': np.nan,
                'ci_upper': np.nan,
                'distribution': []
            }

print("\nSummary statistics calculated.")

In [ ]:
# prompt: Run statistical Test on Ensemble Learners and tell which one performs best on statistical test

import pandas as pd
import numpy as np
from scipy.stats import ttest_rel
from statsmodels.stats.contingency_tables import mcnemar
from typing import Dict, List, Any
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier

# Assuming y_test, averaged_probs, ensemble_predictions, weighted_averaged_probs,
# weighted_ensemble_predictions, stacked_predictions_probs, stacked_predictions,
# stacked_predictions_probs_xgb, stacked_predictions_xgb,
# reconstruction_errors_test, anomaly_predictions,
# best_clf, mlp_model, ocsvm, knn, xgb, rf
# are available from the preceding code block.

# Re-define models dictionary including the original and ensemble models
models = {
    'LSTM-AE': {'probs': reconstruction_errors_test, 'preds': anomaly_predictions}, # Assuming reconstruction_errors_test is the "score" for LSTM-AE
    'Logistic Regression': {'probs': best_clf.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1], 'preds': best_clf.predict(reconstruction_errors_test.reshape(-1, 1))},
    'MLP': {'probs': mlp_model.predict(reconstruction_errors_test.reshape(-1, 1)).flatten(), 'preds': (mlp_model.predict(reconstruction_errors_test.reshape(-1, 1)).flatten() > 0.5).astype(int)},
    'One-Class SVM': {'probs': -ocsvm.decision_function(reconstruction_errors_test.reshape(-1, 1)).flatten(), 'preds': (ocsvm.decision_function(reconstruction_errors_test.reshape(-1, 1)).flatten() < 0).astype(int)}, # Using negative scores for AUC/PR AUC, and thresholding at 0 for binary predictions
    'KNN': {'probs': knn.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1], 'preds': knn.predict(reconstruction_errors_test.reshape(-1, 1))},
    'XGBoost': {'probs': xgb.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1], 'preds': xgb.predict(reconstruction_errors_test.reshape(-1, 1))},
    'Random Forest': {'probs': rf.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1], 'preds': rf.predict(reconstruction_errors_test.reshape(-1, 1))},
    'Averaged Ensemble': {'probs': averaged_probs, 'preds': ensemble_predictions},
    'Weighted Ensemble': {'probs': weighted_averaged_probs, 'preds': weighted_ensemble_predictions},
    'Stacked (LR Meta)': {'probs': stacked_predictions_probs, 'preds': stacked_predictions},
    'Stacked (XGB Meta)': {'probs': stacked_predictions_probs_xgb, 'preds': stacked_predictions_xgb},
}

# Ensure model names are ordered for consistent output
model_order = list(models.keys())

# Define the ensemble models subset for pairwise comparison among themselves
# Assuming 'models' dictionary is already defined and contains ensemble models
ensemble_models = {
    model_name: data for model_name, data in models.items()
    if 'Ensemble' in model_name or 'Stacked' in model_name
}

# Store statistical test results for ensemble comparisons
ensemble_statistical_test_results: Dict[str, Dict[str, Dict[str, float]]] = {}

print("\nPerforming Statistical Testing Among Ensemble Learners (Paired T-test and McNemar only)...")

ensemble_model_names = list(ensemble_models.keys())

# Iterate through all unique pairs of ensemble models
for i in range(len(ensemble_model_names)):
    for j in range(i + 1, len(ensemble_model_names)):
        model1_name = ensemble_model_names[i]
        model2_name = ensemble_model_names[j]

        model1_data = ensemble_models[model1_name]
        model2_data = ensemble_models[model2_name]

        # Retrieve bootstrap distributions
        model1_roc_auc_dist = np.array(metric_distributions[model1_name]['ROC AUC'], dtype=np.float64)
        model1_pr_auc_dist = np.array(metric_distributions[model1_name]['PR AUC'], dtype=np.float64)
        model1_f1_dist = np.array(metric_distributions[model1_name]['F1 Score'], dtype=np.float64)

        model2_roc_auc_dist = np.array(metric_distributions[model2_name]['ROC AUC'], dtype=np.float64)
        model2_pr_auc_dist = np.array(metric_distributions[model2_name]['PR AUC'], dtype=np.float64)
        model2_f1_dist = np.array(metric_distributions[model2_name]['F1 Score'], dtype=np.float64)

        # Retrieve original binary predictions for McNemar test
        model1_preds = model1_data['preds']
        model2_preds = model2_data['preds']


        comparison_key = f'{model1_name} vs {model2_name}'
        ensemble_statistical_test_results[comparison_key] = {}

        # Perform Paired T-test on metric distributions (ROC AUC, PR AUC, F1 Score)
        for metric_name, dist1, dist2 in zip(
            ['ROC AUC', 'PR AUC', 'F1 Score'],
            [model1_roc_auc_dist, model1_pr_auc_dist, model1_f1_dist],
            [model2_roc_auc_dist, model2_pr_auc_dist, model2_f1_dist]
        ):
             min_len = min(len(dist1), len(dist2))
             if min_len >= 2:
                 # Paired t-test (handles NaNs)
                 t_stat, p_value_t = ttest_rel(dist1[:min_len], dist2[:min_len], nan_policy='omit')
                 ensemble_statistical_test_results[comparison_key][f'{metric_name} (T-test p)'] = p_value_t
             else:
                 ensemble_statistical_test_results[comparison_key][f'{metric_name} (T-test p)'] = np.nan


        # McNemar test on binary predictions (requires original predictions)
        try:
            # Ensure both prediction arrays have the same length as y_test
            if len(model1_preds) == len(y_test) and len(model2_preds) == len(y_test):
                # Create a contingency table for McNemar's test
                # (Model1 correct, Model2 incorrect), (Model1 incorrect, Model2 correct)
                n10 = np.sum((model1_preds == y_test) & (model2_preds != y_test))
                n01 = np.sum((model1_preds != y_test) & (model2_preds == y_test))

                # The statsmodels mcnemar function uses the full 2x2 table
                contingency_table_mcnemar = pd.crosstab(model1_preds, model2_preds).reindex(
                    pd.MultiIndex.from_product([[0, 1], [0, 1]], names=[f'{model1_name} Pred', f'{model2_name} Pred']), fill_value=0)

                table_array_mcnemar = np.array([
                    [contingency_table_mcnemar.loc[(0, 0)], contingency_table_mcnemar.loc[(0, 1)]],
                    [contingency_table_mcnemar.loc[(1, 0)], contingency_table_mcnemar.loc[(1, 1)]]
                ])

                # Ensure there are discordant pairs for the test to be meaningful
                if n01 + n10 > 0:
                    mcnemar_result = mcnemar(table_array_mcnemar, exact=True) # Use exact=False for larger samples
                    ensemble_statistical_test_results[comparison_key]['McNemar p'] = mcnemar_result.pvalue
                else:
                    ensemble_statistical_test_results[comparison_key]['McNemar p'] = 1.0 # Perfect agreement on discordant pairs

            else:
                 ensemble_statistical_test_results[comparison_key]['McNemar p'] = np.nan # Length mismatch

        except Exception as e:
             print(f"Error performing McNemar test for {comparison_key}: {e}")
             ensemble_statistical_test_results[comparison_key]['McNemar p'] = np.nan


# Format the ensemble comparison results into a DataFrame
print("\nStatistical Test Results Among Ensemble Learners (Paired T-test and McNemar only):")

ensemble_comparison_data = []
# Define the order of tests for consistent columns
test_keys_order = [
    'ROC AUC (T-test p)',
    'PR AUC (T-test p)',
    'F1 Score (T-test p)',
    'McNemar p'
]

for comparison, tests in ensemble_statistical_test_results.items():
    row = {'Comparison': comparison}
    for test_name_key in test_keys_order:
        p_value = tests.get(test_name_key) # Use get to handle missing keys

        if p_value is not None and not np.isnan(p_value):
            row[test_name_key] = f"{p_value:.4f}"
            row[f'Significant? ({test_name_key})'] = '*' if p_value < alpha else ''
        else:
            row[test_name_key] = "N/A"
            row[f'Significant? ({test_name_key})'] = ''

    ensemble_comparison_data.append(row)

ensemble_comparison_df = pd.DataFrame(ensemble_comparison_data)

# Filter columns to only include those that were actually added
valid_ensemble_columns = ['Comparison']
for test_name_key in test_keys_order:
    if test_name_key in ensemble_comparison_df.columns:
        valid_ensemble_columns.append(test_name_key)
        valid_ensemble_columns.append(f'Significant? ({test_name_key})')

ensemble_comparison_df = ensemble_comparison_df[valid_ensemble_columns]

# Display the table
print(ensemble_comparison_df.to_string())

# --- Visualization of Statistical Differences (Optional, but good) ---

# You can visualize the p-values or significance flags.
# A heatmap of p-values could be insightful.

# Prepare data for heatmap (using T-test p-values)
heatmap_data_p_roc = pd.DataFrame(index=ensemble_model_names, columns=ensemble_model_names, dtype=float)
heatmap_data_p_pr = pd.DataFrame(index=ensemble_model_names, columns=ensemble_model_names, dtype=float)
heatmap_data_p_f1 = pd.DataFrame(index=ensemble_model_names, columns=ensemble_model_names, dtype=float)
heatmap_data_p_mcnemar = pd.DataFrame(index=ensemble_model_names, columns=ensemble_model_names, dtype=float)

# Fill heatmap dataframes
for i in range(len(ensemble_model_names)):
    for j in range(i + 1, len(ensemble_model_names)):
        model1_name = ensemble_model_names[i]
        model2_name = ensemble_model_names[j]
        comparison_key1 = f'{model1_name} vs {model2_name}'
        comparison_key2 = f'{model2_name} vs {model1_name}' # Check both directions

        comparison_results = None
        if comparison_key1 in ensemble_statistical_test_results:
             comparison_results = ensemble_statistical_test_results[comparison_key1]
        elif comparison_key2 in ensemble_statistical_test_results:
            # If key is reversed, the p-values are the same
             comparison_results = ensemble_statistical_test_results[comparison_key2]
        else:
            continue # Skip if no comparison results found

        # Use T-test p-value for heatmap
        p_roc = comparison_results.get('ROC AUC (T-test p)', np.nan)
        p_pr = comparison_results.get('PR AUC (T-test p)', np.nan)
        p_f1 = comparison_results.get('F1 Score (T-test p)', np.nan)
        p_mcnemar = comparison_results.get('McNemar p', np.nan)


        heatmap_data_p_roc.loc[model1_name, model2_name] = p_roc
        heatmap_data_p_roc.loc[model2_name, model1_name] = p_roc # Symmetric

        heatmap_data_p_pr.loc[model1_name, model2_name] = p_pr
        heatmap_data_p_pr.loc[model2_name, model1_name] = p_pr # Symmetric

        heatmap_data_p_f1.loc[model1_name, model2_name] = p_f1
        heatmap_data_p_f1.loc[model2_name, model1_name] = p_f1 # Symmetric

        heatmap_data_p_mcnemar.loc[model1_name, model2_name] = p_mcnemar
        heatmap_data_p_mcnemar.loc[model2_name, model1_name] = p_mcnemar # Symmetric


# Mask the upper triangle and diagonal since it's symmetric and we only need one half
mask = np.triu(np.ones_like(heatmap_data_p_roc, dtype=bool))

# Plotting Heatmaps of p-values
plt.figure(figsize=(10, 8))
sns.heatmap(heatmap_data_p_roc, annot=True, cmap="viridis_r", fmt=".3f", mask=mask, linewidths=.5)
plt.title("Pairwise T-test p-values (ROC AUC) Among Ensemble Learners")
plt.show()

plt.figure(figsize=(10, 8))
sns.heatmap(heatmap_data_p_pr, annot=True, cmap="viridis_r", fmt=".3f", mask=mask, linewidths=.5)
plt.title("Pairwise T-test p-values (PR AUC) Among Ensemble Learners")
plt.show()

plt.figure(figsize=(10, 8))
sns.heatmap(heatmap_data_p_f1, annot=True, cmap="viridis_r", fmt=".3f", mask=mask, linewidths=.5)
plt.title("Pairwise T-test p-values (F1 Score) Among Ensemble Learners")
plt.show()

plt.figure(figsize=(10, 8))
sns.heatmap(heatmap_data_p_mcnemar, annot=True, cmap="viridis_r", fmt=".3f", mask=mask, linewidths=.5)
plt.title("Pairwise McNemar Test p-values Among Ensemble Learners")
plt.show()


# Interpretation of Heatmaps:
# - Small p-values (e.g., < 0.05, colored darker in a reverse viridis cmap) indicate a statistically significant difference in performance metric (for T-tests) or error patterns (for McNemar) between the two models being compared.
# - Large p-values indicate no statistically significant difference found at the chosen alpha level.

# Combine statistical results with mean performance for a final conclusion

print("\nAnalyzing Ensemble Performance and Statistical Differences...")

ensemble_summary_data = []
for model_name in ensemble_model_names:
    summary_info_roc = metric_summary[model_name].get('ROC AUC', {'mean': np.nan, 'ci_lower': np.nan, 'ci_upper': np.nan})
    summary_info_pr = metric_summary[model_name].get('PR AUC', {'mean': np.nan, 'ci_lower': np.nan, 'ci_upper': np.nan})
    summary_info_f1 = metric_summary[model_name].get('F1 Score', {'mean': np.nan, 'ci_lower': np.nan, 'ci_upper': np.nan})

    row = {
        'Model': model_name,
        'Mean ROC AUC': summary_info_roc['mean'],
        'Mean PR AUC': summary_info_pr['mean'],
        'Mean F1 Score': summary_info_f1['mean'],
        'ROC AUC CI': f"({summary_info_roc['ci_lower']:.3f} - {summary_info_roc['ci_upper']:.3f})" if not np.isnan(summary_info_roc['mean']) else "N/A",
        'PR AUC CI': f"({summary_info_pr['ci_lower']:.3f} - {summary_info_pr['upper_ci']:.3f})" if not np.isnan(summary_info_pr['mean']) else "N/A",
        'F1 Score CI': f"({summary_info_f1['ci_lower']:.3f} - {summary_info_f1['ci_upper']:.3f})" if not np.isnan(summary_info_f1['mean']) else "N/A",
        'Sig. Better Count (ROC AUC)': 0,
        'Sig. Worse Count (ROC AUC)': 0,
        'Sig. Better Count (PR AUC)': 0,
        'Sig. Worse Count (PR AUC)': 0,
        'Sig. Better Count (F1 Score)': 0,
        'Sig. Worse Count (F1 Score)': 0,
        'Sig. McNemar Count': 0 # Count how many comparisons show significant McNemar
    }
    ensemble_summary_data.append(row)

ensemble_performance_df = pd.DataFrame(ensemble_summary_data)


# Update counts based on pairwise statistical tests (using T-test and McNemar)
for i in range(len(ensemble_performance_df)):
    model1_name = ensemble_performance_df.loc[i, 'Model']
    mean_roc1 = ensemble_performance_df.loc[i, 'Mean ROC AUC']
    mean_pr1 = ensemble_performance_df.loc[i, 'Mean PR AUC']
    mean_f1_1 = ensemble_performance_df.loc[i, 'Mean F1 Score']


    for j in range(i + 1, len(ensemble_performance_df)):
        model2_name = ensemble_performance_df.loc[j, 'Model']
        mean_roc2 = ensemble_performance_df.loc[j, 'Mean ROC AUC']
        mean_pr2 = ensemble_performance_df.loc[j, 'Mean PR AUC']
        mean_f1_2 = ensemble_performance_df.loc[j, 'Mean F1 Score']

        comparison_key1 = f'{model1_name} vs {model2_name}'
        comparison_key2 = f'{model2_name} vs {model1_name}'

        comparison_results = None
        if comparison_key1 in ensemble_statistical_test_results:
             comparison_results = ensemble_statistical_test_results[comparison_key1]
        elif comparison_key2 in ensemble_statistical_test_results:
             comparison_results = ensemble_statistical_test_results[comparison_key2]


        if comparison_results:
            # Check ROC AUC significance (using T-test)
            p_value_roc_t = comparison_results.get('ROC AUC (T-test p)')
            if p_value_roc_t is not None and not np.isnan(p_value_roc_t) and p_value_roc_t < alpha:
                if mean_roc1 > mean_roc2:
                    ensemble_performance_df.loc[i, 'Sig. Better Count (ROC AUC)'] += 1
                    ensemble_performance_df.loc[j, 'Sig. Worse Count (ROC AUC)'] += 1
                elif mean_roc2 > mean_roc1:
                    ensemble_performance_df.loc[j, 'Sig. Better Count (ROC AUC)'] += 1
                    ensemble_performance_df.loc[i, 'Sig. Worse Count (ROC AUC)'] += 1

            # Check PR AUC significance (using T-test)
            p_value_pr_t = comparison_results.get('PR AUC (T-test p)')
            if p_value_pr_t is not None and not np.isnan(p_value_pr_t) and p_value_pr_t < alpha:
                 if mean_pr1 > mean_pr2:
                     ensemble_performance_df.loc[i, 'Sig. Better Count (PR AUC)'] += 1
                     ensemble_performance_df.loc[j, 'Sig. Worse Count (PR AUC)'] += 1
                 elif mean_pr2 > mean_pr1:
                     ensemble_performance_df.loc[j, 'Sig. Better Count (PR AUC)'] += 1
                     ensemble_performance_df.loc[i, 'Sig. Worse Count (PR AUC)'] += 1


            # Check F1 Score significance (using T-test)
            p_value_f1_t = comparison_results.get('F1 Score (T-test p)')
            if p_value_f1_t is not None and not np.isnan(p_value_f1_t) and p_value_f1_t < alpha:
                 if mean_f1_1 > mean_f1_2:
                     ensemble_performance_df.loc[i, 'Sig. Better Count (F1 Score)'] += 1
                     ensemble_performance_df.loc[j, 'Sig. Worse Count (F1 Score)'] += 1
                 elif mean_f1_2 > mean_f1_1:
                     ensemble_performance_df.loc[j, 'Sig. Better Count (F1 Score)'] += 1
                     ensemble_performance_df.loc[i, 'Sig. Worse Count (F1 Score)'] += 1


            # Check McNemar significance
            mcnemar_p = comparison_results.get('McNemar p')
            if mcnemar_p is not None and not np.isnan(mcnemar_p) and mcnemar_p < alpha:
                ensemble_performance_df.loc[i, 'Sig. McNemar Count'] += 1
                ensemble_performance_df.loc[j, 'Sig. McNemar Count'] += 1 # Count for both models in the pair

# Sort the DataFrame by Mean ROC AUC (or other preferred metric)
ensemble_performance_df_sorted = ensemble_performance_df.sort_values(by='Mean ROC AUC', ascending=False).reset_index(drop=True)

print("\nEnsemble Model Performance Summary with Statistical Significance Counts (T-test and McNemar only):")
print(ensemble_performance_df_sorted.to_string())

# Final Recommendation based on the sorted table and counts:
# Look at the top rows of `ensemble_performance_df_sorted`.
# The model that has high mean values for ROC AUC and PR AUC (and F1 if relevant)
# and a high 'Sig. Better Count' and a low 'Sig. Worse Count' is likely the best performer.
# Consider models that are not significantly worse than any others.

# Example Recommendation Logic:
best_model_candidate = ensemble_performance_df_sorted.loc[0, 'Model'] # Start with the highest mean ROC AUC
print(f"\nInitial Best Candidate (based on Mean ROC AUC): {best_model_candidate}")

# Check if this candidate is significantly worse than any other model (based on T-test for ROC AUC/PR AUC/F1)
is_significantly_worse = (ensemble_performance_df_sorted.loc[0, 'Sig. Worse Count (ROC AUC)'] > 0) or \
                         (ensemble_performance_df_sorted.loc[0, 'Sig. Worse Count (PR AUC)'] > 0) or \
                         (ensemble_performance_df_sorted.loc[0, 'Sig. Worse Count (F1 Score)'] > 0)


if not is_significantly_worse:
    print(f"The initial candidate '{best_model_candidate}' was NOT found to be significantly worse than any other ensemble model (based on T-tests).")
    print(f"Final Recommendation: '{best_model_candidate}' based on highest mean performance and no statistically significant worse performance (based on T-tests).")
else:
    print(f"The initial candidate '{best_model_candidate}' WAS found to be significantly worse than at least one other ensemble model (based on T-tests).")
    # Need a more sophisticated selection if the top model has significant weaknesses.
    # E.g., select the model with the highest mean ROC AUC among those with 0 'Sig. Worse Count'.
    candidates_not_sig_worse = ensemble_performance_df_sorted[
        (ensemble_performance_df_sorted['Sig. Worse Count (ROC AUC)'] == 0) &
        (ensemble_performance_df_sorted['Sig. Worse Count (PR AUC)'] == 0) &
        (ensemble_performance_df_sorted['Sig. Worse Count (F1 Score)'] == 0)
    ]

    if not candidates_not_sig_worse.empty:
        best_stable_model = candidates_not_sig_worse.sort_values(by='Mean ROC AUC', ascending=False).iloc[0]
        print(f"Considering models not significantly worse than others (based on T-tests).")
        print(f"Final Recommendation: '{best_stable_model['Model']}' based on highest mean ROC AUC among stable models (based on T-tests).")
    else:
         print("All models were found to be significantly worse than at least one other model (based on T-tests).")
         print("Recommend reviewing the pairwise comparison table and heatmaps for a nuanced understanding.")
         print("The model with the highest overall 'Sig. Better Count' might be a good choice.")

# Save the detailed ensemble performance summary table
output_dir_tables = "/content/drive/MyDrive/Final_Implementations/DynaBiome_Tables/"
os.makedirs(output_dir_tables, exist_ok=True)
ensemble_performance_df_sorted.to_csv(f"{output_dir_tables}/ensemble_performance_summary_with_sig_counts_ttest_mcnemar.csv", index=False)

# Prepare for LaTeX: Escape special characters
latex_ensemble_perf_df = ensemble_performance_df_sorted.copy()
latex_ensemble_perf_df.columns = latex_ensemble_perf_df.columns.str.replace('_', '\\_')
latex_ensemble_perf_df.columns = latex_ensemble_perf_df.columns.str.replace('%', '\\%')
latex_ensemble_perf_df.columns = latex_ensemble_perf_df.columns.str.replace('(', '{(').str.replace(')', ')}')

latex_ensemble_perf_df.to_latex(f"{output_dir_tables}/ensemble_performance_summary_with_sig_counts_ttest_mcnemar.tex", index=False, float_format="%.3f", escape=False)

print(f"\nEnsemble performance summary saved to {output_dir_tables}/ensemble_performance_summary_with_sig_counts_ttest_mcnemar.csv and .tex")

In [ ]:
from scipy.stats import ttest_rel, wilcoxon, mannwhitneyu
from statsmodels.stats.contingency_tables import mcnemar
from typing import Dict, List, Any
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, roc_curve, precision_recall_curve
import os

# Define required variables (example values)
n_iterations = 1000  # Number of bootstrap resamples
alpha = 0.05         # Significance level for statistical tests (e.g., 0.05)

# Re-define models dictionary including the original and ensemble models
models = {
    'LSTM-AE': {'probs': reconstruction_errors_test, 'preds': anomaly_predictions}, # Assuming reconstruction_errors_test is the "score" for LSTM-AE
    'Logistic Regression': {'probs': best_clf.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1], 'preds': best_clf.predict(reconstruction_errors_test.reshape(-1, 1))},
    'MLP': {'probs': mlp_model.predict(reconstruction_errors_test.reshape(-1, 1)).flatten(), 'preds': (mlp_model.predict(reconstruction_errors_test.reshape(-1, 1)).flatten() > 0.5).astype(int)},
    'One-Class SVM': {'probs': -ocsvm.decision_function(reconstruction_errors_test.reshape(-1, 1)).flatten(), 'preds': (ocsvm.decision_function(reconstruction_errors_test.reshape(-1, 1)).flatten() < 0).astype(int)}, # Using negative scores for AUC/PR AUC, and thresholding at 0 for binary predictions
    'KNN': {'probs': knn.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1], 'preds': knn.predict(reconstruction_errors_test.reshape(-1, 1))},
    'XGBoost': {'probs': xgb.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1], 'preds': xgb.predict(reconstruction_errors_test.reshape(-1, 1))},
    'Random Forest': {'probs': rf.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1], 'preds': rf.predict(reconstruction_errors_test.reshape(-1, 1))},
    'Averaged Ensemble': {'probs': averaged_probs, 'preds': ensemble_predictions},
    'Weighted Ensemble': {'probs': weighted_averaged_probs, 'preds': weighted_ensemble_predictions},
    'Stacked (LR Meta)': {'probs': stacked_predictions_probs, 'preds': stacked_predictions},
    'Stacked (XGB Meta)': {'probs': stacked_predictions_probs_xgb, 'preds': stacked_predictions_xgb},
}

# Define the ensemble models subset for pairwise comparison among themselves
ensemble_models = {
    model_name: data for model_name, data in models.items()
    if 'Ensemble' in model_name or 'Stacked' in model_name
}

# Initialize dictionaries to store bootstrap results
metric_distributions: Dict[str, Dict[str, List[float]]] = {
    model_name: {'ROC AUC': [], 'PR AUC': [], 'F1 Score': [], 'bootstrap_samples_for_plotting': []}
    for model_name in models.keys()
}

# Use a fixed random state for reproducibility of bootstrap samples themselves
bootstrap_rng = np.random.default_rng(seed=42)

# Perform bootstrap resampling
print(f"Performing bootstrap resampling ({n_iterations} iterations)...")

for i in range(n_iterations):
    # Resample the test set with replacement
    bootstrap_indices = bootstrap_rng.choice(len(y_test), size=len(y_test), replace=True)
    y_test_sample = y_test[bootstrap_indices]

    for model_name, data in models.items():
        # Get probabilities and binary predictions for the bootstrap sample
        probs_sample = data['probs'][bootstrap_indices]
        preds_sample = data['preds'][bootstrap_indices] # Assuming binary predictions are available

        # Calculate metrics on the bootstrap sample
        # ROC AUC
        # Check for multiple classes in true labels and variation in scores/predictions
        if len(np.unique(y_test_sample)) > 1 and len(np.unique(probs_sample)) > 1:
            roc_auc = roc_auc_score(y_test_sample, probs_sample)
            metric_distributions[model_name]['ROC AUC'].append(roc_auc)
        else:
            # Handle cases where resampling results in only one class or constant probabilities
            metric_distributions[model_name]['ROC AUC'].append(np.nan) # Append NaN for invalid samples

        # PR AUC
        if len(np.unique(y_test_sample)) > 1 and len(np.unique(probs_sample)) > 1:
             pr_auc = average_precision_score(y_test_sample, probs_sample)
             metric_distributions[model_name]['PR AUC'].append(pr_auc)
        else:
             metric_distributions[model_name]['PR AUC'].append(np.nan)


        # F1 Score (requires binary predictions)
        # Check for multiple classes in true labels and variation in predictions
        if len(np.unique(y_test_sample)) > 1 and len(np.unique(preds_sample)) > 1: # F1 is undefined for single class or constant predictions
             f1 = f1_score(y_test_sample, preds_sample)
             metric_distributions[model_name]['F1 Score'].append(f1)
        else:
             metric_distributions[model_name]['F1 Score'].append(np.nan)


        # Store samples for plotting ROC/PR curves with bands
        # Store a fixed number of samples (e.g., up to 100) for plotting efficiency
        if len(metric_distributions[model_name]['bootstrap_samples_for_plotting']) < 100:
             metric_distributions[model_name]['bootstrap_samples_for_plotting'].append((y_test_sample, probs_sample))


print("Bootstrap resampling complete.")

# Calculate summary statistics (mean, CI) from bootstrap distributions
metric_summary: Dict[str, Dict[str, Dict[str, Any]]] = {}
for model_name, metrics in metric_distributions.items():
    metric_summary[model_name] = {}
    for metric_name, distribution in metrics.items():
        if metric_name == 'bootstrap_samples_for_plotting':
            continue # Skip plotting samples

        # Filter out NaN values before calculating mean and CI
        valid_distribution = np.array(distribution)[~np.isnan(distribution)]

        if len(valid_distribution) > 0:
            mean = np.mean(valid_distribution)
            # Calculate confidence interval using percentile method
            # Need at least 2 points for percentiles
            if len(valid_distribution) >= 2:
                lower_ci = np.percentile(valid_distribution, (alpha / 2) * 100)
                upper_ci = np.percentile(valid_distribution, 100 - (alpha / 2) * 100)
            else:
                lower_ci = np.nan
                upper_ci = np.nan

            metric_summary[model_name][metric_name] = {
                'mean': mean,
                'ci_lower': lower_ci,
                'ci_upper': upper_ci,
                'distribution': list(valid_distribution) # Store valid distribution for effect size calculation
            }
        else:
            metric_summary[model_name][metric_name] = {
                'mean': np.nan,
                'ci_lower': np.nan,
                'ci_upper': np.nan,
                'distribution': []
            }

print("\nSummary statistics calculated.")

# Define the ensemble models subset for pairwise comparison among themselves
# Assuming 'models' dictionary is already defined and contains ensemble models
ensemble_models = {
    model_name: data for model_name, data in models.items()
    if 'Ensemble' in model_name or 'Stacked' in model_name
}

# Store statistical test results for ensemble comparisons
ensemble_statistical_test_results: Dict[str, Dict[str, Dict[str, float]]] = {}

print("\nPerforming Statistical Testing Among Ensemble Learners...")

ensemble_model_names = list(ensemble_models.keys())

# Iterate through all unique pairs of ensemble models
for i in range(len(ensemble_model_names)):
    for j in range(i + 1, len(ensemble_model_names)):
        model1_name = ensemble_model_names[i]
        model2_name = ensemble_model_names[j]

        model1_data = ensemble_models[model1_name]
        model2_data = ensemble_models[model2_name]

        # Retrieve bootstrap distributions
        model1_roc_auc_dist = np.array(metric_distributions[model1_name]['ROC AUC'], dtype=np.float64)
        model1_pr_auc_dist = np.array(metric_distributions[model1_name]['PR AUC'], dtype=np.float64)
        model1_f1_dist = np.array(metric_distributions[model1_name]['F1 Score'], dtype=np.float64)

        model2_roc_auc_dist = np.array(metric_distributions[model2_name]['ROC AUC'], dtype=np.float64)
        model2_pr_auc_dist = np.array(metric_distributions[model2_name]['PR AUC'], dtype=np.float64)
        model2_f1_dist = np.array(metric_distributions[model2_name]['F1 Score'], dtype=np.float64)

        # Retrieve original binary predictions for McNemar test
        model1_preds = model1_data['preds']
        model2_preds = model2_data['preds']


        comparison_key = f'{model1_name} vs {model2_name}'
        ensemble_statistical_test_results[comparison_key] = {}

        # Perform paired tests on metric distributions (ROC AUC, PR AUC, F1 Score)
        for metric_name, dist1, dist2 in zip(
            ['ROC AUC', 'PR AUC', 'F1 Score'],
            [model1_roc_auc_dist, model1_pr_auc_dist, model1_f1_dist],
            [model2_roc_auc_dist, model2_pr_auc_dist, model2_f1_dist]
        ):
             min_len = min(len(dist1), len(dist2))
             if min_len >= 2:
                 # Paired t-test (handles NaNs)
                 t_stat, p_value_t = ttest_rel(dist1[:min_len], dist2[:min_len], nan_policy='omit')
                 ensemble_statistical_test_results[comparison_key][f'{metric_name} (T-test p)'] = p_value_t

                 # Wilcoxon test (filter NaNs, require variation)
                 dist1_valid = dist1[:min_len][~np.isnan(dist1[:min_len])]
                 dist2_valid = dist2[:min_len][~np.isnan(dist2[:min_len])]
                 if len(dist1_valid) >= 2 and len(dist2_valid) >= 2 and len(set(dist1_valid)) > 1 and len(set(dist2_valid)) > 1:
                     try:
                         w_stat, p_value_w = wilcoxon(dist1_valid, dist2_valid)
                         ensemble_statistical_test_results[comparison_key][f'{metric_name} (Wilcoxon p)'] = p_value_w
                     except ValueError:
                         ensemble_statistical_test_results[comparison_key][f'{metric_name} (Wilcoxon p)'] = np.nan
                 else:
                     ensemble_statistical_test_results[comparison_key][f'{metric_name} (Wilcoxon p)'] = np.nan

                 # Mann-Whitney U test (filter NaNs, require sufficient data)
                 if len(dist1_valid) >= 2 and len(dist2_valid) >= 2:
                     u_stat, p_value_u = mannwhitneyu(dist1_valid, dist2_valid)
                     ensemble_statistical_test_results[comparison_key][f'{metric_name} (Mann-Whitney p)'] = p_value_u
                 else:
                      ensemble_statistical_test_results[comparison_key][f'{metric_name} (Mann-Whitney p)'] = np.nan

             else:
                 ensemble_statistical_test_results[comparison_key][f'{metric_name} (T-test p)'] = np.nan
                 ensemble_statistical_test_results[comparison_key][f'{metric_name} (Wilcoxon p)'] = np.nan
                 ensemble_statistical_test_results[comparison_key][f'{metric_name} (Mann-Whitney p)'] = np.nan


        # McNemar test on binary predictions (requires original predictions)
        try:
            # Ensure both prediction arrays have the same length as y_test
            if len(model1_preds) == len(y_test) and len(model2_preds) == len(y_test):
                # Create a contingency table for McNemar's test
                # (Model1 correct, Model2 incorrect), (Model1 incorrect, Model2 correct)
                n10 = np.sum((model1_preds == y_test) & (model2_preds != y_test))
                n01 = np.sum((model1_preds != y_test) & (model2_preds == y_test))

                # The statsmodels mcnemar function uses the full 2x2 table
                contingency_table_mcnemar = pd.crosstab(model1_preds, model2_preds).reindex(
                    pd.MultiIndex.from_product([[0, 1], [0, 1]], names=[f'{model1_name} Pred', f'{model2_name} Pred']), fill_value=0)

                table_array_mcnemar = np.array([
                    [contingency_table_mcnemar.loc[(0, 0)], contingency_table_mcnemar.loc[(0, 1)]],
                    [contingency_table_mcnemar.loc[(1, 0)], contingency_table_mcnemar.loc[(1, 1)]]
                ])

                # Ensure there are discordant pairs for the test to be meaningful
                if n01 + n10 > 0:
                    mcnemar_result = mcnemar(table_array_mcnemar, exact=True) # Use exact=False for larger samples
                    ensemble_statistical_test_results[comparison_key]['McNemar p'] = mcnemar_result.pvalue
                else:
                    ensemble_statistical_test_results[comparison_key]['McNemar p'] = 1.0 # Perfect agreement on discordant pairs

            else:
                 ensemble_statistical_test_results[comparison_key]['McNemar p'] = np.nan # Length mismatch

        except Exception as e:
             print(f"Error performing McNemar test for {comparison_key}: {e}")
             ensemble_statistical_test_results[comparison_key]['McNemar p'] = np.nan


# Format the ensemble comparison results into a DataFrame
print("\nStatistical Test Results Among Ensemble Learners:")

ensemble_comparison_data = []
# Define the order of tests for consistent columns
test_keys_order = [
    'ROC AUC (T-test p)', 'ROC AUC (Wilcoxon p)', 'ROC AUC (Mann-Whitney p)',
    'PR AUC (T-test p)', 'PR AUC (Wilcoxon p)', 'PR AUC (Mann-Whitney p)',
    'F1 Score (T-test p)', 'F1 Score (Wilcoxon p)', 'F1 Score (Mann-Whitney p)',
    'McNemar p'
]

for comparison, tests in ensemble_statistical_test_results.items():
    row = {'Comparison': comparison}
    for test_name_key in test_keys_order:
        p_value = tests.get(test_name_key) # Use get to handle missing keys

        if p_value is not None and not np.isnan(p_value):
            row[test_name_key] = f"{p_value:.4f}"
            row[f'Significant? ({test_name_key})'] = '*' if p_value < alpha else ''
        else:
            row[test_name_key] = "N/A"
            row[f'Significant? ({test_name_key})'] = ''

    ensemble_comparison_data.append(row)

ensemble_comparison_df = pd.DataFrame(ensemble_comparison_data)

# Filter columns to only include those that were actually added
valid_ensemble_columns = ['Comparison']
for test_name_key in test_keys_order:
    if test_name_key in ensemble_comparison_df.columns:
        valid_ensemble_columns.append(test_name_key)
        valid_ensemble_columns.append(f'Significant? ({test_name_key})')

ensemble_comparison_df = ensemble_comparison_df[valid_ensemble_columns]

# Display the table
print(ensemble_comparison_df.to_string())

# --- Visualization of Statistical Differences (Optional, but good) ---

# You can visualize the p-values or significance flags.
# A heatmap of p-values could be insightful.

# Prepare data for heatmap
heatmap_data_p_roc = pd.DataFrame(index=ensemble_model_names, columns=ensemble_model_names, dtype=float)
heatmap_data_p_pr = pd.DataFrame(index=ensemble_model_names, columns=ensemble_model_names, dtype=float)
heatmap_data_p_f1 = pd.DataFrame(index=ensemble_model_names, columns=ensemble_model_names, dtype=float)
heatmap_data_p_mcnemar = pd.DataFrame(index=ensemble_model_names, columns=ensemble_model_names, dtype=float)

# Fill heatmap dataframes
for i in range(len(ensemble_model_names)):
    for j in range(i + 1, len(ensemble_model_names)):
        model1_name = ensemble_model_names[i]
        model2_name = ensemble_model_names[j]
        comparison_key1 = f'{model1_name} vs {model2_name}'
        comparison_key2 = f'{model2_name} vs {model1_name}' # Check both directions

        if comparison_key1 in ensemble_statistical_test_results:
             comparison_results = ensemble_statistical_test_results[comparison_key1]
        elif comparison_key2 in ensemble_statistical_test_results:
            # If key is reversed, the p-values are the same
             comparison_results = ensemble_statistical_test_results[comparison_key2]
        else:
            continue # Skip if no comparison results found

        # Use T-test p-value for heatmap (can choose Wilcoxon or Mann-Whitney)
        p_roc = comparison_results.get('ROC AUC (T-test p)', np.nan)
        p_pr = comparison_results.get('PR AUC (T-test p)', np.nan)
        p_f1 = comparison_results.get('F1 Score (T-test p)', np.nan)
        p_mcnemar = comparison_results.get('McNemar p', np.nan)


        heatmap_data_p_roc.loc[model1_name, model2_name] = p_roc
        heatmap_data_p_roc.loc[model2_name, model1_name] = p_roc # Symmetric

        heatmap_data_p_pr.loc[model1_name, model2_name] = p_pr
        heatmap_data_p_pr.loc[model2_name, model1_name] = p_pr # Symmetric

        heatmap_data_p_f1.loc[model1_name, model2_name] = p_f1
        heatmap_data_p_f1.loc[model2_name, model1_name] = p_f1 # Symmetric

        heatmap_data_p_mcnemar.loc[model1_name, model2_name] = p_mcnemar
        heatmap_data_p_mcnemar.loc[model2_name, model1_name] = p_mcnemar # Symmetric


# Mask the upper triangle and diagonal since it's symmetric and we only need one half
mask = np.triu(np.ones_like(heatmap_data_p_roc, dtype=bool))

# Plotting Heatmaps of p-values
plt.figure(figsize=(10, 8))
sns.heatmap(heatmap_data_p_roc, annot=True, cmap="viridis_r", fmt=".3f", mask=mask, linewidths=.5)
plt.title("Pairwise T-test p-values (ROC AUC) Among Ensemble Learners")
plt.show()

plt.figure(figsize=(10, 8))
sns.heatmap(heatmap_data_p_pr, annot=True, cmap="viridis_r", fmt=".3f", mask=mask, linewidths=.5)
plt.title("Pairwise T-test p-values (PR AUC) Among Ensemble Learners")
plt.show()

plt.figure(figsize=(10, 8))
sns.heatmap(heatmap_data_p_f1, annot=True, cmap="viridis_r", fmt=".3f", mask=mask, linewidths=.5)
plt.title("Pairwise T-test p-values (F1 Score) Among Ensemble Learners")
plt.show()

plt.figure(figsize=(10, 8))
sns.heatmap(heatmap_data_p_mcnemar, annot=True, cmap="viridis_r", fmt=".3f", mask=mask, linewidths=.5)
plt.title("Pairwise McNemar Test p-values Among Ensemble Learners")
plt.show()


# Interpretation of Heatmaps:
# - Small p-values (e.g., < 0.05, colored darker in a reverse viridis cmap) indicate a statistically significant difference in performance metric (for T-tests) or error patterns (for McNemar) between the two models being compared.
# - Large p-values indicate no statistically significant difference found at the chosen alpha level.

# Combine statistical results with mean performance for a final conclusion

print("\nAnalyzing Ensemble Performance and Statistical Differences...")

ensemble_summary_data = []
for model_name in ensemble_model_names:
    summary_info_roc = metric_summary[model_name].get('ROC AUC', {'mean': np.nan, 'ci_lower': np.nan, 'ci_upper': np.nan})
    summary_info_pr = metric_summary[model_name].get('PR AUC', {'mean': np.nan, 'ci_lower': np.nan, 'ci_upper': np.nan})
    summary_info_f1 = metric_summary[model_name].get('F1 Score', {'mean': np.nan, 'ci_lower': np.nan, 'ci_upper': np.nan})

    row = {
        'Model': model_name,
        'Mean ROC AUC': summary_info_roc['mean'],
        'Mean PR AUC': summary_info_pr['mean'],
        'Mean F1 Score': summary_info_f1['mean'],
        'ROC AUC CI': f"({summary_info_roc['ci_lower']:.3f} - {summary_info_roc['ci_upper']:.3f})" if not np.isnan(summary_info_roc['mean']) else "N/A",
        'PR AUC CI': f"({summary_info_pr['ci_lower']:.3f} - {summary_info_pr['ci_upper']:.3f})" if not np.isnan(summary_info_pr['mean']) else "N/A",
        'F1 Score CI': f"({summary_info_f1['ci_lower']:.3f} - {summary_info_f1['ci_upper']:.3f})" if not np.isnan(summary_info_f1['mean']) else "N/A",
        'Sig. Better Count (ROC AUC)': 0,
        'Sig. Worse Count (ROC AUC)': 0,
        'Sig. Better Count (PR AUC)': 0,
        'Sig. Worse Count (PR AUC)': 0,
        'Sig. Better Count (F1 Score)': 0,
        'Sig. Worse Count (F1 Score)': 0,
        'Sig. McNemar Count': 0 # Count how many comparisons show significant McNemar
    }
    ensemble_summary_data.append(row)

ensemble_performance_df = pd.DataFrame(ensemble_summary_data)


# Update counts based on pairwise statistical tests (using T-test and McNemar)
for i in range(len(ensemble_performance_df)):
    model1_name = ensemble_performance_df.loc[i, 'Model']
    mean_roc1 = ensemble_performance_df.loc[i, 'Mean ROC AUC']
    mean_pr1 = ensemble_performance_df.loc[i, 'Mean PR AUC']
    mean_f1_1 = ensemble_performance_df.loc[i, 'Mean F1 Score']


    for j in range(i + 1, len(ensemble_performance_df)):
        model2_name = ensemble_performance_df.loc[j, 'Model']
        mean_roc2 = ensemble_performance_df.loc[j, 'Mean ROC AUC']
        mean_pr2 = ensemble_performance_df.loc[j, 'Mean PR AUC']
        mean_f1_2 = ensemble_performance_df.loc[j, 'Mean F1 Score']

        comparison_key1 = f'{model1_name} vs {model2_name}'
        comparison_key2 = f'{model2_name} vs {model1_name}'

        comparison_results = None
        if comparison_key1 in ensemble_statistical_test_results:
             comparison_results = ensemble_statistical_test_results[comparison_key1]
        elif comparison_key2 in ensemble_statistical_test_results:
             comparison_results = ensemble_statistical_test_results[comparison_key2]


        if comparison_results:
            # Check ROC AUC significance (using T-test)
            p_value_roc_t = comparison_results.get('ROC AUC (T-test p)')
            if p_value_roc_t is not None and not np.isnan(p_value_roc_t) and p_value_roc_t < alpha:
                if mean_roc1 > mean_roc2:
                    ensemble_performance_df.loc[i, 'Sig. Better Count (ROC AUC)'] += 1
                    ensemble_performance_df.loc[j, 'Sig. Worse Count (ROC AUC)'] += 1
                elif mean_roc2 > mean_roc1:
                    ensemble_performance_df.loc[j, 'Sig. Better Count (ROC AUC)'] += 1
                    ensemble_performance_df.loc[i, 'Sig. Worse Count (ROC AUC)'] += 1

            # Check PR AUC significance (using T-test)
            p_value_pr_t = comparison_results.get('PR AUC (T-test p)')
            if p_value_pr_t is not None and not np.isnan(p_value_pr_t) and p_value_pr_t < alpha:
                 if mean_pr1 > mean_pr2:
                     ensemble_performance_df.loc[i, 'Sig. Better Count (PR AUC)'] += 1
                     ensemble_performance_df.loc[j, 'Sig. Worse Count (PR AUC)'] += 1
                 elif mean_pr2 > mean_pr1:
                     ensemble_performance_df.loc[j, 'Sig. Better Count (PR AUC)'] += 1
                     ensemble_performance_df.loc[i, 'Sig. Worse Count (PR AUC)'] += 1


            # Check F1 Score significance (using T-test)
            p_value_f1_t = comparison_results.get('F1 Score (T-test p)')
            if p_value_f1_t is not None and not np.isnan(p_value_f1_t) and p_value_f1_t < alpha:
                 if mean_f1_1 > mean_f1_2:
                     ensemble_performance_df.loc[i, 'Sig. Better Count (F1 Score)'] += 1
                     ensemble_performance_df.loc[j, 'Sig. Worse Count (F1 Score)'] += 1
                 elif mean_f1_2 > mean_f1_1:
                     ensemble_performance_df.loc[j, 'Sig. Better Count (F1 Score)'] += 1
                     ensemble_performance_df.loc[i, 'Sig. Worse Count (F1 Score)'] += 1


            # Check McNemar significance
            mcnemar_p = comparison_results.get('McNemar p')
            if mcnemar_p is not None and not np.isnan(mcnemar_p) and mcnemar_p < alpha:
                ensemble_performance_df.loc[i, 'Sig. McNemar Count'] += 1
                ensemble_performance_df.loc[j, 'Sig. McNemar Count'] += 1 # Count for both models in the pair

# Sort the DataFrame by Mean ROC AUC (or other preferred metric)
ensemble_performance_df_sorted = ensemble_performance_df.sort_values(by='Mean ROC AUC', ascending=False).reset_index(drop=True)

print("\nEnsemble Model Performance Summary with Statistical Significance Counts (T-test and McNemar only):")
print(ensemble_performance_df_sorted.to_string())

# Final Recommendation based on the sorted table and counts:
# Look at the top rows of `ensemble_performance_df_sorted`.
# The model that has high mean values for ROC AUC and PR AUC (and F1 if relevant)
# and a high 'Sig. Better Count' and a low 'Sig. Worse Count' is likely the best performer.
# Consider models that are not significantly worse than any others.

# Example Recommendation Logic:
best_model_candidate = ensemble_performance_df_sorted.loc[0, 'Model'] # Start with the highest mean ROC AUC
print(f"\nInitial Best Candidate (based on Mean ROC AUC): {best_model_candidate}")

# Check if this candidate is significantly worse than any other model (based on T-test for ROC AUC/PR AUC/F1)
is_significantly_worse = (ensemble_performance_df_sorted.loc[0, 'Sig. Worse Count (ROC AUC)'] > 0) or \
                         (ensemble_performance_df_sorted.loc[0, 'Sig. Worse Count (PR AUC)'] > 0) or \
                         (ensemble_performance_df_sorted.loc[0, 'Sig. Worse Count (F1 Score)'] > 0)


if not is_significantly_worse:
    print(f"The initial candidate '{best_model_candidate}' was NOT found to be significantly worse than any other ensemble model (based on T-tests).")
    print(f"Final Recommendation: '{best_model_candidate}' based on highest mean performance and no statistically significant worse performance (based on T-tests).")
else:
    print(f"The initial candidate '{best_model_candidate}' WAS found to be significantly worse than at least one other ensemble model (based on T-tests).")
    # Need a more sophisticated selection if the top model has significant weaknesses.
    # E.g., select the model with the highest mean ROC AUC among those with 0 'Sig. Worse Count'.
    candidates_not_sig_worse = ensemble_performance_df_sorted[
        (ensemble_performance_df_sorted['Sig. Worse Count (ROC AUC)'] == 0) &
        (ensemble_performance_df_sorted['Sig. Worse Count (PR AUC)'] == 0) &
        (ensemble_performance_df_sorted['Sig. Worse Count (F1 Score)'] == 0)
    ]

    if not candidates_not_sig_worse.empty:
        best_stable_model = candidates_not_sig_worse.sort_values(by='Mean ROC AUC', ascending=False).iloc[0]
        print(f"Considering models not significantly worse than others (based on T-tests).")
        print(f"Final Recommendation: '{best_stable_model['Model']}' based on highest mean ROC AUC among stable models (based on T-tests).")
    else:
         print("All models were found to be significantly worse than at least one other model (based on T-tests).")
         print("Recommend reviewing the pairwise comparison table and heatmaps for a nuanced understanding.")
         print("The model with the highest overall 'Sig. Better Count' might be a good choice.")

# Save the detailed ensemble performance summary table
output_dir_tables = "/content/drive/MyDrive/Final_Implementations/DynaBiome_Tables/"
os.makedirs(output_dir_tables, exist_ok=True)
ensemble_performance_df_sorted.to_csv(f"{output_dir_tables}/ensemble_performance_summary_with_sig_counts_ttest_mcnemar.csv", index=False)

# Prepare for LaTeX: Escape special characters
latex_ensemble_perf_df = ensemble_performance_df_sorted.copy()
latex_ensemble_perf_df.columns = latex_ensemble_perf_df.columns.str.replace('_', '\\_')
latex_ensemble_perf_df.columns = latex_ensemble_perf_df.columns.str.replace('%', '\\%')
latex_ensemble_perf_df.columns = latex_ensemble_perf_df.columns.str.replace('(', '{(', regex=False).str.replace(')', ')}')

latex_ensemble_perf_df.to_latex(f"{output_dir_tables}/ensemble_performance_summary_with_sig_counts_ttest_mcnemar.tex", index=False, float_format="%.3f", escape=False)

print(f"\nEnsemble performance summary saved to {output_dir_tables}/ensemble_performance_summary_with_sig_counts_ttest_mcnemar.csv and .tex")

In [ ]:
# Define required variables (example values)
n_iterations = 1000  # Number of bootstrap resamples
alpha = 0.05         # Significance level for statistical tests (e.g., 0.05)

# Ensure the following are already defined in your environment:
# models: Dict[str, Dict[str, np.ndarray]]  # Each model with 'probs' and 'preds'
# y_test: np.ndarray                         # Ground truth labels for test set


In [ ]:
# Define significance level and number of bootstrap iterations
n_iterations = 1000
alpha = 0.05

# Example: Dummy check to ensure required variables exist
assert 'models' in globals(), "Define 'models' dictionary with keys: 'probs', 'preds'"
assert 'y_test' in globals(), "Define 'y_test' numpy array"


In [ ]:
from scipy.stats import ttest_rel
from typing import Dict, List, Any
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# --- Assume the following variables are already defined ---
# models, y_test, n_iterations, alpha (e.g., 0.05)

# Step 1: Bootstrap Metric Distributions
metric_distributions: Dict[str, Dict[str, List[float]]] = {
    model_name: {'ROC AUC': [], 'PR AUC': [], 'F1 Score': []}
    for model_name in models.keys()
}

bootstrap_rng = np.random.default_rng(seed=42)
print(f"Performing bootstrap resampling ({n_iterations} iterations)...")

for i in range(n_iterations):
    indices = bootstrap_rng.choice(len(y_test), size=len(y_test), replace=True)
    y_sample = y_test[indices]

    for model_name, data in models.items():
        probs = data['probs'][indices]
        preds = data['preds'][indices]

        # ROC AUC
        if len(np.unique(y_sample)) > 1 and len(np.unique(probs)) > 1:
            metric_distributions[model_name]['ROC AUC'].append(roc_auc_score(y_sample, probs))
        else:
            metric_distributions[model_name]['ROC AUC'].append(np.nan)

        # PR AUC
        if len(np.unique(y_sample)) > 1 and len(np.unique(probs)) > 1:
            metric_distributions[model_name]['PR AUC'].append(average_precision_score(y_sample, probs))
        else:
            metric_distributions[model_name]['PR AUC'].append(np.nan)

        # F1 Score
        if len(np.unique(y_sample)) > 1 and len(np.unique(preds)) > 1:
            metric_distributions[model_name]['F1 Score'].append(f1_score(y_sample, preds))
        else:
            metric_distributions[model_name]['F1 Score'].append(np.nan)

print("Bootstrap resampling complete.")

# Step 2: Filter Ensemble Models
ensemble_models = {
    name: data for name, data in models.items()
    if 'Ensemble' in name or 'Stacked' in name
}
ensemble_model_names = list(ensemble_models.keys())

# Step 3: Perform Paired T-Tests
print("\nPerforming Paired T-Tests Among Ensemble Learners...")
ensemble_statistical_test_results: Dict[str, Dict[str, float]] = {}

for i in range(len(ensemble_model_names)):
    for j in range(i + 1, len(ensemble_model_names)):
        m1, m2 = ensemble_model_names[i], ensemble_model_names[j]
        key = f"{m1} vs {m2}"
        ensemble_statistical_test_results[key] = {}

        for metric in ['ROC AUC', 'PR AUC', 'F1 Score']:
            dist1 = np.array(metric_distributions[m1][metric], dtype=np.float64)
            dist2 = np.array(metric_distributions[m2][metric], dtype=np.float64)
            min_len = min(len(dist1), len(dist2))

            if min_len >= 2:
                t_stat, p_val = ttest_rel(dist1[:min_len], dist2[:min_len], nan_policy='omit')
                ensemble_statistical_test_results[key][f'{metric} (T-test p)'] = p_val
            else:
                ensemble_statistical_test_results[key][f'{metric} (T-test p)'] = np.nan

# Step 4: Create Results Table
print("\nT-test Results Among Ensemble Learners:")
ensemble_comparison_data = []
for comp, tests in ensemble_statistical_test_results.items():
    row = {'Comparison': comp}
    for metric_key, p_val in tests.items():
        row[metric_key] = f"{p_val:.4f}" if not np.isnan(p_val) else "N/A"
        row[f'Significant? ({metric_key})'] = '*' if not np.isnan(p_val) and p_val < alpha else ''
    ensemble_comparison_data.append(row)

ensemble_comparison_df = pd.DataFrame(ensemble_comparison_data)
print(ensemble_comparison_df.to_string())

# Step 5: Heatmap Visualization of Paired T-test p-values
print("\nVisualizing Paired T-test p-values...")
for metric in ['ROC AUC', 'PR AUC', 'F1 Score']:
    df_p = pd.DataFrame(index=ensemble_model_names, columns=ensemble_model_names, dtype=float)

    for i in range(len(ensemble_model_names)):
        for j in range(i + 1, len(ensemble_model_names)):
            m1, m2 = ensemble_model_names[i], ensemble_model_names[j]
            key1 = f"{m1} vs {m2}"
            key2 = f"{m2} vs {m1}"

            p_val = ensemble_statistical_test_results.get(key1, {}).get(f'{metric} (T-test p)',
                     ensemble_statistical_test_results.get(key2, {}).get(f'{metric} (T-test p)', np.nan))

            df_p.loc[m1, m2] = p_val
            df_p.loc[m2, m1] = p_val

    # Mask upper triangle
    mask = np.triu(np.ones_like(df_p, dtype=bool))

    plt.figure(figsize=(10, 8))
    sns.heatmap(df_p, annot=True, cmap="viridis_r", fmt=".3f", mask=mask, linewidths=.5)
    plt.title(f"Pairwise T-test p-values ({metric}) Among Ensemble Learners")
    plt.show()

# Optional: Save CSV and LaTeX
output_dir = "/content/drive/MyDrive/Final_Implementations/DynaBiome_Tables/"
os.makedirs(output_dir, exist_ok=True)

ensemble_comparison_df.to_csv(f"{output_dir}/ensemble_ttest_comparison.csv", index=False)

latex_df = ensemble_comparison_df.copy()
latex_df.columns = latex_df.columns.str.replace('_', '\\_').str.replace('%', '\\%')
latex_df.columns = latex_df.columns.str.replace('(', '{(').str.replace(')', ')}')

latex_df.to_latex(f"{output_dir}/ensemble_ttest_comparison.tex", index=False, float_format="%.4f", escape=False)

print(f"\nSaved: {output_dir}/ensemble_ttest_comparison.csv and .tex")


In [ ]:
from scipy.stats import ttest_rel
from typing import Dict, List, Any
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score

# -----------------------------------
# --- Required Inputs (Define First) ---
# -----------------------------------

n_iterations = 1000  # Number of bootstrap iterations
alpha = 0.05         # Significance level (e.g., 0.05)

# Ensure `models` is a dictionary:
# models = {'ModelA': {'probs': ..., 'preds': ...}, ...}
# y_test = ground truth labels for test set (numpy array)

# -----------------------------------
# --- Step 1: Bootstrap Metric Calculation ---
# -----------------------------------

metric_distributions: Dict[str, Dict[str, List[float]]] = {
    model_name: {'ROC AUC': [], 'PR AUC': [], 'F1 Score': []}
    for model_name in models.keys()
}

bootstrap_rng = np.random.default_rng(seed=42)
print(f"Performing bootstrap resampling ({n_iterations} iterations)...")

for _ in range(n_iterations):
    indices = bootstrap_rng.choice(len(y_test), size=len(y_test), replace=True)
    y_sample = y_test[indices]

    for model_name, data in models.items():
        probs = data['probs'][indices]
        preds = data['preds'][indices]

        # ROC AUC
        if len(np.unique(y_sample)) > 1 and len(np.unique(probs)) > 1:
            metric_distributions[model_name]['ROC AUC'].append(roc_auc_score(y_sample, probs))
        else:
            metric_distributions[model_name]['ROC AUC'].append(np.nan)

        # PR AUC
        if len(np.unique(y_sample)) > 1 and len(np.unique(probs)) > 1:
            metric_distributions[model_name]['PR AUC'].append(average_precision_score(y_sample, probs))
        else:
            metric_distributions[model_name]['PR AUC'].append(np.nan)

        # F1 Score
        if len(np.unique(y_sample)) > 1 and len(np.unique(preds)) > 1:
            metric_distributions[model_name]['F1 Score'].append(f1_score(y_sample, preds))
        else:
            metric_distributions[model_name]['F1 Score'].append(np.nan)

print("Bootstrap resampling complete.")

# -----------------------------------
# --- Step 2: Summary Statistics (Mean + CI) ---
# -----------------------------------

metric_summary = {}
for model_name, metrics in metric_distributions.items():
    metric_summary[model_name] = {}
    for metric_name, distribution in metrics.items():
        dist = np.array(distribution)
        dist = dist[~np.isnan(dist)]

        if len(dist) >= 2:
            mean = np.mean(dist)
            lower_ci = np.percentile(dist, alpha / 2 * 100)
            upper_ci = np.percentile(dist, (1 - alpha / 2) * 100)
        else:
            mean = lower_ci = upper_ci = np.nan

        metric_summary[model_name][metric_name] = {
            'mean': mean,
            'ci_lower': lower_ci,
            'ci_upper': upper_ci,
            'distribution': dist
        }

# -----------------------------------
# --- Step 3: Paired T-Tests (Only Ensemble Models) ---
# -----------------------------------

ensemble_models = {
    name: data for name, data in models.items()
    if 'Ensemble' in name or 'Stacked' in name
}
ensemble_model_names = list(ensemble_models.keys())

ensemble_statistical_test_results = {}
for i in range(len(ensemble_model_names)):
    for j in range(i + 1, len(ensemble_model_names)):
        m1, m2 = ensemble_model_names[i], ensemble_model_names[j]
        key = f"{m1} vs {m2}"
        ensemble_statistical_test_results[key] = {}

        for metric in ['ROC AUC', 'PR AUC', 'F1 Score']:
            dist1 = metric_summary[m1][metric]['distribution']
            dist2 = metric_summary[m2][metric]['distribution']
            min_len = min(len(dist1), len(dist2))

            if min_len >= 2:
                t_stat, p_val = ttest_rel(dist1[:min_len], dist2[:min_len], nan_policy='omit')
                ensemble_statistical_test_results[key][f'{metric} (T-test p)'] = p_val
            else:
                ensemble_statistical_test_results[key][f'{metric} (T-test p)'] = np.nan

# -----------------------------------
# --- Step 4: Compile Table with Summary + T-tests ---
# -----------------------------------

summary_rows = []

for model in ensemble_model_names:
    row = {
        'Model': model,
        'Mean ROC AUC': f"{metric_summary[model]['ROC AUC']['mean']:.4f}",
        'ROC AUC CI': f"({metric_summary[model]['ROC AUC']['ci_lower']:.4f} - {metric_summary[model]['ROC AUC']['ci_upper']:.4f})",
        'Mean PR AUC': f"{metric_summary[model]['PR AUC']['mean']:.4f}",
        'PR AUC CI': f"({metric_summary[model]['PR AUC']['ci_lower']:.4f} - {metric_summary[model]['PR AUC']['ci_upper']:.4f})",
        'Mean F1 Score': f"{metric_summary[model]['F1 Score']['mean']:.4f}",
        'F1 Score CI': f"({metric_summary[model]['F1 Score']['ci_lower']:.4f} - {metric_summary[model]['F1 Score']['ci_upper']:.4f})",
    }
    summary_rows.append(row)

summary_df = pd.DataFrame(summary_rows)

print("\nEnsemble Learner Performance Summary:")
print(summary_df.to_string(index=False))

# -----------------------------------
# --- Step 5: Heatmap of p-values ---
# -----------------------------------

for metric in ['ROC AUC', 'PR AUC', 'F1 Score']:
    heatmap_df = pd.DataFrame(index=ensemble_model_names, columns=ensemble_model_names, dtype=float)

    for i in range(len(ensemble_model_names)):
        for j in range(i + 1, len(ensemble_model_names)):
            m1, m2 = ensemble_model_names[i], ensemble_model_names[j]
            key1 = f"{m1} vs {m2}"
            key2 = f"{m2} vs {m1}"

            p_val = ensemble_statistical_test_results.get(key1, {}).get(f'{metric} (T-test p)',
                     ensemble_statistical_test_results.get(key2, {}).get(f'{metric} (T-test p)', np.nan))

            heatmap_df.loc[m1, m2] = p_val
            heatmap_df.loc[m2, m1] = p_val

    # Mask upper triangle
    mask = np.triu(np.ones_like(heatmap_df, dtype=bool))

    plt.figure(figsize=(10, 8))
    sns.heatmap(heatmap_df, annot=True, cmap="viridis_r", fmt=".3f", mask=mask, linewidths=.5)
    plt.title(f"Paired T-test p-values for {metric}")
    plt.show()

# -----------------------------------
# --- Optional Save ---
# -----------------------------------

output_dir = "/content/drive/MyDrive/Final_Implementations/DynaBiome_Tables/"
os.makedirs(output_dir, exist_ok=True)

summary_df.to_csv(f"{output_dir}/ensemble_performance_summary.csv", index=False)

# Save LaTeX version
latex_df = summary_df.copy()
latex_df.columns = latex_df.columns.str.replace('_', '\\_').str.replace('%', '\\%')
latex_df.columns = latex_df.columns.str.replace('(', '{(').str.replace(')', ')}')

latex_df.to_latex(f"{output_dir}/ensemble_performance_summary.tex", index=False, float_format="%.4f", escape=False)

print(f"\nSaved to: {output_dir}/ensemble_performance_summary.csv and .tex")


In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_rel
import os

# --- Configuration ---
alpha = 0.05  # Significance level
output_dir = "./results"  # Change path as needed
os.makedirs(output_dir, exist_ok=True)

# --- Assume metric_distributions is precomputed and loaded ---
# Format: {model_name: {'ROC AUC': [...], 'PR AUC': [...], 'F1 Score': [...]}}

# Dummy data for illustration only (Replace with actual bootstrap results)
metric_distributions = {
    'EnsembleA': {
        'ROC AUC': np.random.normal(0.85, 0.01, 100),
        'PR AUC': np.random.normal(0.80, 0.02, 100),
        'F1 Score': np.random.normal(0.78, 0.03, 100),
    },
    'EnsembleB': {
        'ROC AUC': np.random.normal(0.83, 0.01, 100),
        'PR AUC': np.random.normal(0.77, 0.02, 100),
        'F1 Score': np.random.normal(0.76, 0.03, 100),
    },
    'EnsembleC': {
        'ROC AUC': np.random.normal(0.84, 0.015, 100),
        'PR AUC': np.random.normal(0.78, 0.015, 100),
        'F1 Score': np.random.normal(0.77, 0.02, 100),
    },
}

# --- Step 1: Generate Summary Table (Mean and CI) ---
summary_stats = []
for model, metrics in metric_distributions.items():
    row = {'Model': model}
    for metric, values in metrics.items():
        values = np.array(values)
        values = values[~np.isnan(values)]
        mean = np.mean(values)
        ci_lower = np.percentile(values, alpha / 2 * 100)
        ci_upper = np.percentile(values, 100 - alpha / 2 * 100)
        row[f'Mean {metric}'] = mean
        row[f'{metric} CI'] = f"({ci_lower:.3f} - {ci_upper:.3f})"
    summary_stats.append(row)

summary_df = pd.DataFrame(summary_stats)
print("\n--- Ensemble Summary Statistics ---")
print(summary_df.to_string(index=False))
summary_df.to_csv(f"{output_dir}/ensemble_summary_statistics.csv", index=False)

# --- Step 2: Perform Pairwise Paired T-Tests ---
model_names = list(metric_distributions.keys())
comparison_results = []

for i in range(len(model_names)):
    for j in range(i + 1, len(model_names)):
        model1 = model_names[i]
        model2 = model_names[j]
        row = {'Comparison': f"{model1} vs {model2}"}

        for metric in ['ROC AUC', 'PR AUC', 'F1 Score']:
            dist1 = np.array(metric_distributions[model1][metric])
            dist2 = np.array(metric_distributions[model2][metric])

            min_len = min(len(dist1), len(dist2))
            dist1 = dist1[:min_len]
            dist2 = dist2[:min_len]

            t_stat, p_val = ttest_rel(dist1, dist2, nan_policy='omit')
            row[f"{metric} (T-test p)"] = f"{p_val:.4f}" if not np.isnan(p_val) else "N/A"
            row[f"Significant ({metric})"] = "*" if not np.isnan(p_val) and p_val < alpha else ""

        comparison_results.append(row)

pval_df = pd.DataFrame(comparison_results)

print("\n--- Pairwise Paired T-Test P-Values ---")
print(pval_df.to_string(index=False))

# --- Step 3: Save Results ---
pval_df.to_csv(f"{output_dir}/ensemble_pairwise_ttest_pvalues.csv", index=False)
pval_df.to_latex(f"{output_dir}/ensemble_pairwise_ttest_pvalues.tex", index=False, float_format="%.4f", escape=False)

print(f"\nResults saved to {output_dir}")


# Task
Rerun the bootstrap resampling and pairwise statistical tests (T-test, Wilcoxon, Mann-Whitney, McNemar) for ensemble models, apply Holm-Bonferroni correction to all generated p-values, and update the ensemble comparison summary tables with corrected p-values and significance flags to report the statistically significant results.

## Rerun Bootstrap and Ensemble Statistical Tests

### Subtask:
Rerun the bootstrap resampling and pairwise statistical tests (T-test, Wilcoxon, Mann-Whitney, McNemar) among ensemble models to ensure all p-values are correctly generated and stored.


**Reasoning**:
First, I will ensure all necessary variables, including the `models` dictionary with predictions and probabilities, `y_test`, and `n_iterations`, are properly defined before proceeding with the bootstrap resampling. This step is crucial for accurate statistical analysis of ensemble models.



In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score
from typing import Dict, List, Any
import os
from scipy.stats import ttest_rel, wilcoxon, mannwhitneyu
from statsmodels.stats.contingency_tables import mcnemar

# --- Configuration Variables ---
n_iterations = 1000  # Number of bootstrap resamples
alpha = 0.05         # Significance level for statistical tests (e.g., 0.05)

# --- Define models and their predictions/probabilities ---
# These variables are assumed to be available from previous cells in the notebook.
# If any are missing, replace them with appropriate loaded/computed values.
models = {
    'LSTM-AE': {'probs': reconstruction_errors_test, 'preds': anomaly_predictions},
    'Logistic Regression': {'probs': best_clf.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1], 'preds': best_clf.predict(reconstruction_errors_test.reshape(-1, 1))},
    'MLP': {'probs': mlp_model.predict(reconstruction_errors_test.reshape(-1, 1)).flatten(), 'preds': (mlp_model.predict(reconstruction_errors_test.reshape(-1, 1)).flatten() > 0.5).astype(int)},
    'One-Class SVM': {'probs': -ocsvm.decision_function(reconstruction_errors_test.reshape(-1, 1)).flatten(), 'preds': (ocsvm.decision_function(reconstruction_errors_test.reshape(-1, 1)).flatten() < 0).astype(int)},
    'KNN': {'probs': knn.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1], 'preds': knn.predict(reconstruction_errors_test.reshape(-1, 1))},
    'XGBoost': {'probs': xgb.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1], 'preds': xgb.predict(reconstruction_errors_test.reshape(-1, 1))},
    'Random Forest': {'probs': rf.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1], 'preds': rf.predict(reconstruction_errors_test.reshape(-1, 1))},
    'Averaged Ensemble': {'probs': averaged_probs, 'preds': ensemble_predictions},
    'Weighted Ensemble': {'probs': weighted_averaged_probs, 'preds': weighted_ensemble_predictions},
    'Stacked (LR Meta)': {'probs': stacked_predictions_probs, 'preds': stacked_predictions},
    'Stacked (XGB Meta)': {'probs': stacked_predictions_probs_xgb, 'preds': stacked_predictions_xgb},
}

# Ensure y_test is a numpy array
y_test = np.array(y_test)

**Reasoning**:
Now that the `models` dictionary and `y_test` are correctly defined, I will perform the bootstrap resampling process to generate the metric distributions for all models. Then, I will execute all specified pairwise statistical tests (Paired T-test, Wilcoxon test, Mann-Whitney U test, and McNemar test) between ensemble models and store their p-values in the `ensemble_statistical_test_results` dictionary.



In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score
from typing import Dict, List, Any
import os
from scipy.stats import ttest_rel, wilcoxon, mannwhitneyu
from statsmodels.stats.contingency_tables import mcnemar

# Initialize dictionaries to store bootstrap results
metric_distributions: Dict[str, Dict[str, List[float]]] = {
    model_name: {'ROC AUC': [], 'PR AUC': [], 'F1 Score': [], 'bootstrap_samples_for_plotting': []}
    for model_name in models.keys()
}

# Use a fixed random state for reproducibility of bootstrap samples themselves
bootstrap_rng = np.random.default_rng(seed=42)

# Perform bootstrap resampling
print(f"Performing bootstrap resampling ({n_iterations} iterations)...")

for i in range(n_iterations):
    # Resample the test set with replacement
    bootstrap_indices = bootstrap_rng.choice(len(y_test), size=len(y_test), replace=True)
    y_test_sample = y_test[bootstrap_indices]

    for model_name, data in models.items():
        # Get probabilities and binary predictions for the bootstrap sample
        probs_sample = data['probs'][bootstrap_indices]
        preds_sample = data['preds'][bootstrap_indices] # Assuming binary predictions are available

        # Calculate metrics on the bootstrap sample
        # ROC AUC
        # Check for multiple classes in true labels and variation in scores/predictions
        if len(np.unique(y_test_sample)) > 1 and len(np.unique(probs_sample)) > 1:
            roc_auc = roc_auc_score(y_test_sample, probs_sample)
            metric_distributions[model_name]['ROC AUC'].append(roc_auc)
        else:
            # Handle cases where resampling results in only one class or constant probabilities
            metric_distributions[model_name]['ROC AUC'].append(np.nan) # Append NaN for invalid samples

        # PR AUC
        if len(np.unique(y_test_sample)) > 1 and len(np.unique(probs_sample)) > 1:
             pr_auc = average_precision_score(y_test_sample, probs_sample)
             metric_distributions[model_name]['PR AUC'].append(pr_auc)
        else:
             metric_distributions[model_name]['PR AUC'].append(np.nan)


        # F1 Score (requires binary predictions)
        # Check for multiple classes in true labels and variation in predictions
        if len(np.unique(y_test_sample)) > 1 and len(np.unique(preds_sample)) > 1: # F1 is undefined for single class or constant predictions
             f1 = f1_score(y_test_sample, preds_sample)
             metric_distributions[model_name]['F1 Score'].append(f1)
        else:
             metric_distributions[model_name]['F1 Score'].append(np.nan)


        # Store samples for plotting ROC/PR curves with bands
        # Store a fixed number of samples (e.g., up to 100) for plotting efficiency
        if len(metric_distributions[model_name]['bootstrap_samples_for_plotting']) < 100:
             metric_distributions[model_name]['bootstrap_samples_for_plotting'].append((y_test_sample, probs_sample))


print("Bootstrap resampling complete.")

# Calculate summary statistics (mean, CI) from bootstrap distributions
metric_summary: Dict[str, Dict[str, Dict[str, Any]]] = {}
for model_name, metrics in metric_distributions.items():
    metric_summary[model_name] = {}
    for metric_name, distribution in metrics.items():
        if metric_name == 'bootstrap_samples_for_plotting':
            continue # Skip plotting samples

        # Filter out NaN values before calculating mean and CI
        valid_distribution = np.array(distribution)[~np.isnan(distribution)]

        if len(valid_distribution) > 0:
            mean = np.mean(valid_distribution)
            # Calculate confidence interval using percentile method
            # Need at least 2 points for percentiles
            if len(valid_distribution) >= 2:
                lower_ci = np.percentile(valid_distribution, (alpha / 2) * 100)
                upper_ci = np.percentile(valid_distribution, 100 - (alpha / 2) * 100)
            else:
                lower_ci = np.nan
                upper_ci = np.nan

            metric_summary[model_name][metric_name] = {
                'mean': mean,
                'ci_lower': lower_ci,
                'ci_upper': upper_ci,
                'distribution': list(valid_distribution) # Store valid distribution for effect size calculation
            }
        else:
            metric_summary[model_name][metric_name] = {
                'mean': np.nan,
                'ci_lower': np.nan,
                'ci_upper': np.nan,
                'distribution': []
            }

print("\nSummary statistics calculated.")

# Define the ensemble models subset for pairwise comparison among themselves
# Assuming 'models' dictionary is already defined and contains ensemble models
ensemble_models = {
    model_name: data for model_name, data in models.items()
    if 'Ensemble' in model_name or 'Stacked' in model_name
}

# Store statistical test results for ensemble comparisons
ensemble_statistical_test_results: Dict[str, Dict[str, Dict[str, float]]] = {}

print("\nPerforming Statistical Testing Among Ensemble Learners...")

ensemble_model_names = list(ensemble_models.keys())

# Iterate through all unique pairs of ensemble models
for i in range(len(ensemble_model_names)):
    for j in range(i + 1, len(ensemble_model_names)):
        model1_name = ensemble_model_names[i]
        model2_name = ensemble_model_names[j]

        model1_data = ensemble_models[model1_name]
        model2_data = ensemble_models[model2_name]

        # Retrieve bootstrap distributions
        model1_roc_auc_dist = np.array(metric_distributions[model1_name]['ROC AUC'], dtype=np.float64)
        model1_pr_auc_dist = np.array(metric_distributions[model1_name]['PR AUC'], dtype=np.float64)
        model1_f1_dist = np.array(metric_distributions[model1_name]['F1 Score'], dtype=np.float64)

        model2_roc_auc_dist = np.array(metric_distributions[model2_name]['ROC AUC'], dtype=np.float64)
        model2_pr_auc_dist = np.array(metric_distributions[model2_name]['PR AUC'], dtype=np.float64)
        model2_f1_dist = np.array(metric_distributions[model2_name]['F1 Score'], dtype=np.float64)

        # Retrieve original binary predictions for McNemar test
        model1_preds = model1_data['preds']
        model2_preds = model2_data['preds']


        comparison_key = f'{model1_name} vs {model2_name}'
        ensemble_statistical_test_results[comparison_key] = {}

        # Perform paired tests on metric distributions (ROC AUC, PR AUC, F1 Score)
        for metric_name, dist1, dist2 in zip(
            ['ROC AUC', 'PR AUC', 'F1 Score'],
            [model1_roc_auc_dist, model1_pr_auc_dist, model1_f1_dist],
            [model2_roc_auc_dist, model2_pr_auc_dist, model2_f1_dist]
        ):
             min_len = min(len(dist1), len(dist2))
             if min_len >= 2:
                 # Paired t-test (handles NaNs)
                 t_stat, p_value_t = ttest_rel(dist1[:min_len], dist2[:min_len], nan_policy='omit')
                 ensemble_statistical_test_results[comparison_key][f'{metric_name} (T-test p)'] = p_value_t

                 # Wilcoxon test (filter NaNs, require variation)
                 dist1_valid = dist1[:min_len][~np.isnan(dist1[:min_len])]
                 dist2_valid = dist2[:min_len][~np.isnan(dist2[:min_len])]
                 if len(dist1_valid) >= 2 and len(dist2_valid) >= 2 and len(set(dist1_valid)) > 1 and len(set(dist2_valid)) > 1:
                     try:
                         w_stat, p_value_w = wilcoxon(dist1_valid, dist2_valid)
                         ensemble_statistical_test_results[comparison_key][f'{metric_name} (Wilcoxon p)'] = p_value_w
                     except ValueError:
                         ensemble_statistical_test_results[comparison_key][f'{metric_name} (Wilcoxon p)'] = np.nan
                 else:
                     ensemble_statistical_test_results[comparison_key][f'{metric_name} (Wilcoxon p)'] = np.nan

                 # Mann-Whitney U test (filter NaNs, require sufficient data)
                 if len(dist1_valid) >= 2 and len(dist2_valid) >= 2:
                     u_stat, p_value_u = mannwhitneyu(dist1_valid, dist2_valid)
                     ensemble_statistical_test_results[comparison_key][f'{metric_name} (Mann-Whitney p)'] = p_value_u
                 else:
                      ensemble_statistical_test_results[comparison_key][f'{metric_name} (Mann-Whitney p)'] = np.nan

             else:
                 ensemble_statistical_test_results[comparison_key][f'{metric_name} (T-test p)'] = np.nan
                 ensemble_statistical_test_results[comparison_key][f'{metric_name} (Wilcoxon p)'] = np.nan
                 ensemble_statistical_test_results[comparison_key][f'{metric_name} (Mann-Whitney p)'] = np.nan


        # McNemar test on binary predictions (requires original predictions)
        try:
            # Ensure both prediction arrays have the same length as y_test
            if len(model1_preds) == len(y_test) and len(model2_preds) == len(y_test):
                # Create a contingency table for McNemar's test
                # (Model1 correct, Model2 incorrect), (Model1 incorrect, Model2 correct)
                n10 = np.sum((model1_preds == y_test) & (model2_preds != y_test))
                n01 = np.sum((model1_preds != y_test) & (model2_preds == y_test))

                # The statsmodels mcnemar function uses the full 2x2 table
                contingency_table_mcnemar = pd.crosstab(model1_preds, model2_preds).reindex(
                    pd.MultiIndex.from_product([[0, 1], [0, 1]], names=[f'{model1_name} Pred', f'{model2_name} Pred']), fill_value=0)

                table_array_mcnemar = np.array([
                    [contingency_table_mcnemar.loc[(0, 0)], contingency_table_mcnemar.loc[(0, 1)]],
                    [contingency_table_mcnemar.loc[(1, 0)], contingency_table_mcnemar.loc[(1, 1)]]
                ])

                # Ensure there are discordant pairs for the test to be meaningful
                if n01 + n10 > 0:
                    mcnemar_result = mcnemar(table_array_mcnemar, exact=True) # Use exact=False for larger samples
                    ensemble_statistical_test_results[comparison_key]['McNemar p'] = mcnemar_result.pvalue
                else:
                    ensemble_statistical_test_results[comparison_key]['McNemar p'] = 1.0 # Perfect agreement on discordant pairs

            else:
                 ensemble_statistical_test_results[comparison_key]['McNemar p'] = np.nan # Length mismatch

        except Exception as e:
             print(f"Error performing McNemar test for {comparison_key}: {e}")
             ensemble_statistical_test_results[comparison_key]['McNemar p'] = np.nan

**Reasoning**:
The previous attempt at McNemar's test produced an error due to improper construction of the contingency table. I will refine the McNemar test by explicitly calculating the counts for all four cells of the 2x2 contingency table using boolean logic on `model_preds` and `y_test`. This direct approach ensures that `statsmodels.stats.contingency_tables.mcnemar` receives a correctly formatted NumPy array of scalar counts, resolving the `TypeError` and allowing the test to run successfully.



In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score
from typing import Dict, List, Any
import os
from scipy.stats import ttest_rel, wilcoxon, mannwhitneyu
from statsmodels.stats.contingency_tables import mcnemar

# Initialize dictionaries to store bootstrap results
metric_distributions: Dict[str, Dict[str, List[float]]] = {
    model_name: {'ROC AUC': [], 'PR AUC': [], 'F1 Score': [], 'bootstrap_samples_for_plotting': []}
    for model_name in models.keys()
}

# Use a fixed random state for reproducibility of bootstrap samples themselves
bootstrap_rng = np.random.default_rng(seed=42)

# Perform bootstrap resampling
print(f"Performing bootstrap resampling ({n_iterations} iterations)...")

for i in range(n_iterations):
    # Resample the test set with replacement
    bootstrap_indices = bootstrap_rng.choice(len(y_test), size=len(y_test), replace=True)
    y_test_sample = y_test[bootstrap_indices]

    for model_name, data in models.items():
        # Get probabilities and binary predictions for the bootstrap sample
        probs_sample = data['probs'][bootstrap_indices]
        preds_sample = data['preds'][bootstrap_indices] # Assuming binary predictions are available

        # Calculate metrics on the bootstrap sample
        # ROC AUC
        # Check for multiple classes in true labels and variation in scores/predictions
        if len(np.unique(y_test_sample)) > 1 and len(np.unique(probs_sample)) > 1:
            roc_auc = roc_auc_score(y_test_sample, probs_sample)
            metric_distributions[model_name]['ROC AUC'].append(roc_auc)
        else:
            # Handle cases where resampling results in only one class or constant probabilities
            metric_distributions[model_name]['ROC AUC'].append(np.nan) # Append NaN for invalid samples

        # PR AUC
        if len(np.unique(y_test_sample)) > 1 and len(np.unique(probs_sample)) > 1:
             pr_auc = average_precision_score(y_test_sample, probs_sample)
             metric_distributions[model_name]['PR AUC'].append(pr_auc)
        else:
             metric_distributions[model_name]['PR AUC'].append(np.nan)


        # F1 Score (requires binary predictions)
        # Check for multiple classes in true labels and variation in predictions
        if len(np.unique(y_test_sample)) > 1 and len(np.unique(preds_sample)) > 1: # F1 is undefined for single class or constant predictions
             f1 = f1_score(y_test_sample, preds_sample)
             metric_distributions[model_name]['F1 Score'].append(f1)
        else:
             metric_distributions[model_name]['F1 Score'].append(np.nan)


        # Store samples for plotting ROC/PR curves with bands
        # Store a fixed number of samples (e.g., up to 100) for plotting efficiency
        if len(metric_distributions[model_name]['bootstrap_samples_for_plotting']) < 100:
             metric_distributions[model_name]['bootstrap_samples_for_plotting'].append((y_test_sample, probs_sample))


print("Bootstrap resampling complete.")

# Calculate summary statistics (mean, CI) from bootstrap distributions
metric_summary: Dict[str, Dict[str, Dict[str, Any]]] = {}
for model_name, metrics in metric_distributions.items():
    metric_summary[model_name] = {}
    for metric_name, distribution in metrics.items():
        if metric_name == 'bootstrap_samples_for_plotting':
            continue # Skip plotting samples

        # Filter out NaN values before calculating mean and CI
        valid_distribution = np.array(distribution)[~np.isnan(distribution)]

        if len(valid_distribution) > 0:
            mean = np.mean(valid_distribution)
            # Calculate confidence interval using percentile method
            # Need at least 2 points for percentiles
            if len(valid_distribution) >= 2:
                lower_ci = np.percentile(valid_distribution, (alpha / 2) * 100)
                upper_ci = np.percentile(valid_distribution, 100 - (alpha / 2) * 100)
            else:
                lower_ci = np.nan
                upper_ci = np.nan

            metric_summary[model_name][metric_name] = {
                'mean': mean,
                'ci_lower': lower_ci,
                'ci_upper': upper_ci,
                'distribution': list(valid_distribution) # Store valid distribution for effect size calculation
            }
        else:
            metric_summary[model_name][metric_name] = {
                'mean': np.nan,
                'ci_lower': np.nan,
                'ci_upper': np.nan,
                'distribution': []
            }

print("\nSummary statistics calculated.")

# Define the ensemble models subset for pairwise comparison among themselves
# Assuming 'models' dictionary is already defined and contains ensemble models
ensemble_models = {
    model_name: data for model_name, data in models.items()
    if 'Ensemble' in model_name or 'Stacked' in model_name
}

# Store statistical test results for ensemble comparisons
ensemble_statistical_test_results: Dict[str, Dict[str, Dict[str, float]]] = {}

print("\nPerforming Statistical Testing Among Ensemble Learners...")

ensemble_model_names = list(ensemble_models.keys())

# Iterate through all unique pairs of ensemble models
for i in range(len(ensemble_model_names)):
    for j in range(i + 1, len(ensemble_model_names)):
        model1_name = ensemble_model_names[i]
        model2_name = ensemble_model_names[j]

        model1_data = ensemble_models[model1_name]
        model2_data = ensemble_models[model2_name]

        # Retrieve bootstrap distributions
        model1_roc_auc_dist = np.array(metric_distributions[model1_name]['ROC AUC'], dtype=np.float64)
        model1_pr_auc_dist = np.array(metric_distributions[model1_name]['PR AUC'], dtype=np.float64)
        model1_f1_dist = np.array(metric_distributions[model1_name]['F1 Score'], dtype=np.float64)

        model2_roc_auc_dist = np.array(metric_distributions[model2_name]['ROC AUC'], dtype=np.float64)
        model2_pr_auc_dist = np.array(metric_distributions[model2_name]['PR AUC'], dtype=np.float64)
        model2_f1_dist = np.array(metric_distributions[model2_name]['F1 Score'], dtype=np.float64)

        # Retrieve original binary predictions for McNemar test
        model1_preds = model1_data['preds']
        model2_preds = model2_data['preds']


        comparison_key = f'{model1_name} vs {model2_name}'
        ensemble_statistical_test_results[comparison_key] = {}

        # Perform paired tests on metric distributions (ROC AUC, PR AUC, F1 Score)
        for metric_name, dist1, dist2 in zip(
            ['ROC AUC', 'PR AUC', 'F1 Score'],
            [model1_roc_auc_dist, model1_pr_auc_dist, model1_f1_dist],
            [model2_roc_auc_dist, model2_pr_auc_dist, model2_f1_dist]
        ):
             min_len = min(len(dist1), len(dist2))
             if min_len >= 2:
                 # Paired t-test (handles NaNs)
                 t_stat, p_value_t = ttest_rel(dist1[:min_len], dist2[:min_len], nan_policy='omit')
                 ensemble_statistical_test_results[comparison_key][f'{metric_name} (T-test p)'] = p_value_t

                 # Wilcoxon test (filter NaNs, require variation)
                 dist1_valid = dist1[:min_len][~np.isnan(dist1[:min_len])]
                 dist2_valid = dist2[:min_len][~np.isnan(dist2[:min_len])]
                 if len(dist1_valid) >= 2 and len(dist2_valid) >= 2 and len(set(dist1_valid)) > 1 and len(set(dist2_valid)) > 1:
                     try:
                         w_stat, p_value_w = wilcoxon(dist1_valid, dist2_valid)
                         ensemble_statistical_test_results[comparison_key][f'{metric_name} (Wilcoxon p)'] = p_value_w
                     except ValueError:
                         ensemble_statistical_test_results[comparison_key][f'{metric_name} (Wilcoxon p)'] = np.nan
                 else:
                     ensemble_statistical_test_results[comparison_key][f'{metric_name} (Wilcoxon p)'] = np.nan

                 # Mann-Whitney U test (filter NaNs, require sufficient data)
                 if len(dist1_valid) >= 2 and len(dist2_valid) >= 2:
                     u_stat, p_value_u = mannwhitneyu(dist1_valid, dist2_valid)
                     ensemble_statistical_test_results[comparison_key][f'{metric_name} (Mann-Whitney p)'] = p_value_u
                 else:
                      ensemble_statistical_test_results[comparison_key][f'{metric_name} (Mann-Whitney p)'] = np.nan

             else:
                 ensemble_statistical_test_results[comparison_key][f'{metric_name} (T-test p)'] = np.nan
                 ensemble_statistical_test_results[comparison_key][f'{metric_name} (Wilcoxon p)'] = np.nan
                 ensemble_statistical_test_results[comparison_key][f'{metric_name} (Mann-Whitney p)'] = np.nan


        # McNemar test on binary predictions (requires original predictions)
        try:
            # Ensure both prediction arrays have the same length as y_test
            if len(model1_preds) == len(y_test) and len(model2_preds) == len(y_test):
                correct_m1 = (model1_preds == y_test)
                correct_m2 = (model2_preds == y_test)

                n_both_correct = np.sum(correct_m1 & correct_m2)
                n_m1_correct_m2_incorrect = np.sum(correct_m1 & ~correct_m2)
                n_m1_incorrect_m2_correct = np.sum(~correct_m1 & correct_m2)
                n_both_incorrect = np.sum(~correct_m1 & ~correct_m2)

                mcnemar_table = np.array([
                    [n_both_correct, n_m1_correct_m2_incorrect],
                    [n_m1_incorrect_m2_correct, n_both_incorrect]
                ])

                # Ensure there are discordant pairs for the test to be meaningful
                if n_m1_correct_m2_incorrect + n_m1_incorrect_m2_correct > 0:
                    mcnemar_result = mcnemar(mcnemar_table, exact=True)
                    ensemble_statistical_test_results[comparison_key]['McNemar p'] = mcnemar_result.pvalue
                else:
                    ensemble_statistical_test_results[comparison_key]['McNemar p'] = 1.0 # Perfect agreement on discordant pairs

            else:
                 ensemble_statistical_test_results[comparison_key]['McNemar p'] = np.nan # Length mismatch

        except Exception as e:
             print(f"Error performing McNemar test for {comparison_key}: {e}")
             ensemble_statistical_test_results[comparison_key]['McNemar p'] = np.nan

**Reasoning**:
I need to apply Holm-Bonferroni correction to all generated p-values from the ensemble statistical tests. This requires extracting all p-values, applying the correction using `statsmodels.stats.multitest.multipletests`, and then updating the `ensemble_statistical_test_results` dictionary with these corrected values. Finally, I will display the updated comparison table with corrected p-values and significance flags, and save it in CSV and LaTeX formats.



In [ ]:
import numpy as np
import pandas as pd
from statsmodels.stats.multitest import multipletests

# --- Step 1: Extract all p-values for correction ---
all_p_values = []
p_value_keys = []

for comparison_key, tests in ensemble_statistical_test_results.items():
    for test_name, p_value in tests.items():
        if p_value is not None and not np.isnan(p_value):
            all_p_values.append(p_value)
            p_value_keys.append((comparison_key, test_name))

# --- Step 2: Apply Holm-Bonferroni correction ---
if all_p_values:
    corrected_p_values_tuple = multipletests(all_p_values, alpha=alpha, method='holm')
    corrected_p_values = corrected_p_values_tuple[1] # p-value adjusted
    corrected_significant = corrected_p_values_tuple[0] # reject_null
else:
    corrected_p_values = []
    corrected_significant = []

# --- Step 3: Update ensemble_statistical_test_results with corrected p-values ---
corrected_p_value_map = {}
for i, (comparison_key, test_name) in enumerate(p_value_keys):
    corrected_p_value_map[(comparison_key, test_name)] = {
        'p_value_corrected': corrected_p_values[i],
        'significant_corrected': corrected_significant[i]
    }

# Recreate the comparison data with corrected p-values
print("\nStatistical Test Results Among Ensemble Learners (Holm-Bonferroni Corrected):")

ensemble_comparison_data_corrected = []
# Define the order of tests for consistent columns
test_keys_order = [
    'ROC AUC (T-test p)', 'ROC AUC (Wilcoxon p)', 'ROC AUC (Mann-Whitney p)',
    'PR AUC (T-test p)', 'PR AUC (Wilcoxon p)', 'PR AUC (Mann-Whitney p)',
    'F1 Score (T-test p)', 'F1 Score (Wilcoxon p)', 'F1 Score (Mann-Whitney p)',
    'McNemar p'
]

for comparison, tests in ensemble_statistical_test_results.items():
    row = {'Comparison': comparison}
    for test_name_key in test_keys_order:
        # Get original p-value
        original_p_value = tests.get(test_name_key)

        # Get corrected p-value and significance
        corrected_info = corrected_p_value_map.get((comparison, test_name_key))

        if corrected_info:
            p_value_corrected = corrected_info['p_value_corrected']
            significant_corrected = corrected_info['significant_corrected']
            row[test_name_key] = f"{p_value_corrected:.4f}"
            row[f'Significant? ({test_name_key})'] = '*' if significant_corrected else ''
        elif original_p_value is not None and not np.isnan(original_p_value):
            row[test_name_key] = "N/A" # No corrected info, but original existed
            row[f'Significant? ({test_name_key})'] = ''
        else:
            row[test_name_key] = "N/A" # Original p-value was also N/A or NaN
            row[f'Significant? ({test_name_key})'] = ''

    ensemble_comparison_data_corrected.append(row)

ensemble_comparison_df_corrected = pd.DataFrame(ensemble_comparison_data_corrected)

# Filter columns to only include those that were actually added
valid_ensemble_columns_corrected = ['Comparison']
for test_name_key in test_keys_order:
    if test_name_key in ensemble_comparison_df_corrected.columns:
        valid_ensemble_columns_corrected.append(test_name_key)
        valid_ensemble_columns_corrected.append(f'Significant? ({test_name_key})')

ensemble_comparison_df_corrected = ensemble_comparison_df_corrected[valid_ensemble_columns_corrected]

# Display the table
print(ensemble_comparison_df_corrected.to_string())

# Save the detailed ensemble performance summary table
output_dir_tables = "/content/drive/MyDrive/Final_Implementations/DynaBiome_Tables/"
os.makedirs(output_dir_tables, exist_ok=True)
ensemble_comparison_df_corrected.to_csv(f"{output_dir_tables}/ensemble_statistical_comparison_corrected.csv", index=False)

# Prepare for LaTeX: Escape special characters
latex_ensemble_df_corrected = ensemble_comparison_df_corrected.copy()
latex_ensemble_df_corrected.columns = latex_ensemble_df_corrected.columns.str.replace('_', '\\_')
latex_ensemble_df_corrected.columns = latex_ensemble_df_corrected.columns.str.replace('%', '\\%')
latex_ensemble_df_corrected.columns = latex_ensemble_df_corrected.columns.str.replace('(', '{(', regex=False).str.replace(')', ')}')

latex_ensemble_df_corrected.to_latex(f"{output_dir_tables}/ensemble_statistical_comparison_corrected.tex", index=False, float_format="%.4f", escape=False)

print(f"\nEnsemble statistical comparison corrected table saved to {output_dir_tables}/ensemble_statistical_comparison_corrected.csv and .tex")